# Test Bed


In [11]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
module_path = os.path.abspath(os.path.join('../..'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
import numpy as np

from src.spectra.gen_spectra import gen_spectrum
from src.objects import Spectrum
from src.utils import insort_by_index, make_sparse_array, ppm_to_da
from src.scoring import scoring

In [9]:
DEAPNFEANTTIGRIRFH = Spectrum(spectrum=[70.06517028808594, 74.06005859375, 84.04444122314453, 87.05548095703125, 88.03839111328125, 102.05506896972656, 120.08072662353516, 201.12315368652344, 212.1022186279297, 217.08192443847656, 245.07611083984375, 316.1118469238281, 585.966796875, 591.6425170898438, 591.9791870117188, 592.3104248046875, 607.8389282226562, 615.3208618164062, 615.6561279296875, 643.3582763671875, 643.8550415039062, 707.87841796875, 708.3804931640625, 781.4142456054688, 838.4376220703125], abundance=[2547.683349609375, 730.1649169921875, 1565.4937744140625, 958.437744140625, 917.452392578125, 3846.709716796875, 2452.79736328125, 987.6354370117188, 1954.6824951171875, 3886.33349609375, 6083.39990234375, 712.1141357421875, 843.4388427734375, 3621.18359375, 3374.97021484375, 1484.1064453125, 1291.126953125, 968.104248046875, 1789.317138671875, 2742.34521484375, 1139.092041015625, 950.854736328125, 762.1254272460938, 748.9049072265625, 758.2952880859375], total_intensity=47116.769775390625, ms_level=2, scan_number=17, precursor_mass=1045.526854, file_name='/Users/zacharymcgrath/Desktop/nod2 data/single/singleRealSpectrum.mzml', id='NOD2_E3.9670.9702.4.pkl')
DEAPNFEANTTIGRIRFHs = 'DEAPNFEANTTIGRIRFH'

DLQTLEVE = Spectrum(spectrum=[72.08074951171875, 84.04391479492188, 86.0966796875, 101.07064056396484, 102.05500793457031, 130.04966735839844, 132.10191345214844, 148.06016540527344, 183.11273193359375, 197.12803649902344, 201.1233673095703, 212.10264587402344, 215.13906860351562, 229.11734008789062, 243.1343536376953, 247.12913513183594, 357.1775207519531, 358.16217041015625, 440.2137756347656, 458.22381591796875, 472.23712158203125, 553.2945556640625, 571.3069458007812, 682.3330688476562, 700.3469848632812], abundance=[11277.177734375, 2677.902099609375, 3753.010498046875, 1874.318359375, 3304.14306640625, 4072.84765625, 1058.6568603515625, 18233.23046875, 1099.9212646484375, 891.7163696289062, 4566.39990234375, 767.3123779296875, 1045.271728515625, 1623.881103515625, 856.7132568359375, 3777.787109375, 1373.5948486328125, 1266.3497314453125, 1810.636962890625, 2190.313232421875, 787.8353881835938, 2011.8814697265625, 1665.766357421875, 1007.9365234375, 2019.712646484375], total_intensity=75014.31701660156, ms_level=2, scan_number=10, precursor_mass=473.740315, file_name='/Users/zacharymcgrath/Desktop/nod2 data/single/singleRealSpectrum.mzml', id='NOD2_E3.11427.11516.2.pkl')
DLQTLEVEs = 'DLQTLEVE'

DLQTLALWSRM = Spectrum(spectrum=[84.04356384277344, 86.095703125, 101.07088470458984, 201.12228393554688, 202.1270751953125, 212.10423278808594, 229.1165008544922, 246.14280700683594, 306.16021728515625, 357.17578125, 393.1902770996094, 430.2205505371094, 440.2127685546875, 458.2236022949219, 553.2975463867188, 579.2705078125, 624.333251953125, 642.3447875976562, 658.3429565429688, 692.3502197265625, 763.3924560546875, 764.3958129882812, 876.477783203125, 977.5200805664062, 978.5184326171875], abundance=[907.1680908203125, 2089.55615234375, 1746.5712890625, 1797.59326171875, 517.5567626953125, 576.9080200195312, 1723.9498291015625, 829.689208984375, 973.6568603515625, 1102.828857421875, 1670.658203125, 730.5213623046875, 950.6202392578125, 1065.868896484375, 1333.5362548828125, 1928.1298828125, 525.9630126953125, 818.0933227539062, 606.9487915039062, 1494.930908203125, 3529.6337890625, 817.701171875, 911.8171997070312, 1712.318603515625, 685.1273193359375], total_intensity=31047.347290039062, ms_level=2, scan_number=10129, precursor_mass=667.350088, file_name='/Users/zacharymcgrath/Desktop/nod2 data/single/singleRealSpectrum.mzml', id='scan=10130')
DLQTLALWSRMs = 'DLQTLALWSRM'

DLTEYLSRFGEVV = Spectrum(spectrum=[70.06398010253906, 72.08049774169922, 86.09703063964844, 118.08600616455078, 136.07492065429688, 187.0723419189453, 201.12191772460938, 217.15528869628906, 229.1175994873047, 230.12203979492188, 312.15435791015625, 330.1663818359375, 334.13720703125, 441.199951171875, 459.2086486816406, 647.3054809570312, 647.8048095703125, 656.810791015625, 746.8765869140625, 793.4156494140625, 906.4981079101562, 978.4926147460938, 1069.56689453125, 1070.5753173828125, 1311.6202392578125], abundance=[855.636474609375, 3774.81884765625, 812.15185546875, 533.6064453125, 490.1760559082031, 414.03570556640625, 1025.067138671875, 792.000244140625, 1133.7769775390625, 346.464599609375, 612.9151000976562, 488.3008117675781, 484.1494140625, 353.00909423828125, 504.1302185058594, 382.40264892578125, 361.7584228515625, 359.9140319824219, 428.7555847167969, 540.8904418945312, 452.9610290527344, 643.1677856445312, 458.5113525390625, 399.8130187988281, 694.0618896484375], total_intensity=17342.475189208984, ms_level=2, scan_number=11409, precursor_mass=764.388677, file_name='/Users/zacharymcgrath/Desktop/nod2 data/single/singleRealSpectrum.mzml', id='scan=11410')
DLTEYLSRFGEVVs = 'DLTEYLSRFGEVV'

DLQTLEVE = Spectrum(spectrum=[72.08074951171875, 84.04391479492188, 86.0966796875, 101.07064056396484, 102.05500793457031, 130.04966735839844, 132.10191345214844, 148.06016540527344, 183.11273193359375, 197.12803649902344, 201.1233673095703, 212.10264587402344, 215.13906860351562, 229.11734008789062, 243.1343536376953, 247.12913513183594, 357.1775207519531, 358.16217041015625, 440.2137756347656, 458.22381591796875, 472.23712158203125, 553.2945556640625, 571.3069458007812, 682.3330688476562, 700.3469848632812], abundance=[11277.177734375, 2677.902099609375, 3753.010498046875, 1874.318359375, 3304.14306640625, 4072.84765625, 1058.6568603515625, 18233.23046875, 1099.9212646484375, 891.7163696289062, 4566.39990234375, 767.3123779296875, 1045.271728515625, 1623.881103515625, 856.7132568359375, 3777.787109375, 1373.5948486328125, 1266.3497314453125, 1810.636962890625, 2190.313232421875, 787.8353881835938, 2011.8814697265625, 1665.766357421875, 1007.9365234375, 2019.712646484375], total_intensity=75014.31701660156, ms_level=2, scan_number=10, precursor_mass=473.740315, file_name='/Users/zacharymcgrath/Desktop/nod2 data/single/singleRealSpectrum.mzml', id='NOD2_E3.11427.11516.2.pkl')
DLQTLEVEs = 'DLQTLEVE'

bigboi = Spectrum(spectrum=[60.04412841796875, 72.08085632324219, 86.09625244140625, 87.05484771728516, 87.10027313232422, 88.03878784179688, 102.0547103881836, 104.05284881591797, 120.08000946044922, 133.06097412109375, 147.0771942138672, 156.1029510498047, 157.06239318847656, 157.1354522705078, 159.0472412109375, 159.0762481689453, 175.0726776123047, 183.1116943359375, 201.12351989746094, 202.12612915039062, 228.09860229492188, 229.1192169189453, 317.1471252441406, 505.0594177246094, 525.93994140625, 526.434814453125, 595.0115356445312, 595.3402099609375, 638.69482421875, 639.0325927734375, 676.3802490234375, 700.4046630859375, 729.0729370117188, 729.4135131835938, 729.7415161132812, 752.7596435546875, 753.100341796875, 753.4302368164062, 767.0960083007812, 878.6868896484375, 957.5401611328125], abundance=[873.4708251953125, 303.00042724609375, 3144.203125, 800.2400512695312, 396.7620849609375, 453.8421630859375, 344.6172180175781, 429.3058166503906, 320.4771728515625, 1560.115966796875, 471.9178466796875, 413.0157470703125, 404.1276550292969, 335.3738098144531, 445.3636474609375, 309.615234375, 553.0883178710938, 328.8626708984375, 4163.8330078125, 549.1915283203125, 384.2148742675781, 1420.31298828125, 328.160400390625, 406.8067321777344, 382.0265197753906, 351.4310607910156, 606.8508911132812, 327.9308166503906, 444.44659423828125, 696.0427856445312, 367.32196044921875, 340.5555419921875, 482.5303649902344, 534.2802734375, 304.5690002441406, 593.4415893554688, 743.320068359375, 319.4105224609375, 442.2250061035156, 393.402587890625, 421.89605712890625], total_intensity=26891.600952148438, ms_level=2, scan_number=81, precursor_mass=1331.2377625, file_name='/Users/zacharymcgrath/Desktop/nod2 data/single/singleRealSpectrum.mzml', id='NOD2_E3.7258.7258.5.pkl')
bigbois = 'DLSSASAIMGNAKVKAHGKKVITAFN'

ppm_tol = 20

# See how well each of these score on each scoring algorithm
* `score_subsequence(pepspec: list, subseq: str, ppm_tolerance=20)`
* `backbone_score(observed: Spectrum, reference: str, ppm_tolerance: int)` 
* `intensity_backbone_score(observed: Spectrum, reference: str, ppm_tolerance: int)`
* `ion_backbone_score(observed: Spectrum, reference: str, ion: str, ppm_tolerance: int)`
* `intensity_ion_backbone_score(observed: Spectrum, reference: str, ion: str, ppm_tolerance: int)`
* `xcorr(observed: np.ndarray, reference: np.ndarray, value=50)`

## score_subsequence

In [3]:
print(f'DEAPNFEANTTIGRIRFH: {scoring.score_subsequence(DEAPNFEANTTIGRIRFH.spectrum, DEAPNFEANTTIGRIRFHs, ppm_tol)}')
print(f'DLQTLEVE: {scoring.score_subsequence(DLQTLEVE.spectrum, DLQTLEVEs, ppm_tol)}')
print(f'DLQTLALWSRM: {scoring.score_subsequence(DLQTLALWSRM.spectrum, DLQTLALWSRMs, ppm_tol)}')
print(f'DLTEYLSRFGEVV: {scoring.score_subsequence(DLTEYLSRFGEVV.spectrum, DLTEYLSRFGEVVs, ppm_tol)}')
print(f'DLQTLEVE: {scoring.score_subsequence(DLQTLEVE.spectrum, DLQTLEVEs, ppm_tol)}')

DEAPNFEANTTIGRIRFH: (2, 5)
DLQTLEVE: (5, 2)
DLQTLALWSRM: (5, 8)
DLTEYLSRFGEVV: (5, 5)
DLQTLEVE: (5, 2)


In [7]:
s = sorted(gen_spectrum(bigbois)['spectrum'])
sum([1 for m in s for x in bigboi.spectrum if m - .05 <= x <= m +.05])

12

In [20]:
print(gen_spectrum('DLSS', charge=1, ion='b')['spectrum'][-1])
print(gen_spectrum('DLSS', charge=2, ion='b')['spectrum'][-1])
print(gen_spectrum('FN', charge=1, ion='y')['spectrum'][-1])
print(gen_spectrum('FN', charge=2, ion='y')['spectrum'][-1])

403.182339435
202.094807935
280.129182135
140.568229285


In [15]:
ppm_to_da(202.12612915039062, 200)

0.040425225830078126

In [16]:
202.12612915039062 - 0.04

202.08612915039063

In [1]:
s = ['GLVASNL', 'CLKVRG', 'VASDAKS', 'DAKSFV', 'HFNPR', 'IVCNTK', 'TEHRE', 'EHREP', 'ITEVCI', 'KLPDGH', 'LPDGHE', 'EFKFP', 'MNPAVF', 'SLADLR', 'LLLLVT', 'VTSIFT', 'SIFTPI', 'EILNNA', 'RFSQM', 'NQVVFA', 'ARVDCD', 'QHSDIA', 'HSDIAQ', 'YRISK', 'RISKY', 'LKLFR', 'KLFRN', 'GQRSVK', 'RSVKAL', 'IRQQK', 'QKSNPV', 'SENYR', 'ERVASI', 'LHDDCA', 'FGDLSK', 'PERYN', 'IYKPPG', 'RSAPDM', 'MVYLGS', 'VYLGSM', 'YLGSMT', 'NWIQD', 'DKCVPL', 'GEELTE', 'ELTEEG', 'EGLPFL', 'LFHMK', 'FHMKD', 'LHADCD', 'CDKFR', 'FRHPL', 'SFRHM', 'PGKLKQ', 'KQFVF', 'FDLHSG', 'PGEQDQ', 'QDQDVA', 'DVASSPP', 'ASSPPES', 'LLRDR', 'LRDRD', 'MASRVL', 'RLPAAF', 'APLPRL', 'RLPTLA', 'RPLSTT', 'CPEGIR', 'RRPGAL', 'ALQSALA', 'QSALALA', 'SALALAQ', 'AQVPGTV', 'TVTHLC', 'RQYSD', 'KLYDK', 'PEKLSV', 'HFMKD', 'MKDLGL', 'DSLDQV', 'VEIIMA', 'MEDEF', 'GFEIPD', 'ADKKDV', 'HPFGSQ', 'QQVVPC', 'QVVPCQ', 'PEQVCC', 'AAGCKVE', 'AGCKVEA', 'VQGEEL', 'QGEELC', 'LCRQR', 'RVLRM', 'MAYSEA', 'EAGDYL', 'LVAIEE', 'FLRAY', 'SDNSRV', 'PLCFSC', 'LCFSCC', 'GDLLVGC', 'CTNKLV', 'FTLKY', 'IDNITP', 'NITPVE', 'ITPVEI', 'SFCVGY', 'YVAVMS', 'SDLEVL', 'LESDPI', 'DPIHGE', 'PQETSN', 'QETSNP', 'NPLKEA', 'VICLKP', 'IGEKCE', 'VKLEST', 'EKVKY', 'SLQLLP', 'QSGFHP', 'ENDLSP', 'NDLSPK', 'LFCFF', 'VKSVEL', 'VELMSV', 'LHVITS', 'RCSAAVA', 'EEDLY', 'EDLYM', 'DVCALR', 'VCALRI', 'RIQLF', 'LLTKAE', 'TKAEPE', 'IPERR', 'ERRES', 'RKDASV', 'KDASVR', 'ASVRSGT', 'SGTPHVA', 'YKEMV', 'HLLSEA', 'LLSEAH', 'LSEAHL', 'SEAHLL', 'RAALMD', 'FKESCG', 'QSHLAL', 'SGLSLAE', 'VLARVD', 'RVDWT', 'QIFHM', 'NIDPLT', 'CGVSPVL', 'GVSPVLV', 'LVTLTK', 'LTKAAVA', 'TKAAVAL', 'LKMGDL', 'KMGDLD', 'RNEMK', 'GFILEP', 'IVPTEL', 'ELAIDL', 'LAIDLK', 'LGLQKN', 'GLQKND', 'KNDKIG', 'FKVLCG', 'KDEDAV', 'DFWEA', 'PNVVLE', 'LFFKL', 'FFKLI', 'AEDLIN', 'EDLINA', 'SHYGLV', 'VHVLTT', 'LQSLIC', 'SEDTVAG', 'EYEQC', 'CIDTLL', 'RCPEAV', 'QRVKSG', 'KETLSV', 'VGLDLR', 'LPEDGAA', 'VNPTVF', 'EPLGRV', 'FADKVP', 'IIPGFM', 'IPGFMC', 'DFTRH', 'FTRHN', 'EKFED', 'KFEDE', 'ILKHTG', 'GPGILSM', 'MANAGPN', 'KTEWL', 'LDGKHV', 'VFGKVK', 'VKEGMN', 'MNIVEA', 'NIVEAM', 'TSKKIT', 'KITISD', 'LSRALL', 'CLALAW', 'AWAARV', 'VGADALE', 'DNVLVL', 'HKYLL', 'LVEFY', 'APWCGH', 'AKAAAKL', 'AAAKLKA', 'GSEIRL', 'GDTASPK', 'KEYTAG', 'RTGPAAT', 'EAIDDI', 'ITSNSGV', 'FSKYQ', 'KYQLD', 'YQLDK', 'QLDKDG', 'KKFDE', 'EGEITK', 'GEITKE', 'EQTAPK', 'TAPKIF', 'RILEF', 'ILEFF', 'KEECPA', 'ECPAVR', 'PAVRLI', 'AVRLIT', 'EMTKY', 'MTKYK', 'DELTAE', 'LTAEKI', 'CHRFL', 'EGKIKP', 'WDKQP', 'FDEKK', 'CGHCKQ', 'KQLAPI', 'IWDKL', 'ENIIIA', 'NIIIAK', 'EVEAVK', 'ADRTVI', 'GGQDGAGD', 'QDGAGDD', 'DGAGDDE', 'GAGDDED', 'EALEPD', 'ALEPDM', 'DDQKAV', 'LSDDLI', 'SDDLIN', 'TVLGGPK', 'PKLPGR', 'KLPGRV', 'IRDQGS', 'GSCWAF', 'SCWAFG', 'GRVNVE', 'DLLTCC', 'GCNGGYP', 'SFWTK', 'GVYNSH', 'YNSHVG', 'HVNGSR', 'PCTGEGD', 'TGEGDTP', 'EGDTPR', 'GYSPSY', 'EDKHF', 'GYTSYS', 'EIYKN', 'YKNGPV', 'GPVEGAF', 'AFTVFS', 'FLTYK', 'LTYKSG', 'VYKHE', 'GHAIRI', 'GVENGVP', 'ENGVPY', 'VPYWL', 'FFKIL', 'FKILR', 'ENHCGI', 'NHCGIE', 'DQYWG', 'DVELTV', 'VIGARR', 'RRASW', 'RASWR', 'SWRII', 'WRIIS', 'ISSIEQ', 'KMIRE', 'CDILDV', 'GESKVF', 'NSLVAY', 'KAASDIA', 'AASDIAM', 'IAMTEL', 'PIRLGL', 'SVFYY', 'EILNSP', 'ELDTLS', 'LDTLSE', 'KDSTLI', 'EALQDV', 'ALQDVE', 'RGRPGK', 'GRPGKR', 'GKRRR', 'RRRVS', 'RPACVK', 'VKLGAGAG', 'ASSRRP', 'GAPDGME', 'PDGMEE', 'DGMEEP', 'EEPGMD', 'PARPLN', 'ARPLNC', 'PLNCVE', 'PAPAQPP', 'AQASVSN', 'NGEDAGGG', 'AGGGVGKE', 'LKIIW', 'KIIWN', 'IIWNK', 'WNKTK', 'DVKVPL', 'VKVPLD', 'PLDSTGS', 'ITGLPPA', 'PPAMQK', 'PAMQKV', 'QKVMY', 'KVMYK', 'YKGLVP', 'EIKVTS', 'IKVTSGA', 'VTSGAKI', 'TINDVL', 'VNTPKD', 'QHRKV', 'DKGKPE', 'KGKPED', 'EDVMPS', 'GAQERL', 'PTVPLSG', 'PLSGMY', 'GKVRLT', 'DQLWI', 'EKLPMG', 'KNVVSE', 'VSEPIE', 'LGPTEAS', 'PTEASY', 'CTLLLL', 'AALAPTQ', 'APTQTR', 'PTQTRA', 'GPHSLR', 'HSLRY', 'VGYVDD', 'GYVDDT', 'ENPRY', 'NPRYE', 'LRTLLG', 'YYNQS', 'NQSKGGS', 'SKGGSHT', 'GRLLRG', 'YQQYA', 'QQYAY', 'IALNED', 'ALNEDL', 'KTWTAA', 'GEAERL', 'ATLLRT', 'ALGFYP', 'LGFYPA', 'GFYPAD', 'LNGEEL', 'NGEELI', 'VETRPA', 'TRPAGDG', 'RPAGDGT', 'SVVVPLG', 'PLGKEQ', 'QYYTC', 'LPEPLT', 'PEPLTL', 'ATVAVLV', 'AVLVVLG', 'VLVVLGA', 'GKGGDYA', 'LAPGSQT', 'LPDCKV', 'QSINIT', 'SINITE', 'FLSTSI', 'IAQLKV', 'EAKDCL', 'DCLNVL', 'CLNVLN', 'NVLNKS', 'VLNKSN', 'YVPGKL', 'PGKLHD', 'GKLHDV', 'LIDVGTG', 'VEKTAE', 'KIQPAL', 'IQPALQ', 'IQQLTA', 'QQLTAL', 'QLTALGA', 'NKVLAK', 'KVLAKL', 'LAKLGVA', 'KLGVAGQ', 'GLEEET', 'EEETLG', 'SPACALL', 'FPLTAQ', 'PLTAQH', 'QHENF', 'RKKQI', 'LKGQEV', 'MKQTIG', 'CGTIGLI', 'TEKLSP', 'SPEDRA', 'AAHDSVA', 'GQCRVD', 'NFHFI', 'FHFIL', 'HFILF', 'HLYEL', 'FPVNHG', 'VNHGASS', 'AVALCKA', 'GYTHLS', 'TGDLLR', 'LRAEVS', 'EKGELV', 'LDTVLD', 'RDAMLA', 'MLAKVD', 'LIDGYP', 'QKIGQP', 'KRGETS', 'GRVDDN', 'KKRLE', 'GIVRKV', 'VDTVFS', 'GSQWKA', 'CVMVFA', 'EARQVA', 'RQVAAE', 'HGFGVR', 'GVRKLP', 'YHNGLA', 'RRSLH', 'KRQLE', 'ERDPR', 'RGYRD', 'DGTPGLD', 'GTPGLDL', 'WELGY', 'TGKGVTI', 'NYNADA', 'RYTDD', 'DWFNS', 'NSHGTR', 'AGEVSAAA', 'ASNNICG', 'NNICGVG', 'CGVGVAY', 'SISHMP', 'YSASWG', 'GKTVDGP', 'VDGPRE', 'GPRELT', 'ELTLQA', 'LTLQAM', 'AMADGVN', 'RGGKGSI', 'KGSIYV', 'SIYVW', 'GDGGSYD', 'CNCDGY', 'INDGRT', 'SCSSTLA', 'CSSTLAS', 'SSTLAST', 'RKRNP', 'GVATTDL', 'TTDLYG', 'LYGNCT', 'YGNCTL', 'TLRHSG', 'LRHSGT', 'LEANLD', 'EANLDL', 'RNQLH', 'FNHLF', 'VLDAGAM', 'LDAGAMV', 'KDWKT', 'SVQNPE', 'VLTLKT', 'HVQAVI', 'NATRRG', 'ATRRGD', 'TKSILL', 'SRRPR', 'DDDSKV', 'TWGEDA', 'LELGFV', 'LKEWT', 'KEWTL', 'TLMLHG', 'LMLHGT', 'RDYQS', 'QSKLAM', 'KLAMSK', 'LAMSKK', 'ELDEAV', 'LDEAVE', 'AVERSL', 'MEESSS', 'IAMLVQ', 'VQEIGE', 'QEIGEP', 'EIGEPE', 'TAEGVLS', 'AEGVLSP', 'DQQRK', 'PLVHVI', 'LLIGKK', 'QASSTLA', 'TLASNDG', 'CHATSR', 'LEAHVV', 'TMVDIP', 'MVDIPV', 'AVQSAAM', 'VQSAAMA', 'AESGRR', 'YAYEQ', 'AYEQY', 'QQRML', 'QRMLK', 'AWKHAG', 'YPIFE', 'PLGLLAS', 'SMSAAPGG', 'SAAPGGVD', 'GVDAVVI', 'VIAIGDS', 'SVQVQI', 'QVQICS', 'SSDPPSS', 'SDPPSSS', 'PLEQST', 'TEEGVH', 'EGVHLN', 'GVHLNQ', 'EVMSDS', 'VMSDSE', 'QKIISS', 'VNVIVE', 'ERLPSA', 'RLPSAE', 'SPSPSEA', 'TGCVGAGN', 'HADKCT', 'TVDIGGL', 'SAEKKD', 'AEKKDS', 'SLAAEAL', 'AAEALVT', 'LVTMPI', 'IRAAEL', 'ELTRAS', 'LLDPDT', 'PDTGQR', 'GQRQVS', 'QRQVSG', 'QVSGPLA', 'LKNSTD', 'SNSTVVA', 'EKLRE', 'LRERT', 'RERTD', 'SDDDIL', 'QDNAQC', 'DNAQCQ', 'QCQPNS', 'RLCNY', 'HIAENS', 'QLRNH', 'LRNHE', 'REQHC', 'ASNEPR', 'QEGNKP', 'NKPSTQ', 'TQKQY', 'TSTTYV', 'KPYRC', 'PSLKSH', 'MWKHA', 'HASDQN', 'QVNKAI', 'NKAIND', 'QSARVL', 'KSRGKP', 'SEERTG', 'ASELEK', 'TGCSSDV', 'PGTEYC', 'GTEYCV', 'CICGFE', 'SKESLL', 'MKEHE', 'EIVSII', 'FASFPT', 'DVSHGSA', 'DALASAAG', 'SAAGHLD', 'GHLDDL', 'DLPGALS', 'ALSALSD', 'SHCLLV', 'FTPAVH', 'HASLDK', 'LASVSTV', 'ASVSTVL', 'LDHGVQ', 'GPVAGTGD', 'DFGPLK', 'KEATATG', 'FISCAQ', 'LGRINV', 'MVKEAP', 'EKLKGT', 'VFDTEG', 'LMTQAD', 'RFSEE', 'FPPDVC', 'YRNLC', 'RNLCY', 'SPILPF', 'LSLLLL', 'SLLLLL', 'PRAATAS', 'ATASLSQ', 'TASLSQA', 'TCKAHD', 'CKAHDV', 'KAHDVC', 'PRPLPP', 'RPLPPS', 'PVRVSL', 'SLYYE', 'LYYES', 'YYESL', 'YESLCG', 'CGACRY', 'GNAQER', 'VSGTWE', 'TCQHGE', 'LNMVEA', 'EKKLGP', 'VYAPEV', 'IMECAT', 'ENAQLT', 'LTDALH', 'ALHPPH', 'DPSELL', 'ELLSIV', 'SSIADSP', 'ADSPRK', 'RKVCY', 'IKLQSS', 'KLQSSD', 'SDGEIF', 'DVEIAK', 'EIAKQS', 'KQSVTI', 'QSVTIK', 'MLEDLG', 'LEDLGM', 'AILKKV', 'KVIQW', 'QWCTH', 'KDDPPP', 'PPPEDD', 'NKEKR', 'VWDQE', 'VANMIK', 'MIKGKT', 'EEIRK', 'RKENQ', 'QQGAGLT', 'QGAGLTM', 'GLTMAEG', 'LTMAEGG', 'EEFCF', 'TAIYIS', 'GQWRE', 'VCTDLQ', 'RLEPGT', 'PGTMAPT', 'TMAPTR', 'KKCLGE', 'LICTLN', 'VPAGTEV', 'PAGTEVV', 'AGTEVVC', 'GTEVVCA', 'PKIAVAA', 'AVAAQNC', 'AAQNCY', 'TNGAFTG', 'GMIKDL', 'MIKDLG', 'TWVVLG', 'HVFGES', 'DELIGQ', 'QKVSHA', 'IGEKLD', 'DEREAG', 'EREAGI', 'TGKTATP', 'TPQQAQ', 'LKSNVN', 'KSNVND', 'DGVAQST', 'AQSTRI', 'KELASQ', 'DIINAK', 'FQLFD', 'LYSQCG', 'YSQCGD', 'MRALGQ', 'NAEVLK', 'VAKNKD', 'VEGLRV', 'MGAEIR', 'FVRHI', 'PAAPAGEG', 'QTQAQV', 'VDEVVD', 'VNVDKV', 'ERDQK', 'RADALQ', 'LQAGASQ', 'KLKRK', 'MMIILG', 'YINRT', 'INRTVA', 'SDGRMI', 'RMIVGT', 'VLGLYI', 'LGLYIV', 'IVRGDN', 'IGEIDE', 'EDELLG', 'DELLGK', 'ELLGKL', 'DLDPES', 'ATLPAGF', 'PAGFRQ', 'RQKDQ', 'TQKAATG', 'QKAATGP', 'AATGPFD', 'DREHL', 'REHLL', 'HLLMY', 'LMYLE', 'EQKDR', 'QKDRE', 'FVPFTG', 'GEKKGR', 'KVTLDP', 'VTLDPE', 'AAVLGVH', 'KFDEE', 'KGPVRN', 'RNVVKG', 'EKAKPV', 'NPTNVE', 'EASLQQ', 'ASLQQM', 'MKANDP', 'NDPSLQ', 'PSLQEV', 'PIPTLK', 'TLKSLN', 'LKSLNV', 'LNVESN', 'ITGTGIL', 'NQRQQ', 'QRQQL', 'RILKF', 'FTKQGP', 'RTRVAA', 'VAAAITK', 'KQEPLG', 'RIQQE', 'EIAVQN', 'VQNPLV', 'LELSVL', 'YKEYA', 'AEDDNI', 'KYSYI', 'SYIRK', 'IRKTR', 'KTRPDG', 'SHLEAL', 'LEALLD', 'EALLDD', 'KELQR', 'AKSKED', 'SQGFTE', 'YLRLL', 'TSGYLQ', 'RESKF', 'EHFIE', 'ALAQALS', 'AQALSVS', 'VSIQVE', 'QVEYM', 'RGEGGTT', 'EGGTTNP', 'TTNPHV', 'VYLLY', 'PSPEMI', 'LEYIE', 'EKLRQ', 'KLRQQ', 'DYNPY', 'NPYQGV', 'NGEESH', 'GEESHL', 'IILEAL', 'ENEPPS', 'FPVDTP', 'YETQQ', 'ERKLK', 'FPLMY', 'VEEQY', 'EEQYT', 'NQKRE', 'RERVD', 'SGQLGLP', 'RRESK', 'SKDQLS', 'RLVNAV', 'GSGRSQS', 'SGRSQSG', 'QSGPNGD', 'AARLLQ', 'AGEKPNG', 'ALPDGLS', 'LSVEDI', 'SVEDIL', 'VVNQKS', 'VNQKSP', 'YFPNQ', 'FPNQY', 'GPGKSRA', 'ANQIPK', 'NDQELG', 'VPSPVSS', 'EMERL', 'RLAKVS', 'KRIPVG', 'PNRQY', 'QYLDE', 'AERYE', 'RVLSSI', 'LSSIEQ', 'EGSEEK', 'EVKEY', 'TELRGV', 'ELRGVC', 'LIKGAGD', 'DAESRV', 'SRVFY', 'VFYLK', 'GDYYR', 'RIIDSA', 'IIDSAR', 'SAYQEA', 'NPIRLG', 'ADLHTL', 'ADSAGEE', 'EAPEEP', 'APEEPQ', 'ILLLCV', 'MVLGEQ', 'VLGEQE', 'LGEQEV', 'VSDNEL', 'YINKE', 'EIQNAV', 'IQNAVQ', 'VQGVKH', 'IEKTNA', 'SLEEAK', 'RVCRSG', 'RSGSGLV', 'VGQQLE', 'QVLDAM', 'VLDAMQ', 'ASGIIDT', 'FFARE', 'RELHD', 'HDPHY', 'DPHYF', 'FSPIGF', 'PIGFPH', 'HKRPH', 'RPHFL', 'PHFLY', 'HFLYP', 'FLYPK', 'LMSPSH', 'YGPPSF', 'HNMFQ', 'MFQPF', 'QPFFE', 'PFFEM', 'FEMIH', 'QAQQAM', 'AMDVQL', 'LHSPAF', 'GQCEKC', 'EILSVD', 'AQANLR', 'NDSLQV', 'TEQYK', 'SSLLEQ', 'TQGEDK', 'LRVSTV', 'SEVPSR', 'TEVVVK', 'LPEEVS', 'GALALLL', 'IVEISL', 'VQNMAL', 'LYADVS', 'GKQFPV', 'FPVTRG', 'VTRGQD', 'TRGQDV', 'WSLEH', 'KSAHAGT', 'HAGTYE', 'TYEVR', 'NNEDVS', 'NEDVSI', 'EDVSII', 'PPLFTV', 'FTVSVD', 'SVDHRG', 'IGIVIY', 'IVIYY', 'VIYYL', 'YYLAF', 'YLAFSA', 'FSAKSH', 'MFQGAD', 'QGADSQA', 'ADSQAGK', 'SMKPPGG', 'KPPGGES', 'GISSSKP', 'PNRMAS', 'TNPPGGK', 'PGGKGSGI', 'SGIFDE', 'GIFDES', 'TRQRL', 'NPPGGKT', 'GGKTSDI', 'VTATAPL', 'SKSDLK', 'TDSTPR', 'PRGEQS', 'EPRLGP', 'GPRPRS', 'KVLNPP', 'KSSLSF', 'GEDFAM', 'EGSRAR', 'GSRARM', 'QRSVEG', 'WILFV', 'TGVHEE', 'ATEEDI', 'GEIKNI', 'NLDRR', 'ETYKE', 'AQAAMEG', 'NGQDLM', 'VDWCF', 'DWCFV', 'FVRGPP', 'TMKNPS', 'LNLGCR', 'LSDEHA', 'GNIMIQ', 'KHDGIT', 'DGITVAV', 'DGAKIGN', 'DTKRR', 'CPGGQLP', 'GQLPFL', 'RYPKL', 'LNPESN', 'FAKFSA', 'IKNSNP', 'DNLEKG', 'NLEKGL', 'LEKGLL', 'LKALKV', 'KALKVL', 'SPLPEE', 'DCNLLP', 'VQVVCK', 'KYRGF', 'FTIPEA', 'TIPEAF', 'PEAFRG', 'HRYLS', 'YAREE', 'EQVARA', 'AASTAAGK', 'AGKQRI', 'KVAKVK', 'ITAEQL', 'TAEQLL', 'KEREL', 'ERELE', 'RKNRT', 'QWEES', 'KEIQR', 'RSIYE', 'SIYER', 'DVDYR', 'YRNIT', 'EMEMK', 'EMKNR', 'MKNRQ', 'DRAITT', 'RAITTL', 'TYMEE', 'GARQVF', 'EQAWH', 'YKEVE', 'AVEFFG', 'HMDEH', 'DEHLY', 'ALDRIS', 'ISKQEA', 'FKNYT', 'YTIFE', 'GDRRGI', 'EDIIVS', 'DIIVSK', 'YEEEV', 'NYDAW', 'ERAIAN', 'NVPPIQ', 'IQEKR', 'LYEEL', 'LEAKDP', 'TRQVY', 'SLELIP', 'KKFTF', 'EIRQK', 'RRALGT', 'LGTSIGK', 'YIELE', 'EFDRC', 'DRCRK', 'RCRKL', 'AELETI', 'AISQPR', 'EVLWK', 'ERTRN', 'TRNLY', 'QRTQH', 'HVKVW', 'LSSGKEG', 'KEERL', 'EERLM', 'DEFGTV', 'KVQADD', 'YIFPE', 'NLKLLA', 'AEQDPD', 'CIESLI', 'FQKYS', 'NNTQLS', 'TQLSKT', 'MNTELA', 'FLNLIG', 'ITASQF', 'FDADGSG', 'GYLEGK', 'RKKAGL', 'SPEMKS', 'FVDQY', 'VELAHV', 'LFRCQ', 'DHSGFI', 'DTKLAE', 'NDGKLE', 'DGKLEL', 'ARLLPV', 'FLLKF', 'ELYDQ', 'NELDAL', 'ELDALL', 'LDALLK', 'DALLKD', 'ALLKDL', 'INNITT', 'LYRTD', 'YRTDL', 'AEAAAAPA', 'APAAGPGQ', 'GQMSFT', 'FTFASP', 'TLTGAFG', 'PTLQVL', 'QVLRPG', 'LVVVHT', 'GTTTKY', 'VQLLAE', 'DLGAARA', 'LGAARAN', 'GAARANL', 'ARANLE', 'LEKAQS', 'KAQSEL', 'NEALVK', 'PLEEAL', 'EEALDV', 'VMSNLD', 'CVFLSC', 'CIAMMC', 'FEGCPD', 'ASVSELA', 'LACIYS', 'CIYSAL', 'DDEVTV', 'EVTVTE', 'DKINAL', 'KINALI', 'INALIK', 'LIKAAGV', 'VSVEPF', 'VEPFW', 'EPFWP', 'ANVNIGS', 'CNVGAGGP', 'GAGGPAPAA', 'AGGPAPAAG', 'GGPAPAAGA', 'MGFGLF', 'SRALIL', 'TMLSLC', 'LSLCGGE', 'GEDDIE', 'DDIEAD', 'GTTVYQ', 'QSPGDIG', 'SPGDIGQ', 'QYTHE', 'ELFYV', 'TVWRL', 'VWRLP', 'GGLQNIA', 'IAAEKH', 'AAEKHN', 'HNLGIL', 'VFPKSP', 'FPKSPV', 'WLRNS', 'LTFIPS', 'WEPEI', 'PMSELT', 'VVCALGL', 'IQGLRS', 'QGLRSGG', 'LRSGGTS', 'IKKETG', 'KETGFS', 'ETGFSH', 'SHSQIT', 'YSRFT', 'LDKGEN', 'GENGTLS', 'NPLGDR', 'PLGDRI', 'EGEDQV', 'RGFMR', 'FMRTL', 'HFRPI', 'PLNSRS', 'HFAFR', 'AFRLY', 'VNISDE', 'TIQEAD', 'AISFTE', 'GKLFVGG', 'DTNEQA', 'TNEQAL', 'QISEVV', 'EVVVVK', 'SRGFGF', 'ENIDDA', 'NIDDAK', 'IDDAKD', 'DAMMAM', 'AMMAMN', 'NGKSVDG', 'RQIRV', 'AGKSSDN', 'GYRGGSA', 'YRGGSAG', 'RGGGDRG', 'SRSQGGS', 'VWQQD', 'QWDKV', 'WDKVK', 'KDSGRD', 'FESSSL', 'LGQQLN', 'GQQLNL', 'LLENW', 'LENWD', 'WDTLGS', 'TVSQLQ', 'QLQER', 'WDNLE', 'KETDW', 'TDWVR', 'RQEMN', 'WKEDV', 'APLGAEL', 'AELQES', 'ELQESA', 'RQKLQ', 'LQGRLS', 'THVDSL', 'TQLAPH', 'APHSEQ', 'LAQRLA', 'LNEYH', 'HLKTLG', 'KTLGEK', 'LGEKAR', 'EKARPA', 'EDLRH', 'QVQSVI', 'GGRKGGH', 'VRQYT', 'TSPEEI', 'PEEIDA', 'AEKQKA', 'GDGASGDP', 'GDPKKE', 'KKSLDS', 'DDYQQ', 'NPNRVA', 'RVAQTT', 'AQTTKK', 'TTKKVT', 'EREEI', 'REEIE', 'AKERY', 'KADLAR', 'DDATLSG', 'DGPVQGT', 'GPVQGTI', 'IHFEQ', 'LSGQITG', 'SGQITGL', 'EGQHGF', 'FHVHQ', 'TQGCTSA', 'CTSAGPH', 'GPHFNP', 'GGPADEE', 'EERHV', 'VGDLGNV', 'GDLGNVT', 'NVSIED', 'RVISLS', 'IGRTMV', 'KQDDLG', 'QDDLGK', 'LGKGGNE', 'GNAGSRL', 'AGSRLAC', 'TEPPPE', 'PEIEGE', 'DFMAAL', 'LDGDEK', 'KPCLDT', 'DTSQVE', 'DYPDY', 'QNWPE', 'EDASFC', 'DTDQAE', 'NEHRD', 'EHRDD', 'ADLLFV', 'FTERD', 'DNPSED', 'SYGMLP', 'APNSPCS', 'VTIETL', 'TTDVVH', 'STDTTPG', 'ITKPDV', 'TKPDVI', 'STESDM', 'NIILPT', 'ILPTEP', 'LPTEPD', 'TEPDES', 'APPMEE', 'ETTLPI', 'TTLPIK', 'KMDLAP', 'PPEDVL', 'APAKGMV', 'SLSEIE', 'SEIEEA', 'ITTLTK', 'TLTKDV', 'LTKDVT', 'TKDVTL', 'KDVTLP', 'VTLPLE', 'LPLEAE', 'PLVTDM', 'MTPSLE', 'PSLETE', 'PTETNL', 'MSPLPE', 'PLPESE', 'LPESEV', 'ESEVTL', 'VVILPE', 'VILPET', 'ETKVAE', 'TPLSEE', 'VKDMSP', 'IVDNSM', 'NSMAPAS', 'SMAPASD', 'TVPIKD', 'QLASMQ', 'VPPCTAS', 'PPCTASP', 'KAAEQM', 'QMSTLP', 'SPLENL', 'TPGSQPS', 'PSEPCSG', 'SEPCSGV', 'EPCSGVS', 'GVSRQE', 'QEEAKA', 'GVTGNDI', 'KAKTQP', 'TSLPKQ', 'SLPKQP', 'TSGGLNK', 'PAAATATA', 'RPSTLP', 'TLPARD', 'LPARDV', 'KRTSPS', 'RTSPSK', 'PSKPSSA', 'SKPSSAP', 'SSAPALK', 'SKATSPS', 'ATSPSTL', 'TLVSTGP', 'IKTEGK', 'GAAPPAGM', 'SRSSGAL', 'RSSGALS', 'SVDKKP', 'DKKPTS', 'TKPSSSA', 'KPSSSAP', 'SVRSKV', 'NAVTKAA', 'AQKPPAG', 'KVQIVS', 'VQIVSK', 'IVSKKV', 'VSSKCGS', 'HKPGGGD', 'KEKAQA', 'EKAQAK', 'GSEALPC', 'PEAAPDA', 'AAPDAGAP', 'GLSGHTT', 'GHTTLSG', 'HTTLSGG', 'SGGGDQR', 'TLDSQI', 'GSEPSVT', 'VAKRNS', 'YDITR', 'VLLEQA', 'DATESF', 'SPDARE', 'REMLK', 'DVHPSD', 'VHPSDL', 'KGDDKD', 'DDKDPS', 'DPSKNN', 'VPIVGAI', 'ILIGFL', 'HFWAD', 'RVRQI', 'RRLVK', 'VMYEK', 'DGENYA', 'DCQRR', 'CQRRL', 'LEAAYT', 'EAAYTD', 'YKEAR', 'ARVVLD', 'LDSVKL', 'MGLLEL', 'QVLGVR', 'GEVRRG', 'KVSLQV', 'RFQIL', 'YAVLSD', 'DWDAY', 'RLLFK', 'EKTYK', 'KGSEEE', 'GDMDQI', 'EPRIR', 'RIRNI', 'QKAIES', 'KAIESK', 'AIESKE', 'AQEEAK', 'EEAKEA', 'EAKEAE', 'KEAELS', 'DNLKAL', 'NLKALI', 'QKDRQ', 'KDRQK', 'DRQKE', 'AQMEAK', 'CKPSKGG', 'EVAAAVL', 'LGEGDGGA', 'GDGGAFGS', 'YILGVL', 'GVLQEE', 'VERWS', 'RWSET', 'WSETR', 'VTTKVK', 'DEVQAI', 'ATLIEK', 'RKAALL', 'NVSSDR', 'TNVEDV', 'KLQRE', 'LQREK', 'QREKD', 'REKDK', 'AEQEPT', 'EQEPTA', 'QEPTAE', 'EPTAEQ', 'KPPAQK', 'AQKSIQ', 'SLRKY', 'AVSADPN', 'GDLESF', 'LKEGVE', 'RVNRE', 'DYMVGS', 'RAEEY', 'YEFLT', 'TPMEEA', 'PMEEAP', 'KGMLAR', 'LARGSY', 'ARGSYN', 'HLSWE', 'STAVLAL', 'QVFALP', 'NSPMTK', 'CVLEVI', 'LEVISD', 'SPECLE', 'ILRHQ', 'LRHQN', 'RHQNL', 'LQDLAL', 'QEQQH', 'DAAAEVP', 'VPSRDT', 'EKRKD', 'AFPEPN', 'PNQESP', 'MMGDSE', 'SPGEDTA', 'PQATGDS', 'DSERGL', 'LSAQQQ', 'EEAVAR', 'EAVARE', 'AVAREK', 'VAREKA', 'PEEVPT', 'EVPTAAS', 'IQKDDG', 'GDGKTEA', 'ASEALPS', 'GELEHS', 'VGTPQGL', 'GLFPQGG', 'EHKQE', 'HKQEE', 'EEERL', 'LAKELT', 'EKRLE', 'LEGEDD', 'SFRTR', 'RTRAY', 'RDPGPQ', 'PGPQLR', 'PQLRR', 'RGWRP', 'DSVEAR', 'VEARSD', 'DFEEK', 'KEEEGS', 'SANRRA', 'ELESLS', 'AHQLQA', 'RPCTPP', 'FEFLL', 'ACPDPAP', 'PVQLIV', 'LEQASK', 'KPSVNE', 'PSVNEQ', 'QPPPDN', 'PPPDNK', 'KRNRV', 'LKLLAL', 'LKWDL', 'KWDLD', 'LEKSLS', 'VPVLNM', 'ELLCIS', 'LLCISK', 'ALPPTTA', 'AIRTIV', 'PVKQAK', 'VKQAKP', 'PPQLNV', 'DCILVL', 'LEAALR', 'EAALRL', 'AVEPGTV', 'EPGTVNG', 'PGTVNGE', 'GTVNGET', 'VCYDLG', 'FQQGST', 'TDSAIY', 'TKELLA', 'AEIGSLS', 'LSLHCT', 'SLHCTI', 'EKRLAG', 'SQQLTP', 'SGSYQE', 'TFTLPV', 'VQFRQ', 'LRELF', 'VLEGRT', 'FNQLF', 'QLFLR', 'SRSINV', 'INVEKA', 'ASDSLKG', 'GLEDLQ', 'EDLQY', 'LQYVF', 'YVFMI', 'KDEER', 'DEERK', 'EERKL', 'DQMRK', 'MRKRS', 'LCIKPV', 'NIGQLE', 'QLILSV', 'VDPWR', 'NKWEV', 'VYSSVI', 'HCSTVK', 'LFTACL', 'CLELVT', 'RTHEAG', 'THEAGVG', 'QSGERP', 'LPDIPL', 'QVVAEE', 'ENEYL', 'LTLQVP', 'LQVPAF', 'QVPAFL', 'FLFQN', 'PYVKLG', 'KLGQLL', 'GQLLAAT', 'GPKESR', 'KESRR', 'SVSNQH', 'RGNDGR', 'IKQRE', 'IMYRS', 'YRSEL', 'KKLRE', 'EPLVLT', 'LVLTII', 'VLTIIL', 'IVNDIT', 'AEHISI', 'SIWPSS', 'SIPNLQ', 'IPNLQS', 'NLQSVD', 'SVDFEA', 'WLIIQ', 'LIIQAD', 'IIQADI', 'DFFNK', 'FNKAVP', 'KRMIK', 'IKCCSL', 'KCCSLL', 'HTQVAI', 'CQFLR', 'FLREI', 'EIDYK', 'TAFKSL', 'QNSHDA', 'NSHDAM', 'SHDAMD', 'HDAMDS', 'MDSYY', 'YLTYL', 'TYLHH', 'QTELNA', 'SNPEEV', 'EVLQLA', 'AGNIDDA', 'IDDALQ', 'YQKAY', 'TVDLKP', 'GKGYSR', 'ANNLQL', 'TRSLLS', 'NKPSDL', 'GVDLGSM', 'GSMDEE', 'EEEEAA', 'ATPPPPP', 'KELGND', 'GNDAYK', 'LKHYD', 'AKELDP', 'DPTNMT', 'QAAVHF', 'AAVHFE', 'LCEKAI', 'SYFKE', 'TPDVLK', 'EQERL', 'KHYTE', 'KRNPR', 'ALKDCE', 'KAAALEA', 'AAALEAM', 'KALDLD', 'DSSCKE', 'QRCMM', 'SPEDVK', 'DVKRR', 'SDPAMR', 'KDPQAL', 'ALSEHL', 'VIAQKI', 'AAVGVVR', 'FCHVAT', 'ATPHTF', 'PLRAPL', 'LRAPLC', 'RAPLCN', 'TQDTPN', 'QDTPNP', 'NPNSLK', 'LKFIPG', 'RTMDF', 'FPTPAAA', 'PTPAAAF', 'SPLARQ', 'QLFRI', 'LFRIE', 'ELDWN', 'WNLLK', 'IMDFF', 'LPLVTE', 'VTEETP', 'TEETPP', 'EETPPP', 'GEAGSEE', 'YRGFE', 'DGIVRL', 'SIITLK', 'IITLKS', 'GIQNML', 'QFYIP', 'FYIPE', 'SDEKEA', 'EKEANS', 'TTMASSA', 'SSAQRK', 'RKRMS', 'SPKPEL', 'KQEIR', 'QEIRE', 'DGTGTID', 'GTGTIDI', 'KVAMRA', 'FKLFD', 'ELGENL', 'GEVNEQ', 'EFLRI', 'RIMKK', 'ALCASDV', 'PGEGNCP', 'NCPCCK', 'LGALWD', 'ECCDCV', 'GMCNPR', 'STVEEL', 'PIPSLF', 'QLNWN', 'NIVSFP', 'PSNNVH', 'VHAPFP', 'DKERM', 'ISCESM', 'YRWF', 'WFHNA', 'HNACCE', 'DYGSKT', 'DKEFM', 'FMWAL', 'YVAKGE', 'PLHYAA', 'AADCGQL', 'GQLEIL', 'AVYEGH', 'HVSCVK', 'VKLLLS', 'TVKGPDG', 'GLTALEA', 'EATDNQ', 'PELARS', 'LARSLL', 'LAALLAP', 'ALLAPAL', 'ALLVSPA', 'VSPASGR', 'PPLPPR', 'PPREDG', 'PREDGP', 'EDGPRV', 'STIKEV', 'PFADII', 'SISDGPP', 'GPPGEGTG', 'YMYLS', 'HIMMSG', 'LFVRH', 'RHPEM', 'ISRIW', 'LDYFGG', 'YFGGPK', 'KVVTPE', 'VVTPEE', 'EYFNV', 'LTDAEK', 'AEKAAVS', 'AVSCLW', 'NSDEVGG', 'DEVGGEA', 'EVGGEAL', 'LGRLLV', 'YFDSF', 'KAHGKK', 'NDGLNH', 'DGLNHL', 'GLNHLD', 'IVLGHH', 'VLGHHL', 'PAAQAAF', 'AGVATALA', 'SNPAAIP', 'PAAIPHA', 'EDIQGD', 'HNRFV', 'FSPLHA', 'LNFGIGG', 'LKNGEL', 'NGELEN', 'GELENI', 'IVVWVG', 'VWVGTN', 'GTNNHE', 'TAEEVAG', 'EEVAGGI', 'EVAGGIE', 'IEAIVQ', 'EAIVQL', 'PQAKII', 'QAKIIV', 'LGLLPR', 'KNAKVN', 'NAKVNQ', 'AKVNQL', 'LLKVSL', 'KVSLPK', 'KLANVQ', 'DMFDF', 'MFDFL', 'FDFLH', 'VYSPTT', 'SPTTEAA', 'GTGRGAR', 'AYFVLG', 'ILKGLQ', 'LKGLQL', 'KGLQLL', 'LLLSLL', 'SECGLCG', 'EFVSCS', 'LLSLLL', 'SLLLLI', 'CTPVHD', 'RVDTGK', 'YITLGT', 'FVSTHS', 'STHSGTS', 'SAEIAAI', 'KPENNA', 'PENNAK', 'ENNAKV', 'NNAKVE', 'MASRLV', 'RLVSAM', 'SPRTPD', 'TPDRVS', 'QRLLH', 'LLHGVM', 'GVMEQL', 'VMEQLG', 'GIARPR', 'AHQAMN', 'QAMNLV', 'SIEGGAH', 'NIPNIV', 'QGYPDP', 'DTAEFS', 'EFSRE', 'GLGKWN', 'MKGGQR', 'SVPHFS', 'RGLQTV', 'LAGVRR', 'GVRRGK', 'RGKPVL', 'GPTQSLA', 'QSLASGI', 'LASGISAG', 'GQLYST', 'SLHSII', 'LHSIIS', 'HSIISN', 'IISNTE', 'AVRGSVS', 'FIREL', 'ISNASDA', 'SNASDAL', 'LRHKL', 'LQTDAK', 'IGMTQE', 'ELVSNL', 'SNLGTIA', 'ARSGSKA', 'IGQFGVG', 'SDGSGVF', 'EIAEASG', 'SGVRPGT', 'RPGTKI', 'CKDFAS', 'PLYLNG', 'DPKDIS', 'RFTLH', 'LYSRK', 'LIQTKA', 'VDSEDI', 'DSEDIP', 'SEDIPL', 'RLIKF', 'LIKFF', 'IKFFI', 'KFFID', 'FFIDQ', 'QSKKDA', 'SKKDAE', 'YAKFF', 'REGIVT', 'TAEQDI', 'DIAKLL', 'LRYES', 'YESSAL', 'ESSALPA', 'ALPAGQL', 'PAGQLTS', 'TRNIY', 'IYYLC', 'LCAPNR', 'YYEAM', 'MKQKH', 'LFCYE', 'EQFDE', 'LHLRE', 'EFDKK', 'VETDIV', 'ETDIVV', 'SPADER', 'AWMRN', 'WMRNA', 'SRVTNV', 'DTHPAM', 'LRMQQ', 'RMQQL', 'AQLLQP', 'PELAQL', 'DQIYE', 'MIAAGLV', 'VDDPRA', 'VGRLND', 'IMGPVR', 'MGPVRL', 'GPVRLE', 'LPSKCE', 'SKCEVC', 'EALSRT', 'RHVPY', 'GSLRYA', 'KGLVQK', 'LELWD', 'DEPSVE', 'MLEEF', 'EEFED', 'HHQEQ', 'FLCER', 'PASETAC', 'LREAW', 'TGKEKI', 'GNPKHD', 'LNEPLS', 'NEPLSN', 'RVISSI', 'AYREK', 'REKIE', 'VCQDVL', 'QDVLSL', 'LSLLDN', 'LIKNCS', 'ETQYE', 'KVFYL', 'AEVATGE', 'KRATVV', 'RATVVE', 'KAYSEA', 'AYSEAH', 'KEHMQ', 'ALNYSV', 'YSVFY', 'EIQNAP', 'IQNAPE', 'CHLAKT', 'TLNEDS', 'QQDDDG', 'ILDALE', 'LDALEN', 'DALENL', 'FKMKL', 'RIPRGA', 'MDAIDL', 'SYYLE', 'YYLES', 'YLESY', 'LTMTVL', 'ESGAVAAA', 'VPAQSTA', 'TGHFVD', 'RQALIA', 'QALIAR', 'LIARVT', 'ARVTEV', 'VLDALH', 'QAVRAE', 'VRAETT', 'QDKMR', 'DKMRK', 'KMRKL', 'GAAAAATAA', 'GSAGEQE', 'SAGEQEG', 'AAGSGSGGGG', 'GSGGGGSAAG', 'SGGGGSAAGG', 'GGSAEAEG', 'AEAEGAK', 'ASKNEE', 'EDEGHS', 'HTEAAAA', 'GLSWDT', 'SKFGEV', 'ESVDKV', 'QKEHK', 'PKRAKA', 'FVGGLSP', 'EEKIR', 'EKIRE', 'REYFG', 'FGEVES', 'TNKRR', 'FAGRAR', 'GRARGR', 'RARGRG', 'WNQGY', 'YWNQG', 'GGYDYT', 'GYDYTG', 'SRRGGH', 'RGGHQN', 'DLREH', 'REHTE', 'LKDCPT', 'DEFKK', 'FPYGDA', 'VFRTF', 'DGTIDF', 'REFII', 'LEIVQA', 'EIVQAI', 'PEDEST', 'EDESTP', 'DESTPE', 'TTTTTF', 'TFKGVD', 'PNSRNS', 'NSSRVL', 'SLGFDE', 'LGFDEP', 'EPAEQP', 'RKNKM', 'EENPPS', 'NPPSWA', 'PSWAKS', 'QRSNSS', 'LDLKGE', 'APVPSPV', 'PVPSPVA', 'VPSPVAP', 'VSRRR', 'SRRRV', 'APGAAGTQ', 'PGAAGTQA', 'KYSFK', 'SFKGPH', 'GSVWTK', 'ENWEV', 'NWEVE', 'DGLAIW', 'ENQGLD', 'DGPVFGS', 'DNDGKK', 'NDGKKN', 'DGKKNN', 'KKNNPA', 'PAIVVIG', 'VVIGNNG', 'NYDHQ', 'YDHQN', 'NDGATQA', 'DGATQAL', 'LASCQR', 'CQRDF', 'RNKPY', 'TLTVMI', 'KNDYE', 'DYEFC', 'NMVIPT', 'FGISAAT', 'AATGGLAD', 'ADDHDV', 'TAEKDI', 'KGHPDL', 'PADDIF', 'FESIGD', 'ESIGDR', 'SIGDRE', 'QNRIH', 'RIHLE', 'YVSSLT', 'GQPGQVS', 'ELDTVV', 'LDTVVK', 'NSMSET', 'TVRLVS', 'GVYETT', 'QHFMD', 'PSNEKP', 'PKCPDL', 'KCPDLP', 'CLSTIH', 'VIFVVV', 'GFLLGGI', 'FLLGGIG', 'IGELNK', 'GELNKN', 'NKNRH', 'RHPNF', 'EDTFR', 'RQFLN', 'QFLNR', 'EMVRH', 'QRSIPA', 'KEHPY', 'HPYDAA', 'IESKEA', 'QEALAAA', 'AAGDKLV', 'DKLVVV', 'KLVVVD', 'KPFFH', 'KVGEFS', 'EKLEAS', 'MPGTALS', 'LLLLLS', 'LLSWAS', 'APDQDE', 'AKQPSF', 'SGYLRA', 'GYLRAS', 'WFVES', 'NDPKNS', 'NGGPGCSS', 'GGPGCSSL', 'PGCSSLD', 'GLLTEH', 'LLTEHG', 'GPFLIQ', 'IESPAGV', 'GFSYSD', 'DDKMY', 'VTNDTE', 'TNDTEV', 'EVAENN', 'ENNYE', 'LKDFF', 'LFPEY', 'EYKDN', 'YKDNK', 'LTGESY', 'SYAGIY', 'YIPTLA', 'VMQDPS', 'DPSMNL', 'MNLQGL', 'YEQND', 'HCCAQN', 'AQNKCN', 'KCNFY', 'NFYDN', 'DPECVN', 'PECVNN', 'CVNNLL', 'NLLEVS', 'IVGKSGL', 'VGKSGLN', 'SGLNIY', 'APCAGGVP', 'EDTLVV', 'DTLVVQ', 'TLVVQD', 'PLKRR', 'LMRSGD', 'WDMCN', 'RLYQS', 'KLLSSQ', 'LLSSQK', 'VDSLNQ', 'RPWLV', 'VDYGES', 'GAGHMVP', 'FSRFL', 'RFLNK', 'EWQLV', 'KVEADL', 'VEADLAG', 'KKHGCT', 'TVLTALG', 'VLTALGT', 'KKGQHA', 'QHAAEI', 'HAAEIQ', 'AEIQPL', 'EIQPLA', 'IQPLAQ', 'QSHATK', 'SGDFGAD', 'GAMSKAL', 'LELFR', 'ALAALCI', 'LGGVTGAP', 'PRLRQ', 'LRQFL', 'TGKQEL', 'LLSEPN', 'SEPNQT', 'QTENDA', 'DALEPE', 'ALEPED', 'EDLPQA', 'QDEMR', 'EMRLE', 'AMAPRE', 'APRTLL', 'EPRYI', 'SDWRL', 'AADMAAQ', 'QITRR', 'ITRRK', 'KWEQS', 'WEQSGA', 'PRSKGE', 'RSKGEV', 'GEVTLR', 'GEELTQ', 'CRVYH', 'PSTDSY', 'VIVAVLG', 'IVAVLGV', 'VAVLGVL', 'VLGAMAI', 'MAIIGAV', 'IGAVVAF', 'SQSSEM', 'QSSEMS', 'SPSQVR', 'NYHQD', 'HQDAEA', 'EAAINR', 'AAINRQ', 'FDRDD', 'FAKYF', 'RIFLQ', 'VNQSLL', 'KLATDK', 'FIETY', 'YYLSE', 'SEQVKS', 'IKELGD', 'ELGDHV', 'LRKMGA', 'DKHTLG', 'EDERK', 'DERKM', 'ERKML', 'GDPKEE', 'REHCE', 'EKCVKA', 'DNRVSS', 'ELFDF', 'CVAHKL', 'HKLFK', 'QQLDCA', 'MRRLP', 'NLSDLI', 'LSDLID', 'SDLIDL', 'LIDLVP', 'QPLKIA', 'RDKVVG', 'CDYNR', 'MPSARL', 'LEVEAN', 'EVEANN', 'YFEGGV', 'GGVSSVY', 'VSSVYL', 'SVYLW', 'GFAGVIL', 'KKIKGC', 'DSIHVV', 'VQEKSS', 'KSGSGTM', 'DCSPHI', 'NIGRLV', 'STLNEI', 'IVNGLR', 'LRSLDA', 'RSLDAI', 'NHKFK', 'KQLQR', 'QLQRE', 'SQVLTQ', 'EKIADP', 'RMLEK', 'TEEENG', 'ENGKKP', 'KPVSED', 'DMVSIL', 'VSILQL', 'SILQLV', 'IQNIGE', 'EQGHMA', 'AYISTL', 'ISTLDK', 'KEKLR', 'EKLRK', 'NGCAYF', 'EREGLA', 'REGLAK', 'IHSRY', 'SRYED', 'YEDFV', 'QLGLPF', 'CNTMFG', 'TMFGSQ', 'GRLPLL', 'LPLLLV', 'VANAGTAA', 'ANAGTAAV', 'AAVGHTD', 'GHTDKI', 'HTDKIG', 'IWLHV', 'CDSMTL', 'DSMTLT', 'LGLPAVP', 'LPAVPAV', 'AVPAVTL', 'LYKHD', 'YKHDD', 'HDDPAL', 'TLVAGLT', 'NKPADK', 'KPADKL', 'ADKLRA', 'HACHLS', 'QRLQE', 'VVFRF', 'VSNIAPA', 'LTVIDL', 'MTAEGLG', 'TRGEDV', 'QSKLPV', 'KLPVLT', 'QLREE', 'RYEHA', 'DPEGEK', 'PEGEKI', 'KIHTGL', 'HTGLLK', 'GLLKKL', 'LESDLT', 'SMKSCI', 'IYIGMA', 'VSELVE', 'SELVET', 'REIEE', 'VVRKGI', 'IPVVGSV', 'PVVGSVL', 'GSVLNW', 'NLTAGSL', 'TEYTY', 'EYTYV', 'VHKVQG', 'HKVQGT', 'TGVTPPP', 'GVTPPPT', 'GTRSKQ', 'RSKQR', 'PGQKPF', 'KPFKR', 'RSLRGS', 'SVSHIE', 'NLEAHS', 'EAHSPE', 'AHSPEQ', 'QPPRAT', 'PRATDL', 'RATDLT', 'TDLTAR', 'ARQTEA', 'TEALQN', 'EELER', 'HNLLSS', 'NLLSSH', 'NPEFVA', 'AEVPKE', 'VPKEPT', 'LQCPES', 'SGRLFP', 'FPISRG', 'LAELEV', 'AELEVL', 'LEVLCT', 'LYVGTD', 'YVGTDL', 'ALLELI', 'ELIDSP', 'LIDSPE', 'PECLSK', 'LLLEQG', 'TTSYAQ', 'INPLPI', 'RIDIR', 'LNYVAS', 'PKLAPF', 'KLAPFV', 'APFVIQ', 'PFVIQA', 'QALIQV', 'IQVIAK', 'QVIAKL', 'IAKLTK', 'AKLTKL', 'FREII', 'IADVKK', 'QGTVEH', 'EHCIIG', 'RKIATS', 'KIATSF', 'TSLKDI', 'LKQVLA', 'SLVMQV', 'LKLVLS', 'LGSSADE', 'DLCTVQ', 'LCTVQI', 'TWRTI', 'PLLSQL', 'ASTRRS', 'ERAKY', 'QGLSDPG', 'ANFTIT', 'SLQHW', 'SVPFVK', 'VPFVKS', 'KSAEPH', 'KAFITS', 'RLESVA', 'AIVVRD', 'DPLDDT', 'TATVFQ', 'CTVSRC', 'CEYEK', 'EYEKT', 'CTLLVQ', 'GLAVDMA', 'VDMAIQ', 'AIQEGR', 'QEGRLA', 'LIYLVG', 'GGRLTY', 'KIELAV', 'VLWFL', 'FLDQF', 'RKTYV', 'TSKVYA', 'TKIVTN', 'LKYWG', 'PVISRT', 'VGYILL', 'KIDAVK', 'KFMLK', 'EHFPF', 'GISETY', 'TYNVGD', 'TALTRL', 'DLGEDE', 'LGEDED', 'EDEFE', 'MLPLTV', 'LRGIAF', 'YTMLF', 'AYLPVL', 'AIERW', 'GEPACTT', 'TTPILK', 'ILKLLA', 'QRLNF', 'NFDVSS', 'DVSSPNG', 'VSSPNGI', 'NGILLF', 'LLFRE', 'ICTYGN', 'LSLGSLS', 'SLSKDK', 'KDKIY', 'PMKLKG', 'ICYSAL', 'YSALKS', 'LCGNYV', 'YVSFGV', 'VFKLY', 'VLQAFV', 'SHSDLL', 'YRKLS', 'YYPLL', 'TNLEPP', 'NLEPPV', 'LEPPVL', 'EPPVLL', 'SSSCCTS', 'YMVTY', 'TYLFK', 'EGKKPL', 'QAGQRL', 'LHFMQ', 'FMQQN', 'FEDCR', 'RPLLGL', 'LNEKY', 'EKYFS', 'KYFSE', 'LRASLI', 'RASLIN', 'LINSQP', 'INSQPL', 'SQPLPK', 'KQEVLG', 'QEVLGQ', 'QCFRN', 'LMEGVE', 'EGVEQN', 'NLSVKN', 'RSDGHT', 'GQLRHG', 'QLRHGG', 'AQESLE', 'LEVAIQ', 'AIQCLE', 'LPQTLP', 'DPRAPD', 'DRTPPS', 'PPSEED', 'KTEGNE', 'TEGNEQ', 'FEAAVH', 'AAVHLY', 'SLNKHA', 'NKHAEA', 'AYYKK', 'YYKKA', 'DPDNDT', 'LREAPS', 'PSPTGGVG', 'VGSLDIA', 'FITMAS', 'TPGSSPQ', 'QAGQQF', 'GQQFAQ', 'SQVVRS', 'HEEQQ', 'AALGGNSS', 'GGNSSPSA', 'KILDSV', 'IEADDD', 'LNKVIS', 'VISELN', 'ELNGKN', 'NGKNIE', 'GKNIED', 'EDVIAQ', 'IAQGVGK', 'AQGVGKL', 'QGVGKLA', 'VGKLASV', 'GKLASVP', 'LASVPAGG', 'VAVSAAPG', 'GSAPAAAE', 'AAEEKK', 'VKVRGL', 'KVRGLP', 'WSCSAD', 'RFFSD', 'GRPSGEA', 'DEVKLA', 'EVKLAL', 'VKLALK', 'KKDRE', 'FGCSKE', 'GRSTGEA', 'SQEIAE', 'KKHKE', 'KSSRAE', 'VRTHY', 'YDRPGA', 'RPGAGRG', 'PGAGRGY', 'NSIGRGA', 'IGRGAGF', 'RMRRG', 'NDGYGF', 'SDHRY', 'STFQST', 'GLPYRA', 'DIYNF', 'YNFFS', 'FSPLNP', 'VHIEIG', 'THEDAV', 'AVAAMSK', 'YVELF', 'STAGASGGA', 'AYEHR', 'ASGGAYGS', 'GSQMMGG', 'SQQLSGG', 'LSGGYGGG', 'SGYDQV', 'LQENSS', 'SSDFQS', 'LSPSGNT', 'EPVAEE', 'QEELR', 'RRELT', 'QTLSQV', 'ELKRK', 'RKLGIS', 'ISSLQE', 'ATNAYK', 'SAAFSSV', 'SVITKK', 'SPTFKS', 'GAKPAGGD', 'NSTANAT', 'STMTTE', 'MTTEPP', 'RNLHK', 'LHKTAV', 'KTAVHN', 'TAVHNGA', 'LFVHR', 'FVHRD', 'TPENNP', 'IEAIVK', 'EAIVKN', 'QAAAVLP', 'LPVLDL', 'AQRQNG', 'GWLPIS', 'YNRKP', 'NRKPVG', 'DSILET', 'SILETL', 'TTPDKL', 'SGRFCC', 'FCCEPA', 'PGFGVQA', 'GEGALAPG', 'ALAPGLQ', 'DTDTAR', 'VAGPEAGA', 'PEAGAEP', 'AGAEPAGP', 'AEPAGPE', 'EPAGPEA', 'VRAAEAG', 'VDCSDE', 'SALQLAA', 'SVNSRN', 'SRNHY', 'NHYGW', 'CGHVSVA', 'LLLDHG', 'GADVNAQ', 'SVLTVAS', 'LEAGAIV', 'EAGAIVD', 'TPSGESP', 'GESPATGG', 'GITALMA', 'TALMAAV', 'QHGHEA', 'AVVRLL', 'EWGADP', 'KLNVAQ', 'LNVAQQ', 'NVAQQL', 'QLVEKG', 'PDHLSV', 'SVLEKT', 'DEEKR', 'KRRPD', 'GNFQLV', 'LVKEIA', 'VKEIAD', 'LAAVTGH', 'AAVTGHL', 'VHGWTA', 'GWTALM', 'QATYHG', 'NQGADVA', 'QGADVAL', 'ALRAKN', 'NGYTAF', 'GYTAFD', 'LNDPDT', 'PDTELV', 'RQPWS', 'QTLPRG', 'GLAANQP', 'PLPFSD', 'DEPELA', 'EPELAL', 'TMRAPP', 'RTSHLG', 'GPPEAAH', 'ATKDSGP', 'KDSGPGN', 'NPRRE', 'VLLTTM', 'LPSDKL', 'DKLKAV', 'KLKAVI', 'PPFLPP', 'HTCHNG', 'SRAHPV', 'HPVGCVG', 'PVKFPS', 'PASPASSG', 'ASSGSFN', 'NHSPHS', 'GIGGMSR', 'GGMSRLG', 'GMSRLGG', 'RSGGSVD', 'GSVDSVL', 'GLCEQK', 'QSSPVGP', 'SSKKLD', 'SKRPPS', 'PSPSPKG', 'GILKKL', 'YQPIF', 'DGDLQE', 'GDLQEL', 'DLQELG', 'LQELGI', 'GIKTDGS', 'IHNFH', 'TRAPGNG', 'RAPGNGP', 'SDRKAV', 'KAVIKN', 'ADMSED', 'QQDAVD', 'EKYNI', 'KYNIE', 'YNIEK', 'KKEFD', 'KEFDK', 'DKKYN', 'FGSYVT', 'VAILLF', 'IQAGQCG', 'QAGQCGN', 'AGQCGNQ', 'EHGIDP', 'YNEASS', 'VPRAIL', 'PRAILV', 'HLFRP', 'AKGHYT', 'HYTEGA', 'YTEGAE', 'GTGSGMGT', 'TLLISK', 'LLISKV', 'NTFSVV', 'YNATLS', 'IDNEAL', 'GDLNHL', 'VTTSLR', 'SLRFPG', 'FPGQLN', 'QLNADL', 'KLAVNM', 'FPRLH', 'PRLHF', 'HFFMP', 'FMPGFA', 'GFAPLTA', 'LTVPEL', 'QQMFD', 'YLTVAT', 'FRGRM', 'IQSKNS', 'PNNVKV', 'PPRGLK', 'LKMSST', 'IGNSTAI', 'TAIQEL', 'LFKRI', 'FRRKA', 'RRKAF', 'AFLHW', 'DEMEF', 'MNDLVS', 'NDLVSE', 'YQDATA', 'MYEDD', 'MGKRW', 'LLLLPT', 'LLLPTN', 'LPTNASA', 'LNETVT', 'ETVTVK', 'MESVRG', 'RGNQSL', 'LFHCAS', 'HCASFI', 'KGPTHE', 'RRRST', 'TVLIKT', 'TKESLV', 'PIYKPG', 'VRFRV', 'RFRVV', 'VSLDEN', 'RIAQW', 'QNFRL', 'PETITI', 'EMNVSV', 'GIYTYG', 'RKYSN', 'SCFGEE', 'CFGEES', 'ESLAFC', 'SLAFCE', 'FCEKF', 'SQLVKT', 'LVKTKS', 'QLKRQ', 'LKRQE', 'VNAKIQ', 'IQEEGT', 'DTHFR', 'LLVDGR', 'ANQNIN', 'NGLARF', 'TSLTVR', 'WLTEE', 'PYKLR', 'ILNDEA', 'DEAVLE', 'LERKE', 'ERKEL', 'ELVFY', 'MAKGGIV', 'GIVRAGT', 'VTQGHK', 'GHFSIL', 'VARLVL', 'PNGEVVG', 'GEVVGDT', 'IEKCLA', 'EKCLAN', 'KCLANK', 'AFLSVM', 'FLSVMA', 'RAVDQS', 'DQSVLL', 'VLLTKP', 'TKPEAE', 'ASLVYD', 'PVKDLT', 'KGVNQQ', 'GVNQQE', 'NGILYS', 'PVQNTN', 'LKDMGL', 'VCERLG', 'CERLGV', 'IPAAYH', 'PAAYHL', 'AAYHLV', 'SQGHMD', 'GHMDAF', 'TRSYF', 'RSYFP', 'DLVIVD', 'DSTGVAE', 'VAEMEV', 'MEVTVP', 'PDTITE', 'ITEWK', 'AGAFCLS', 'SPVIDF', 'VDLTMP', 'IRVGVQ', 'RVGVQL', 'FLATPE', 'ATPEEK', 'ERHTM', 'SWAVIP', 'VNFTVS', 'DTIIKS', 'TIIKSL', 'IKSLLV', 'EPEGLE', 'LLCPTGA', 'CPTGAEV', 'SEQISL', 'ISLKLP', 'LKLPSD', 'SDVVEE', 'DVVEES', 'LGDILGS', 'GEQNMV', 'MVLFAP', 'LFAPNI', 'APNIYV', 'YVLDY', 'IKTKAI', 'TYLNTG', 'LNYKH', 'YIFID', 'SSGSLLN', 'LLNNAM', 'NAMKGGV', 'TLSAYI', 'LSAYIT', 'SAYITI', 'EMSLPV', 'VVRNAL', 'FCLDTA', 'KSARRG', 'ARRGASG', 'SGNHVY', 'LDEEAV', 'AVKEDN', 'HWTRA', 'KPRVPA', 'RVPADL', 'ADLWY', 'LTTELV', 'AMLIVK', 'KWLTK', 'WLTKQ', 'QQNSHG', 'SHGGFSS', 'LHALSK', 'AATFTR', 'ATFTRA', 'TRAKKA', 'AHVTIQ', 'TAKVAGE', 'KEFPF', 'FALVVQ', 'LVVQTL', 'CEDLKA', 'LKAHTT', 'NISYIG', 'IADVKM', 'KMVSGF', 'GFIPLK', 'IPLKPT', 'KMLER', 'VSNNHV', 'LDKVSN', 'FFMVQ', 'EKDEF', 'KYSAPC', 'APCSAGY', 'CSAGYGN', 'MADQLT', 'ADQLTE', 'DGDGTIT', 'LGTVMR', 'RSLGQN', 'RKMKD', 'EEEIR', 'EEIRE', 'NLGEKL', 'EMIRE', 'SSGNAKI', 'PAPNFK', 'PNFKAT', 'YVVFF', 'TFVCPT', 'CPTEII', 'CQVIGAS', 'QVIGASV', 'GASVDSH', 'LISDPK', 'VLKADE', 'DEGISF', 'GLFIID', 'TINDLP', 'INDLPV', 'SVDEII', 'KHGEVC', 'TIKPDV', 'SKEYF', 'KEYFS', 'EYFSK', 'YFSKQ', 'VATEPTG', 'KGLELL', 'GLELLE', 'KAAGMLS', 'AAGMLSQ', 'LKYLM', 'PALQGAL', 'ALQGALT', 'KQVNPS', 'QVNPSK', 'KRLDH', 'DHLQR', 'RAREH', 'EHFVH', 'AYPNLV', 'YPNLVA', 'EVEHR', 'SALKSAV', 'KSAVESG', 'SAVESGQ', 'DERVR', 'EYHLL', 'HSSLPE', 'SLPEKP', 'VSDWY', 'EQHQK', 'KYGVLP', 'LPDRGI', 'AAQQQE', 'SEEKAL', 'DDWKD', 'GISPGQL', 'SPGQLVA', 'QLVAVF', 'KSSPEE', 'ALKKLV', 'LKKLVA', 'TGSEGQV', 'QLLQSS', 'SSHKES', 'SHKESS', 'SSFDVI', 'ALTLSGL', 'ELQRE', 'REALSP', 'LSPEEV', 'DSLRSV', 'VGSSSQL', 'SSQLKL', 'PVVDPAA', 'VDPAAAK', 'DLIDSD', 'ASLKAPS', 'LKAPSCG', 'PSCGEGK', 'GEGKKR', 'CKNCTC', 'ELERE', 'LEREQ', 'QSSQPK', 'PKSACGN', 'CGNCYL', 'GDAFRC', 'CANCPY', 'PYLGMP', 'QVLLSN', 'RPLDAV', 'AEPEEV', 'EEDFE', 'EDFEQ', 'ASLTRE', 'LTRER', 'RERGR', 'PARNRG', 'KTQVEA', 'ESALNH', 'PRAEKA', 'KAQEMA', 'AQEMAK', 'QLVRR', 'VRRIE', 'MASGNAQ', 'AQIGKSA', 'TATAVVD', 'SDHAED', 'GCEVLGV', 'LGVSVDS', 'PLNIPL', 'KSLSQN', 'SQNYGV', 'LKNDEG', 'KNDEGI', 'DEGIAY', 'ITVNDL', 'RSVDEA', 'FQYTD', 'EHGEVC', 'TIKPNV', 'NVDDSK', 'PAAAPKL', 'APKLED', 'ASPGLPK', 'LPKGEK', 'NEQASE', 'QASEEI', 'VEQKY', 'PFFQK', 'IPNFW', 'LLGEED', 'ENPYF', 'NPYFE', 'PYFEN', 'KEFHL', 'ESGDPSS', 'SGDPSSK', 'GDPSSKS', 'TEIKW', 'SRKRQ', 'TWFTD', 'DHSDAGA', 'HSDAGAD', 'KDDIW', 'EGLEDI', 'DIDEEG', 'IDEEGD', 'GKMLGGD', 'EKDPDA', 'AAKKEE', 'AKMEAE', 'ANSEGSL', 'PPGCGDE', 'PGCGDEP', 'GDEPEE', 'ESILDT', 'ILDTVI', 'KYLPGP', 'QDMFK', 'PEAASAE', 'EKAEEA', 'KAEEAK', 'KRLQK', 'DSGDYN', 'GDYNMA', 'AKAKMK', 'NKQLPA', 'QLPAAAP', 'PAAAPDK', 'AAAPDKT', 'IPTPQD', 'PTPQDL', 'TPQDLP', 'NPSQLL', 'SQLLPL', 'IVGTLLG', 'VLEDVT', 'ITPEGR', 'RRITK', 'ITMLVP', 'TMLVPGG', 'RVALVL', 'VLGLLR', 'LLRIY', 'LRIYL', 'LPEDTP', 'VYTLNG', 'YTLNGT', 'FSVDPN', 'SVDPNF', 'EREDE', 'NLVAEK', 'KVVILV', 'VPENIP', 'AEDKDT', 'SGGSVTY', 'GGSVTYS', 'SVTYSL', 'DRHSGV', 'RHSGVL', 'HSGVLR', 'LQAGATL', 'QAGATLD', 'LDYEK', 'GGGRLRG', 'ADMVFS', 'DMVFSA', 'VTINVE', 'TVVAIDG', 'ILYRL', 'LYRLL', 'ESDGIF', 'SDGIFE', 'LRREV', 'VQVTEV', 'VNDSDQ', 'KFNLR', 'NLRLVG', 'PQTVLN', 'VLNEAQ', 'LNEAQV', 'QVTIIV', 'VTIIVE', 'IIVENS', 'IEVNTP', 'DTNDNV', 'TSHYY', 'NVVAVTA', 'VAVTAVD', 'AVTAVDP', 'AVDPDTG', 'PDTGPW', 'LIHPST', 'TGLIYT', 'NFYVK', 'GRYSLA', 'LAEVFV', 'TLLDVN', 'PLKIEA', 'LKIEAT', 'KIEATD', 'IEATDQ', 'EATDQD', 'TDQDAE', 'DAEEPN', 'AEEPNN', 'PNNLVD', 'SITRAE', 'TRAEPV', 'DIDAHT', 'SLEALH', 'ALHNIT', 'HNITPS', 'QAKDRG', 'ALLKID', 'IDITDT', 'RLSRGS', 'DNPMKA', 'LAGVMAI', 'NKVLPV', 'RRRPS', 'KFRRA', 'FRRAK', 'RRAKAA', 'DSPNEN', 'MVPPRA', 'PPPPKM', 'MVAQQT', 'QQTVPT', 'QPSPQL', 'TPKTLGG', 'PKTLGGP', 'KTLGGPV', 'TLGGPVQ', 'PDRPVS', 'PVSLEQ', 'DKYEN', 'KYEND', 'EKIRK', 'KIRKM', 'MRNYS', 'WMDII', 'DIITIC', 'CKDTLP', 'KIKMF', 'FEEHL', 'IRYIL', 'WIRIS', 'HRFTL', 'DEKNY', 'ADHFDA', 'QYMSF', 'DTLTKV', 'TLTKVL', 'LTKVLV', 'VLVALY', 'HLGAATP', 'EATVEE', 'LKAKLV', 'MAEPQP', 'DETAFS', 'FSLYF', 'YFLAY', 'YEDKN', 'SRKATL', 'KATLEL', 'LTHNW', 'SDPRGF', 'PRGFGH', 'GHIGIAV', 'GIAVPDV', 'VYSACK', 'EELGVK', 'GVKFVK', 'MKGLAF', 'GYWIE', 'WIEIL', 'AAEDVVA', 'TGADPSE', 'ELEGGGL', 'IFTSPL', 'IFLLY', 'VRGDQP', 'DDEPPP', 'DEPPPL', 'PPPLPR', 'PRLKR', 'FTPAEL', 'GVQDPR', 'ILMAIN', 'INGKVF', 'FDVTKG', 'DVTKGR', 'GRKFY', 'RKFYG', 'FYGPEG', 'GPEGPYG', 'ASRGLAT', 'LATFCL', 'ATFCLD', 'CLDKEA', 'LKDEY', 'KDEYD', 'DDLSDL', 'TPAQQE', 'PAQQET', 'QFTFK', 'HVGKLL', 'PLPRAL', 'LPRALP', 'PAPASAR', 'SARGRQ', 'GLCGSLE', 'LETCVN', 'CQKVPV', 'VPPGALL', 'SRTGFT', 'YTQRV', 'VIAQEL', 'YLWHG', 'PARSLQ', 'KTLRR', 'YLPYL', 'RIDHE', 'SPENIL', 'YVAHTS', 'ELSPKV', 'GDIDKQ', 'AALGAYT', 'ALGAYTA', 'EPRYL', 'YLVHGS', 'ARAPRP', 'APRPFS', 'PRPFSA', 'SATALSQ', 'TALSQR', 'SQRWP', 'DTLLQS', 'TLLQSL', 'DRLGPL', 'GPLKEE', 'EEKADS', 'GRGAEGQ', 'LQDHR', 'QDHRL', 'LSEVDD', 'EVDDPV', 'LLEKSS', 'REEIK', 'PEEVLS', 'FERKP', 'KPNSEP', 'PNSEPQ', 'NSEPQP', 'QPRRL', 'PRRLE', 'PELWE', 'AQGPPSGG', 'PSGGLQL', 'LEVQPS', 'SEEQQG', 'PLSPEK', 'LMDQVA', 'SSFFAD', 'LTILQS', 'LQSGIR', 'PKGKLK', 'GKLKLL', 'HQEEQ', 'SIACILG', 'LGVLLAS', 'LLASSLA', 'SLAYCL', 'YKLKD', 'DPSADAT', 'DATEAY', 'RPQDR', 'TSRINS', 'SRINSV', 'RINSVS', 'SSQFSD', 'GPMPSPS', 'EPVQSN', 'PVQSNM', 'TGHMIL', 'RLEKE', 'EALCAY', 'ALCAYQ', 'LCAYQA', 'AYQAEP', 'SLVAQR', 'QREEN', 'ENAPKN', 'APKNRS', 'YDHSR', 'LKSQNS', 'DPRNPA', 'AYIATQ', 'PLPATVA', 'ESGCAVI', 'VIVMLT', 'IVMLTP', 'VMLTPL', 'NGVRQC', 'SEHIW', 'VRSFY', 'WYDQG', 'YDQGVP', 'QGVPSST', 'STRSLL', 'RRKVN', 'RGRSCP', 'IIVHCS', 'IVHCSD', 'VLNKMA', 'LEHLR', 'EHLRD', 'HLRDQ', 'RDQRP', 'ALTAVAE', 'EEVNAI', 'VESSYS', 'SDFISC', 'TGRRNA', 'GRRNAV', 'RNAVPD', 'SVRKLA', 'EGQAEGS', 'AEGSTPD', 'STPDKE', 'SSYRR', 'RRMFG', 'YVTTST', 'TTSTRT', 'GAYVTR', 'RSSAVR', 'LLQDSV', 'LQDSVD', 'DSVDFS', 'SVDFSL', 'DFSLAD', 'RFANY', 'NKILLA', 'ILLAEL', 'KSRLGD', 'DLYEE', 'EMREL', 'LRRQV', 'RRQVD', 'DNLAED', 'NLAEDI', 'REKLQ', 'EMLQR', 'MLQRE', 'LQREE', 'SLARLD', 'RDVRQ', 'YKSKF', 'ADLSEAA', 'EAANRN', 'DALRQA', 'AKQESN', 'SNEYR', 'QVQSLT', 'EVDALK', 'LERQM', 'MEENF', 'DTIGRL', 'MKEEM', 'EYQDL', 'LNVKMA', 'NVKMAL', 'VKMALD', 'ALDIEI', 'LDIEIA', 'SRISLP', 'TFSSLN', 'TNLESL', 'NLESLP', 'LESLPL', 'LVDTHS', 'VETRDG', 'TRDGQV', 'SDAAVDT', 'DAAVDTS', 'EVVEEA', 'VVEEAE', 'EAENGR', 'PANGNAQ', 'EADNEV', 'EAEAPTG', 'AEAPTGK', 'VAEDDE', 'SQQSEV', 'TRILDG', 'QLVDIS', 'LVDISQ', 'VDISQD', 'LAGNPKA', 'PPQIVN', 'YELFV', 'LFVEAV', 'VEAVEQ', 'AMKGLGT', 'DSILNL', 'SILNLL', 'LLTSRS', 'LTSRSN', 'QRQEI', 'GRDLVD', 'VDDLKS', 'EKLIVA', 'KLIVAM', 'LIVAMM', 'VAMMKP', 'KGAGTDE', 'GAGTDEK', 'TPEELS', 'KQVYE', 'QVYEE', 'VYEEE', 'RDPDTA', 'DDAQVE', 'DAQVEL', 'AQVELD', 'QVELDA', 'VELDAQ', 'AQALFQ', 'QALFQA', 'GTDEEK', 'DEEKF', 'FGTRSV', 'SHLRR', 'VFDKY', 'YMTISG', 'GNLEQL', 'SIRSIP', 'AYLAET', 'YYAMK', 'GAGTDDH', 'IRVVVS', 'VVSRSE', 'FNIRK', 'KNFATS', 'IKGDTSG', 'TSGDYK', 'KALLLL', 'SVFLLV', 'LVPGTQG', 'TEDGDM', 'VEIFY', 'GRWGTV', 'LLDAHV', 'NATQALG', 'GRAAFGP', 'RAAFGPG', 'GLHILD', 'SGELSDA', 'GCDLFI', 'GQGYED', 'AHTLIL', 'WQVVGS', 'QVVGSSV', 'VDAECM', 'ECMPVV', 'RRIEV', 'LHKLAS', 'ATELQD', 'ELQDY', 'CGRLFA', 'LFATLL', 'LPQDPT', 'ARATGDS', 'EPLTQS', 'PLTQSE', 'ESWSAV', 'PTTLIQ', 'LIQALL', 'QALLPK', 'LDLLKA', 'LKAVDQ', 'IASHED', 'ASHEDI', 'LYQDH', 'YKPRL', 'KPRLY', 'LYTSST', 'TSSTWS', 'YNSYQ', 'YQSFQ', 'FLFKD', 'LPTMQS', 'SCWNY', 'YGFSCT', 'LTTSSY', 'SYSNPT', 'VLILCGG', 'VDVTSF', 'GSKAPIP', 'TNSTDM', 'QSVEIP', 'VQENSP', 'NSPGHR', 'PFFDF', 'GSRLNK', 'DLLKAN', 'LKANVE', 'KMLIY', 'LREASV', 'ASVTPSN', 'PSNLWG', 'NLWGGQ', 'QGLLGVS', 'FCSFDG', 'VLEVES', 'ESNSPAA', 'AALAGLR', 'GLRPHS', 'SDYIIG', 'AKPLKL', 'LYVYN', 'TDTDNC', 'EVIITP', 'TPNSAW', 'RPFEE', 'PITPLK', 'PLKDGF', 'DGFTEV', 'TEVQLS', 'LSPPGTT', 'SLSGLSI', 'LSGLSIS', 'LSISSAP', 'SNVLSTG', 'VLSTGVP', 'LSTGVPT', 'TGVPTVP', 'LLPPQV', 'LPPQVN', 'PQVNQS', 'SMPPMN', 'NFNLPA', 'MPGVGLP', 'PGVGLPE', 'GLPELGS', 'ELGSPGL', 'PSLPPR', 'RNLPGI', 'LPMLSD', 'LPSFPL', 'SLPAMGP', 'AMGPPSD', 'GPPSDPV', 'TTAKADA', 'DVTSPAS', 'EKPVSD', 'PVSDADA', 'AQVAQPG', 'EIPAKI', 'GHLMIV', 'HLMIVG', 'MIVGKK', 'VGKKCAA', 'KCAADLG', 'YRMVV', 'VYHIH', 'HLHVLG', 'QMNWP', 'LLSVAVA', 'SEAVRD', 'QLEASM', 'SMHSVL', 'MHSVLS', 'SVLSDF', 'IPDGTAV', 'RVSIPT', 'IPTDLI', 'LIASSGE', 'IASSGEI', 'ASSGEII', 'LHWDP', 'LPLDTD', 'GVHYIS', 'VSAARLG', 'AARLGAN', 'RLGANGS', 'SHVPQT', 'HVPQTS', 'QTSSVF', 'NEPQSV', 'SVRAASS', 'ACAADEP', 'LTVILD', 'RMQSF', 'GPGNAKK', 'AKKVVE', 'CSLNQN', 'LNQNSV', 'IRGVET', 'VETPAR', 'GAMSAQL', 'AMSAQLG', 'QLGYPV', 'IANKKP', 'PKRLR', 'QIHATP', 'IQEPPS', 'PPSRIV', 'PSRIVP', 'SRIVPT', 'PTPTSPA', 'PAIAPPT', 'PPTETM', 'MAPPVR', 'IRTRGA', 'RTRGAI', 'IIQTPT', 'IQTPTL', 'EAGTTVP', 'VPGQIR', 'PGQIRP', 'GQIRPT', 'IPGYVE', 'TTTKKP', 'ATPSTDS', 'STTTTR', 'TPRPVP', 'PVPRVT', 'VPRVTT', 'RVTTKA', 'APITRL', 'RLETAS', 'SPPTRI', 'TSGVPRG', 'GVPRGGE', 'EVKIPS', 'KLREQ', 'LREQQ', 'REQQL', 'EQQLVG', 'QQLVGE', 'QLVGEK', 'YFMHA', 'TDKGGLS', 'HVHKR', 'RFKAR', 'KKIALV', 'KIALVK', 'IALVKK', 'ALVKKL', 'LVKKLA', 'KKLAFA', 'SITLQN', 'NNTLPL', 'TLPLEP', 'LPLEPC', 'EQIIGL', 'NGKPRP', 'PRPAFS', 'ALSIAVT', 'TGSGSCR', 'GSCRHL', 'QFIPVA', 'PSPGSSAA', 'SPGSSAAP', 'PEKSSE', 'IPAVVVA', 'AILLIAG', 'LIAGIIA', 'TLEDQA', 'LEDQAT', 'PIIFAD', 'ADELDD', 'LDDSKP', 'SMPLIL', 'PNQSMP', 'MPETTP', 'ETTPLN', 'TTPLNQ', 'TVGEYT', 'APPYQP', 'KGSRPK', 'MTPYR', 'PPPYVP', 'DMNEY', 'GSKINAS', 'ASKNQQ', 'NQQDDG', 'EYLSR', 'TIKTDP', 'IKTDPV', 'VTGRSR', 'KLDGKL', 'LKGKEP', 'KKVFVG', 'GLSPDTS', 'PDTSEE', 'GEIENI', 'TNERR', 'LESRY', 'GKCEIK', 'KVAQPK', 'VAQPKE', 'AQPKEV', 'QQKGGR', 'QKGGRGA', 'GGRGAAAGG', 'GAAAGGRGG', 'ARGRGR', 'YNSAYG', 'TGYNYG', 'GYADYS', 'YADYSG', 'YSGQQS', 'HQNNY', 'FRQNL', 'RQNLF', 'EDSLNV', 'PLDIPI', 'LDIPIP', 'DIPIPD', 'DPPPKD', 'PPPKDD', 'GNEKLL', 'KLLALL', 'WTLKE', 'KIEDGN', 'KTKVEA', 'QTTISK', 'AVAKASK', 'LRAFY', 'ELYHI', 'HIISSN', 'SSNLEK', 'NPKGEE', 'ESAMET', 'AMETLI', 'DKYKL', 'DVQKDA', 'VQKDAD', 'ELDENG', 'VACNNF', 'SCLVLH', 'CLVLHS', 'FHVSW', 'GATFCVA', 'AVDSLNG', 'RGDNIP', 'IPKVLV', 'YFLRA', 'TVDTPH', 'HGMFVS', 'GTPFEQ', 'GSGPYGH', 'PYGHTV', 'GHTVKK', 'YNSLGD', 'QVLGTPG', 'VLGTPGK', 'GTGLNPL', 'YVDDTG', 'DDTGEM', 'MILWL', 'GTGPAKF', 'NIPHSV', 'HSVTLD', 'VTLDAVG', 'LGAWDN', 'PSAVRF', 'LSRLSV', 'SRLSVL', 'QLADQV', 'VDRKTG', 'HSRTPA', 'MSVFGK', 'FGKLFG', 'GKAGKGGP', 'NKRAAL', 'LKRKK', 'NTNTEV', 'TNTEVL', 'MKAAHD', 'AISKPVG', 'EEFDE', 'PSVALPS', 'MAEDFG', 'SESGAPE', 'AFLAQQ', 'LAQQES', 'QESEIA', 'ENDPGF', 'NDPGFGA', 'GFGAPAAS', 'SEDMST', 'EANGPAD', 'IAQADR', 'AQADRL', 'QKKRL', 'KKRLQ', 'KRLQE', 'RIADKA', 'QQPDAD', 'DTIGYV', 'PKSSKQ', 'SLKQTP', 'KQTPLS', 'ASAMKAV', 'PLSNED', 'KAYRE', 'LETVCN', 'LALLDK', 'FQYES', 'KKNSVV', 'KNSVVE', 'VEASEAA', 'SEAAYK', 'EQACLL', 'CLLAKQ', 'LAKQAF', 'FGPVNH', 'EKHCR', 'NFDFQ', 'FDFQN', 'FQNHK', 'EVERGS', 'LPEFY', 'LAQESQ', 'AQESQD', 'IGSQANS', 'EDRHL', 'SDNPAGL', 'NPAGLAE', 'KRPAAE', 'PAAEDSS', 'TEENVS', 'NVSDGSP', 'VSDGSPN', 'TVEQTP', 'AVLCLGT', 'VLCLGTA', 'LATPKF', 'DQTFSA', 'AEWHQ', 'GTNEEE', 'RAIWE', 'EKNMR', 'EMNAFG', 'RQVVNG', 'HQKHK', 'QKHKK', 'KIPKSV', 'WREKG', 'VKNQGQ', 'KNQGQC', 'WAFSAS', 'QMFLK', 'ISLSEQ', 'SLSEQN', 'LVDCSH', 'AQGNQGC', 'FAFQY', 'YIKEN', 'DGSCKY', 'KYRAE', 'TVGPISV', 'GPISVAM', 'SHPSLQ', 'FYSSGI', 'LVGYGY', 'YEGTDS', 'TDSNKN', 'NKNKY', 'KNSWGS', 'GMEGYI', 'HCGLATA', 'TADARH', 'ADARHP', 'DARHPA', 'HPAGTSC', 'CPSDAAL', 'LLRDTG', 'LRDTGL', 'RDTGLL', 'SDTVRP', 'AVNVRD', 'VRDRE', 'EAAWPP', 'TEPRPA', 'RPALTI', 'PVFAKL', 'TLNWH', 'NWHSQ', 'WHSQD', 'HSQDGAG', 'LSQGLR', 'YVFLE', 'FTNTGH', 'TGHKVQ', 'VQGWVS', 'LVLQAK', 'QAKPQV', 'LLLLKA', 'GHRLSV', 'HRLSVG', 'TSFGLF', 'GLFLVK', 'VKPDNP', 'EQRGW', 'KRQFV', 'AEIPGGQ', 'IPGGQEA', 'LENHY', 'ENHYL', 'LFKHK', 'PRRSR', 'DRVTW', 'ERSKR', 'PMWNQ', 'LHVIPV', 'ITGKGVV', 'TGKGVVI', 'GKGVVIT', 'ITVLDD', 'WNHTD', 'YDFND', 'RYDLT', 'NNHKCG', 'YNSKVG', 'NSKVGGI', 'IEASSIG', 'FNPGHV', 'VDIYSA', 'SWGPND', 'GRLAQK', 'LAQKAF', 'KQGRQG', 'QGRQGK', 'SGNGGRQ', 'CDCDGY', 'DCDGYT', 'CDGYTD', 'SSTLATS', 'ATSYSSG', 'QRITSA', 'THTGTSA', 'SRFGFG', 'LADPRT', 'DPRTW', 'TWRNV', 'WRNVP', 'CVVKDN', 'RALKAN', 'VIVEIP', 'IVEIPT', 'PTRACE', 'IEYSR', 'VGTSTVL', 'RERDT', 'PNGFKN', 'GENPVGT', 'NPVGTW', 'VGTWTL', 'RMQNE', 'NWKLI', 'WKLIL', 'EHMKQ', 'PTQKSL', 'DEQVQG', 'QSAFSK', 'SKQSPK', 'QSPKKS', 'ESFYE', 'FYNTK', 'KPYKH', 'KHRDD', 'RDDRL', 'ALMDIL', 'MATLQL', 'RAPLLC', 'LLCVLL', 'LLWVF', 'LWVFC', 'GSVGLDK', 'HLEGVI', 'IDQPEA', 'LQLHY', 'FKMHD', 'DYDGNS', 'YDGNSL', 'GNSLLDG', 'AITHVH', 'KEEGSE', 'EEGSEQ', 'EQAPVM', 'PVMSED', 'DELVSI', 'ELVSII', 'IDGVLR', 'DGVLRD', 'GVLRDD', 'SDVLEL', 'FFAPW', 'NKYGVS', 'LKIFR', 'RDGEEA', 'AYDGPR', 'GPRTADG', 'SVPLRT', 'KDASVVG', 'LFSDGH', 'ASNLRD', 'NYRFA', 'TNIESL', 'NIESLV', 'KEYDD', 'FRPLH', 'KFEDK', 'YTEKK', 'FGLCPH', 'KDLIQG', 'DLIQGK', 'LIQGKD', 'IQGKDL', 'QGKDLL', 'YYDVD', 'YDVDY', 'DYEKN', 'RNRVM', 'ELSDFG', 'SDFGLE', 'DFGLES', 'GEVPVVA', 'EEFSR', 'RYLKS', 'KSEPIP', 'SEPIPE', 'EPIPES', 'GHCKNL', 'LSKDPN', 'SKDPNI', 'DPNIVI', 'IVIAKM', 'VIAKMD', 'DVPSPY', 'EVKGFP', 'PANKKL', 'ANKKLT', 'SYLQR', 'KAVTEQ', 'VTEQGH', 'QGHELS', 'ERNEK', 'RNEKK', 'DICNDV', 'ICNDVL', 'RYLSE', 'SEVASGE', 'GENKQT', 'ILNSPE', 'KEVWD', 'MFWW', 'WWLY', 'YATNPC', 'TNPCKN', 'VMWLQ', 'LQGGPGGS', 'GGPGGSSTG', 'GGSSTGFG', 'SSTGFGN', 'GFGNFE', 'RNTTW', 'TGFSYV', 'TDAYAK', 'FFDCH', 'FYIFS', 'KMAAGIS', 'QQGTIK', 'YLYSM', 'DIAEQV', 'IAEQVL', 'AEQVLD', 'EQVLDA', 'ATQLWG', 'NFYNI', 'FYNIL', 'NILTKS', 'KSSPEK', 'LEFLR', 'RLCQR', 'GPIKKK', 'IPEDIS', 'MKPAID', 'KLLAAGV', 'DLIVDT', 'LIVDTI', 'WVQKL', 'KLKWP', 'LKWPQ', 'KWPQL', 'MVPSDQ', 'AWAPLD', 'LDPTSGS', 'LSGLQR', 'QRVKF', 'LAPIRV', 'APIRVL', 'VLLAFI', 'LLAFIV', 'LAFIVL', 'FLLWP', 'LLWPF', 'WLQVAG', 'GLTEEQ', 'RGQRAS', 'GQRASR', 'SRLEAP', 'PVLVAAP', 'DPIVLL', 'LLPCDL', 'PCDLPK', 'CDLPKV', 'DLPKVV', 'VSRAEN', 'VPVIGAL', 'GALLRF', 'LRFNQ', 'RATSGGK', 'GGKWPQ', 'FKPGAF', 'AFIAGVP', 'AGVPVQP', 'VQPVLI', 'WAWRG', 'WRGPGV', 'GPGVLKV', 'ESKDPT', 'RVMAQA', 'AQALGIP', 'QALGIPA', 'QLKVAL', 'LKVALE', 'ALEPQL', 'EPQLW', 'PQLWE', 'ELAKVL', 'LAKVLQ', 'GLSPGFV', 'DMGAEPG', 'GAEPGRS', 'MISQEA', 'AFAQQL', 'LQLSDP', 'SYFQQ', 'NVALALA', 'ALDGGRS', 'AEEQAE', 'EGSDRL', 'TTLHAE', 'QPGCSQG', 'KPRSTS', 'PRSTSQ', 'PSSPTAL', 'SPTALAN', 'VQAPKQ', 'DVPLLN', 'VPLLND', 'NDYKQ', 'DYKQD', 'FLLKR', 'LLKRF', 'KRFPQ', 'PQTVLGG', 'QTVLGGP', 'YIYVN', 'MPWALG', 'TGTLLY', 'LRDYT', 'LSGGLMV', 'GLMVFT', 'VIQLIS', 'IQLISV', 'DQHEF', 'AGAETVK', 'KFLIPG', 'FLIPGK', 'YVANTV', 'FHSVLA', 'VLAGLVC', 'CIGEYS', 'TETATF', 'TATFQT', 'IFFFV', 'HRFIV', 'IPALEQ', 'LPFLW', 'ALGTLPP', 'PDALLF', 'VLEFGL', 'SMSTHL', 'MFIVSA', 'VSAGAAVV', 'GAAVVSY', 'IPSTVGV', 'STVGVVL', 'VLFMTG', 'VTRHR', 'TRHRV', 'HEARPG', 'LFIAVL', 'IAVLGMA', 'AVLGMAL', 'LHHYV', 'PQVSRS', 'SRSGAQA', 'LMVLLS', 'MVLLSI', 'FYPKD', 'VFYDK', 'YDKQN', 'KLLKIG', 'ILLTLV', 'LDSSLH', 'MVWQN', 'WQNTE', 'ETALLE', 'LLETVV', 'TVVVSAV', 'WNRSL', 'LDTGIR', 'DTGIRL', 'TGIRLL', 'RDRLI', 'RLIQF', 'LWTEK', 'EKKQH', 'QHRKT', 'VGFPRP', 'GFPRPV', 'VQSWPG', 'SWPGAVG', 'TAACVCT', 'CTDSVF', 'YHQMV', 'TAALQTA', 'AMAAGSLG', 'GSLGLLL', 'LPGSHY', 'PGSHYL', 'LIWIM', 'IWIMI', 'WIMIL', 'ETSCHT', 'TAEAQR', 'AEAQRV', 'QREHP', 'REHPQ', 'HPQVCS', 'ENVLTP', 'VLPVKL', 'ARNVLT', 'SPDNLK', 'KEFKD', 'CSKRPS', 'SKRPST', 'PSTQEN', 'NTGEASP', 'LPPAANS', 'ANSSPCP', 'CPESLE', 'DEPAIK', 'EPAIKN', 'KNRKE', 'NRKEK', 'RKEKL', 'KDLPCT', 'DLPCTK', 'LPCTKL', 'CTKLPI', 'TKLPIP', 'IPGSVDS', 'DSQNPD', 'SQNPDD', 'DLYRT', 'DVAYIP', 'VAYIPL', 'AYIPLV', 'VEFSCS', 'SCSQSR', 'LLSLPE', 'LPEEW', 'STPRSK', 'YQFVL', 'VLEEIA', 'YIGLFG', 'GIDNGVP', 'PGQLVR', 'GQLVRV', 'FHLVR', 'RKAFR', 'LMVDKA', 'GPIEDF', 'EDFKE', 'SEEQW', 'WKRAI', 'CLGYES', 'GYESSM', 'YESSMG', 'VHIGKL', 'ERYSI', 'RYSIQ', 'IEGYVP', 'GPPPADL', 'DLCHAL', 'YNHIK', 'KEKASL', 'KASLPGV', 'ASLPGVK', 'GSGAADAK', 'DAKDDD', 'AKDDDD', 'FGSDDE', 'KLREE', 'PAVVAKS', 'VRSIQA', 'WGSSKL', 'FEDYV', 'VQSMDV', 'LLVQPT', 'ECMEGV', 'CKMYE', 'SPSITY', 'QLFDF', 'TYQPY', 'NKDWI', 'GSALAVR', 'GVWGMK', 'GMKVLQ', 'VLQTRG', 'TRGFVS', 'IREAGGA', 'QLAALR', 'LAALRK', 'KEIER', 'ERLQK', 'RLQKQ', 'KQIER', 'ASRRW', 'LVLLLP', 'RTREN', 'GGQVYPG', 'LHLSKA', 'KAKISK', 'LTDYR', 'KYLVF', 'LVFFF', 'INTEVV', 'PRRQGG', 'GGLGPIR', 'LGPIRI', 'LLSDLN', 'SKDYGV', 'SGHTLR', 'TLNDLP', 'LNDLPV', 'AEDLVE', 'RKETH', 'EELEGV', 'YQHDD', 'SSPVGCK', 'LTEEW', 'TEEWL', 'PNTETN', 'FGKRPA', 'QSKNAGA', 'KNAGAVI', 'AGAVIGKG', 'IGKGGKN', 'GKGGKNI', 'LRTDY', 'RTDYN', 'TDYNAS', 'DYNASV', 'NASVSVP', 'ASVSVPD', 'IGEILK', 'LPLESD', 'LNYQH', 'KGSDFD', 'IHQSLA', 'HQSLAGG', 'IGVKGAK', 'GVKGAKI', 'KELRE', 'NTQTTI', 'ECCPHS', 'STDRVV', 'GKPDRV', 'ILDLIS', 'IKGRAQ', 'FYDET', 'TYDYGG', 'YGGFTM', 'FTMMF', 'RPVGFP', 'PMRGRG', 'DRMPPG', 'MPPGRGG', 'RGGRPM', 'SPRRGP', 'PPPPGRG', 'PPPPRGG', 'GDLMAY', 'YDRRG', 'GMVGFSA', 'FSADET', 'DYSYAG', 'SYAGGRG', 'YAGGRGS', 'AGGRGSY', 'GSYGDLG', 'SYGDLGG', 'YGDLGGP', 'LGGPIIT', 'PIITTQ', 'IITTQV', 'VTIPKD', 'IIGKGGQ', 'QRIKQ', 'SGASIKI', 'GSEDRI', 'LLQNSV', 'FLPLLA', 'LWEPK', 'LYLVCG', 'LVCGER', 'CGERGF', 'FFYTP', 'KSRRE', 'LELGGSP', 'GSPGDLQ', 'QTLALE', 'VDQCCT', 'ESKCPL', 'VLDAVR', 'PAVDVAV', 'ESGELH', 'ELHGLT', 'PFHEF', 'GHRHY', 'HYTIAA', 'NIYTY', 'IYTYD', 'YTYDD', 'YDDME', 'ISEAPR', 'ERRKS', 'RRKSQ', 'VLKQLA', 'HEREV', 'VLQKAL', 'KENRE', 'REANLA', 'AAIIER', 'ERLQE', 'RLQEK', 'EVRRN', 'VRRNK', 'APFLLL', 'PLLMLS', 'LSAVLAV', 'AVLAVPV', 'PPQEDS', 'QEDSQA', 'LETDGH', 'ELKRQ', 'QFEHL', 'TATRDL', 'RYLES', 'ERKLQ', 'RKLQE', 'HREHP', 'REHPK', 'NVPGSQA', 'SQAQLK', 'LKEVW', 'VWEEL', 'EELDGL', 'GLDPNR', 'FFILH', 'MREHV', 'REHVM', 'EFLAST', 'FLASTQ', 'LASTQR', 'TGEGWK', 'GEGWKT', 'MSPAYT', 'SPAYTE', 'YTEEE', 'EELKR', 'AELNAR', 'ELNARA', 'LNARAQ', 'LGRSQD', 'QKREL', 'KRELQ', 'RELQQ', 'LQQAVL', 'RKQQL', 'LQEQSA', 'DAPVPAP', 'VPEQPP', 'LPQLDS', 'LRTALT', 'LTFCSP', 'PRPCVP', 'PSAAAVR', 'GTKLKK', 'ASELYS', 'EVNEAL', 'ALAENPG', 'SCYEDG', 'YEDGW', 'AYEKY', 'EKYVK', 'VAAAAVAV', 'AAVAVAAV', 'GARSAGR', 'AGRWLA', 'GRWLAA', 'AALRSPG', 'ALRSPGA', 'EERDQ', 'IPGLKAA', 'GWSELS', 'YKEFS', 'FKNFN', 'MSRVAL', 'SRVALQ', 'VQITLT', 'ITLTSH', 'TLTSHD', 'LTSHDC', 'TSHDCGG', 'DVKLAQ', 'ASGFNAT', 'EDAQTL', 'DEDAII', 'AIIGILA', 'YKSTIG', 'TIGRDL', 'LKSELS', 'LIEILA', 'IRRIN', 'QTYQQ', 'TYQQQ', 'AAGRDEG', 'LDDALM', 'GEKRW', 'EKRWG', 'VFDEY', 'YKRIS', 'RLYKS', 'TDDNTL', 'VSRAEI', 'LDIRAS', 'SFIKGD', 'LLVLCGG', 'LVLCGGD', 'AVLLFL', 'SDLLLL', 'TPGEATP', 'PGEATPP', 'GEATPPP', 'RKKKD', 'DADMAR', 'ADMARL', 'DDIEEG', 'DIEEGD', 'IEEGDL', 'GDLPEH', 'PEHKR', 'EHKRP', 'KKGKTL', 'KGKTLM', 'VTVSGNP', 'TVSGNPT', 'SLWQGS', 'MLRDGS', 'RDGSYA', 'RCAEVT', 'TLEGQM', 'TKPEKA', 'KKEGDP', 'KEGDPK', 'GDPKPR', 'AGSRRE', 'ILPEKA', 'GSDFLM', 'MKRLQ', 'VTSKLAG', 'KLAGGQV', 'PEGKLE', 'EGKLET', 'GKLETK', 'KAGHPPA', 'KHPHTG', 'KEERD', 'EERDK', 'QEWES', 'GDKDFP', 'KDFPPA', 'AAAQVAH', 'QKPHAS', 'KPHASM', 'ELEKR', 'ELILSP', 'ILSPRS', 'KESVPD', 'PDFPLS', 'DFPLSP', 'LSPPKK', 'SHEAEV', 'EHEKE', 'VLQKAI', 'RKNKE', 'KDPADE', 'GQVKVF', 'FRALY', 'LYTFE', 'EEGDII', 'TDMSDT', 'WWKGT', 'EQAESI', 'ESIDNP', 'LHEAAK', 'KRGNLS', 'LSWLR', 'LDNRVG', 'VGVNGLD', 'NGLDKAG', 'GHKDIV', 'QNKLGD', 'LHAAAW', 'KGYADI', 'KQQGTD', 'QQGTDGA', 'GTDGART', 'CLLSFS', 'DGRTGLG', 'GKKEAAA', 'IHKSW', 'WCGACK', 'EISELS', 'YFYVS', 'AEQVVQ', 'KHFQD', 'HFQDE', 'RLGLRG', 'GAALRLA', 'MPSVCAA', 'QLRSSS', 'DNAPKE', 'SLTLLE', 'TLLEIS', 'PGPVSAAA', 'GPVSAAAP', 'VSAAAPAS', 'PASEAAE', 'EAAEEE', 'KERTH', 'EIKNY', 'NLVQAK', 'VESLPQ', 'EIKANV', 'KAEAEK', 'HFDQE', 'FDQEE', 'QEEIR', 'SLSIEE', 'IEEFM', 'GRVIDI', 'IFDTDG', 'FDTDGN', 'TSQFSV', 'VKGDEE', 'EQKLR', 'AFRIY', 'NLKDW', 'LVDKSI', 'VDKSIL', 'DKSILV', 'KSILVL', 'ELQGMV', 'QGMVNE', 'EIDKDG', 'DGNGTVD', 'TKLGEK', 'QAADTDG', 'RFCRS', 'CRSSSM', 'EALRDA', 'DAATAVE', 'TAVEQE', 'SIQNSQ', 'QDMRQ', 'RQISDG', 'ISDGER', 'EREEL', 'DEVVSK', 'NAKSHL', 'MSLYSA', 'SSEVPPG', 'CALEDQ', 'KKIKR', 'RRLET', 'NIDNSD', 'EHAKGAG', 'SKSLQN', 'SLQNTD', 'VDSRAE', 'ISRLDA', 'DAELVK', 'YKDQI', 'IKKMR', 'LKQKR', 'EQQRD', 'FNMEQ', 'VDAMKL', 'MKLGVK', 'KLGVKE', 'LGVKEM', 'YKEVK', 'QEALGR', 'ALGRSY', 'DELLAD', 'LLADED', 'LDEAASA', 'PTDTKN', 'DEFGLP', 'FGLPQI', 'QDAAIVG', 'VGYKDS', 'TFVSIT', 'TPAEVGV', 'EVGVLVG', 'VGVLVGK', 'SFFVNG', 'VNGLTLG', 'GGQKCSV', 'QKCSVI', 'SVIRDS', 'LLQDGE', 'EGVHGGL', 'GLINKK', 'KKCYE', 'KCYEM', 'HLRRS', 'MRAPAQ', 'GFLLLL', 'LSASLGE', 'ASQDIGS', 'LNWLQ', 'QQEPDG', 'LIYATS', 'SLDSGVP', 'DYYCL', 'LQYASS', 'TFGGGTK', 'PLVVFV', 'KEGKIV', 'IVPVEI', 'PVEITI', 'EITISL', 'TISLLK', 'LKREM', 'KREMD', 'REMDQ', 'QDNLQG', 'KADVSF', 'FVLFF', 'RCLER', 'EKRIQ', 'LESTKP', 'ESTKPI', 'GKVKKI', 'VKKIDA', 'KSVDEV', 'DAQPQL', 'EADDDL', 'PPQKSL', 'TKYKK', 'KYKKT', 'TVTRLS', 'ALKKDT', 'NKDIVS', 'VSGLKY', 'SGLKYV', 'KYVQH', 'HTYRT', 'TGMRVD', 'SYGPRP', 'VEEAPK', 'DHLTW', 'KKDWT', 'ASGSQLL', 'SQLLVL', 'SPLALGI', 'GIPMYS', 'SEETIV', 'EAHDAQ', 'QVLTSE', 'VLTSEK', 'SEKTVL', 'GHLRSV', 'LRSVSI', 'SDKEGH', 'VEKAVM', 'EKAVMV', 'TPGSTVL', 'TVDNNL', 'NNLLPV', 'KTVVIL', 'GIPVKR', 'WKIRA', 'IRAFY', 'VEPTET', 'LEVSII', 'DGDKKI', 'KISLAH', 'SLTRVV', 'VGDAVLT', 'DAVLTR', 'KVLMEG', 'ADALVGK', 'VGKSLY', 'GKSLYV', 'SLYVSV', 'YVSVTV', 'TVILHS', 'PKFFK', 'KFFKP', 'PAMPFD', 'VFVTNP', 'VTQGSNA', 'QGSNAKA', 'KALTQD', 'KLSINT', 'QPLTIT', 'AHPYST', 'STMHNS', 'RMELK', 'MELKPG', 'FHLRT', 'VMNKGK', 'NKGKLL', 'LKAGRQ', 'LPITPE', 'YYTLI', 'SGQREV', 'VADSVW', 'ADSVWV', 'GTLVVKG', 'VKGDPR', 'GDPRDN', 'GARVGLV', 'VGLVAVD', 'VEKADI', 'ADIGCTP', 'NYAGVF', 'LAFKTS', 'FKTSQG', 'QGLQTE', 'GLQTEQ', 'RRRSV', 'QLMER', 'RMDKAG', 'KAGQYT', 'RRARL', 'ENCIKA', 'AFIDCC', 'WTIEE', 'EPEKNG', 'LKDSIT', 'SLSDKK', 'GICVADP', 'CVADPY', 'PYEIR', 'QDFFI', 'LPYSVV', 'YFQTI', 'TIKIPP', 'KIPPKS', 'VIVPLK', 'IGQQEV', 'EVEVKA', 'FNHFI', 'HFISDG', 'KTVAIH', 'AIHTLD', 'PEKLGQ', 'GQGGVQK', 'VPAADLS', 'DLSDQV', 'DQVPDT', 'QVPDTD', 'DTDSET', 'VVQMAE', 'AEDAVDG', 'RLKHL', 'LIVTPAG', 'AGCGEQN', 'HYLDQ', 'IEKRQ', 'AFKQPS', 'NRPPST', 'DSHVLC', 'GAVKWL', 'KWLIL', 'WLILE', 'KQKPDG', 'ADVSLTA', 'FVLIAL', 'VNSLPGS', 'YMNLQ', 'LQRPY', 'WEEPD', 'VEATSY', 'TSYALL', 'ALLLLK', 'VPPVVR', 'VVRWL', 'RYYGGG', 'GGYGSTQ', 'FMVFQ', 'FQALAQ', 'QTDVPD', 'LNMDVS', 'LLWEN', 'EAFSLT', 'FSLTAK', 'LTAKGKG', 'AKGKGRG', 'AKLKSK', 'LKSKVT', 'RPAPET', 'PETAKK', 'ETAKKP', 'TAKKPE', 'SMMTGF', 'DLELLA', 'LLASGVD', 'RYISK', 'YISKY', 'SNKNTL', 'KVHQY', 'QYFNV', 'GLIQPGS', 'YNLEE', 'YHPEK', 'KDDGML', 'SKLCHS', 'CAEENC', 'LDKACE', 'YVYKT', 'FDEYT', 'YLMWG', 'EKPNTS', 'TWVEH', 'VEHWP', 'PEAEEC', 'YQKQC', 'QCEELG', 'MVVYGC', 'VRKSST', 'RKSSTP', 'IILEEG', 'ILVGDVG', 'VGDVGQT', 'GDVGQTV', 'QTVDDP', 'YETKE', 'ENAPLK', 'MIYASS', 'IYASSK', 'YASSKD', 'KKKLTG', 'LTGIKH', 'ELQANC', 'YEEVK', 'DRCTLA', 'TLAEKL', 'LGAPFAP', 'GAPFAPL', 'SRKAGPG', 'GGTRSAAG', 'GSSSGFH', 'TSVSSVS', 'RFRGAA', 'LSNGPEG', 'NGPEGCV', 'PEGCVVA', 'LQALND', 'AGYIDK', 'NRSLEG', 'SLEGEAA', 'LRQQQ', 'RGAVLR', 'LGAARGQ', 'GAARGQL', 'RLEQE', 'LLEDIA', 'VRQRL', 'EAARVE', 'QKKAQA', 'LQEECG', 'EEVGEL', 'ELLGQI', 'LLGQIQ', 'QIQGCGA', 'EARDAL', 'ARDALK', 'TSALRE', 'RAQLEG', 'VQSTLQ', 'QSEEW', 'RLDRL', 'SAQEEI', 'TELEAL', 'ERQRS', 'RQRSE', 'LEDRH', 'NTKWE', 'IGFGPSP', 'GQTEEI', 'IRVTEG', 'EEEGAAA', 'GAAATSPP', 'ATSPPAE', 'PAEEAAS', 'AEEAASP', 'SPGEAKS', 'AKSPAEA', 'AEAKSPA', 'GEAKSPS', 'AKSPATV', 'EAKSPAA', 'PAAVKSP', 'KSPIEV', 'SPIEVK', 'PIEVKS', 'IEVKSP', 'PEKAKT', 'EKAKTP', 'SPVKED', 'AKPPEK', 'KPPEKA', 'PPEKAK', 'PEKAKP', 'KAKPLD', 'DVKSPE', 'PVQEEA', 'AKHPTD', 'AKEKAK', 'EKVAPK', 'KVAPKK', 'VAPKKE', 'VKAKEP', 'KAKEPP', 'AKEPPK', 'PTPKTE', 'PKTEAK', 'SKKDEA', 'PKEAPK', 'KEAPKP', 'EAPKPK', 'TPTEKP', 'KPKDST', 'AEAKKE', 'AKKEEA', 'EEETPA', 'EETPAK', 'LGVKEE', 'TKAKEP', 'DKSLSK', 'SKEPSK', 'PKTEKA', 'PPEKTT', 'LISDLD', 'DLDRR', 'PSLKTM', 'RLAPNP', 'LTPVMI', 'MIRSY', 'IRSYK', 'PPLSPY', 'EEIKR', 'IKRVGP', 'VLMDVV', 'NILCIV', 'IVMAALG', 'ALGPTVL', 'ISSGHIG', 'SSGHIGI', 'GHIGIGI', 'CLALFT', 'INYRT', 'TLTHIS', 'LSSDSY', 'SDSYSL', 'PILMVV', 'ILMVVC', 'CAVYAF', 'AFFILG', 'ILGSTAL', 'YLIFI', 'IQMFM', 'STFRR', 'TFRRS', 'LIKMY', 'HDIRK', 'REKKL', 'EKAGYV', 'YVQSGN', 'QSGNSAL', 'SALAPIV', 'ALAPIVS', 'IVSTFT', 'IFLKR', 'FSVIAM', 'SVIAMF', 'IAILPF', 'ILPFSV', 'KAVAEAS', 'AEASVSL', 'RMKKI', 'PPSYIT', 'PEDPDT', 'DPDTIL', 'PDTILL', 'LTWEQ', 'KSDPPK', 'QIQKR', 'QKRHV', 'RPELY', 'DQGVASP', 'VLGICGN', 'LGICGNV', 'QLQKGV', 'VAVNGPL', 'AWIFH', 'GLQKDL', 'LPYGDL', 'VNLSGGQ', 'PLSAVDA', 'AVDAHVG', 'HVGKHV', 'KKTLKG', 'KTLKGK', 'VVLVTH', 'HQLQF', 'LLEDGE', 'LEDGEI', 'GEICEK', 'EICEKG', 'CEKGTH', 'LMEER', 'EHIYN', 'TLKESP', 'SPAQRD', 'DEGKEP', 'IQTESP', 'ASGGYLV', 'LVSFLV', 'VSFLVL', 'LFFLM', 'WWLGI', 'WLGIW', 'LDRGSQ', 'QNNKTA', 'HNRVF', 'VRSPMS', 'PTGRLM', 'RLPFH', 'QFFMV', 'VFPVVL', 'FPVVLV', 'VLVVLAG', 'LAVIFL', 'AVIFLI', 'GVQELK', 'NISRSP', 'PWFSH', 'SIQGLGV', 'GLGVIHA', 'YDKKD', 'NSSHLL', 'LLYFN', 'IVTFVV', 'LLQVCV', 'TETQAK', 'FTSAEL', 'TFKDY', 'YRDNT', 'QSGQTVG', 'GQTVGIV', 'QTVGIVG', 'LVEPASG', 'EPASGTI', 'ASGTIII', 'IPQDPV', 'PQDPVL', 'LWHVL', 'TFMRD', 'FMRDT', 'QAEVTE', 'NGENFS', 'SVGERQ', 'MARALL', 'ALLRNS', 'ILLDEA', 'SMDSKT', 'KTDTLV', 'TDTLVQ', 'VQSTIK', 'LNTVLN', 'VLNCDL', 'LNCDLV', 'NCDLVL', 'VMENGK', 'MENGKV', 'LAEKPD', 'LLAAEVG', 'WAALLV', 'ALLVLF', 'LLVLFA', 'AASLITV', 'LWLSR', 'LSRADGG', 'ADGGRDS', 'SAAEPGGS', 'GSLREL', 'SLRELG', 'WRSLL', 'LLRLR', 'KSFRE', 'NWQRA', 'LNEQAC', 'GSSIQIA', 'RASISH', 'HVTCVD', 'QLSAEE', 'ISVTQQ', 'LPPTQL', 'TQLEVS', 'GLLVSW', 'LSLKVL', 'TRERD', 'ERDEE', 'RDEEQ', 'QPELST', 'ELSTVE', 'LSTVEE', 'LIKDAI', 'TQPAMM', 'QPAMMV', 'VNLRAC', 'ACSAPGGL', 'LVPSEK', 'PSIPRP', 'SIPRPT', 'LFLRQ', 'FLRQL', 'EELCCA', 'EDLALD', 'GPQSRE', 'SSCGDAE', 'GDAELLG', 'TLPVGSP', 'SPSRPM', 'PSRPMS', 'MSRRQ', 'LTPGPGK', 'KSLSPAA', 'VTAELH', 'EQGSPR', 'TPRPSI', 'PRPSIT', 'RPSITP', 'GTVVTTV', 'PRVDGK', 'LDSPSR', 'RSPSKV', 'LSESSGP', 'ESSGPSN', 'GPSNASH', 'QLTEPS', 'RAAKKT', 'SGVSKVP', 'PIAQDE', 'SLGYAAS', 'LEASMQ', 'PPSDPSA', 'PSDPSAT', 'DPSATSP', 'TSPGPVD', 'PGPVDAL', 'SERPSV', 'ALETRS', 'SFLRSG', 'KLIFR', 'SHDDLS', 'TTATPSV', 'LIKRF', 'NLLGSY', 'LKEPAL', 'KEPALN', 'EPALNE', 'LNEANL', 'KGDEDD', 'GDEDDK', 'DEDDKG', 'DKGPPCG', 'PVNCNE', 'GNNFGVA', 'VAVQEK', 'AVQEKV', 'LLVAIF', 'GDKHTL', 'DAEIAR', 'AEIARL', 'YVAFLG', 'LGALALI', 'FHDFL', 'GILFSH', 'HPRDF', 'FTPVCT', 'TTELGR', 'IDSVED', 'NAYNGE', 'NNMPVT', 'MPVTAR', 'IFGPDK', 'ILYPAT', 'VDSLQL', 'DSLQLT', 'TGTKPVA', 'GTKPVAT', 'VDWKK', 'PTLSEE', 'LSEEEA', 'TKELPS', 'EERQD', 'VAVTGTE', 'WDLQI', 'DLQIAL', 'ALASFY', 'SQATPSS', 'SVSRGTA', 'SRGTAPS', 'GQRFY', 'RFYAGG', 'KGAKEH', 'GAKEHGA', 'VAVERV', 'AVERVT', 'PFAGGGY', 'GERRR', 'KLWKT', 'LRSYQ', 'RSYQD', 'EDHRD', 'DHRDE', 'HRDED', 'FVKPKG', 'FKAFTG', 'KLGSTAP', 'LGSTAPQ', 'PQVLNT', 'ILINEA', 'INEAEP', 'EPTTNI', 'PTTNIQ', 'QKFNH', 'LFIVDA', 'IVDARP', 'PNKELA', 'ADENQT', 'TLKEAN', 'EANLLN', 'VAAALLL', 'AALLLLG', 'ALLLLGA', 'AVRAEE', 'KKEDVG', 'VVGIDLG', 'VGIDLGT', 'GIDLGTT', 'DLGTTY', 'TTYSCV', 'EIIAND', 'IIANDQ', 'AKNQLT', 'NQLTSN', 'NPENTV', 'DPSVQQ', 'PSVQQD', 'TAEAYL', 'GKKVTH', 'AQRQAT', 'KDAGTIA', 'NEPTAAA', 'EPTAAAI', 'AYGLDK', 'GEKNIL', 'VVATNGD', 'VATNGDT', 'RVMEH', 'MEHFI', 'EHFIK', 'YKKKT', 'KKTGKD', 'TGKDVR', 'VLEDSD', 'SDLKKS', 'VKEFF', 'GKEPSR', 'KEPSRG', 'RGINPD', 'DEAVAY', 'GAAVQAGV', 'VQAGVLS', 'GVLSGDQ', 'LVLLDV', 'IETVGGV', 'VVPTKK', 'QIFSTA', 'DNQPTV', 'GERPLT', 'DNHLLG', 'LTGIPPA', 'RGVPQI', 'VNGILR', 'NGILRV', 'ILRVTA', 'AEDKGTG', 'GNKNKI', 'ITITND', 'EEIER', 'EIERM', 'MVNDAE', 'VNDAEK', 'KLKER', 'LKNQIG', 'KNQIGD', 'NQIGDK', 'GGKLSSE', 'WLESH', 'SHQDAD', 'DADIED', 'PIISKL', 'GSGGPPPT', 'SGGPPPTG', 'ETKGIK', 'TKGIKQ', 'GIKQDL', 'YLEDH', 'ANEEDV', 'PIELPV', 'EPPEKA', 'LESKKA', 'RAARFG', 'RFGISS', 'SSVPTKG', 'SDTKPM', 'RAQRF', 'SSISRK', 'SISRKS', 'KLKKR', 'LKKRK', 'SMGIKM', 'QSPKSM', 'KSMSMS', 'MSVGER', 'GERVTL', 'VVTYVS', 'SPKLLI', 'YGASNR', 'GASNRY', 'ATDFTL', 'VQAEDL', 'GQGYSY', 'VARVGAV', 'QYIFT', 'NGKRCP', 'CPTCHN', 'NCKLVT', 'DECRR', 'CRRKL', 'EQYYA', 'IEVDPV', 'EVDPVL', 'GLLIEQ', 'LLIEQG', 'LIEQGI', 'EQGIPK', 'IPKAKL', 'LKEIVA', 'KEIVAD', 'VMLKEG', 'HGIPVF', 'QAGVYH', 'GVYHSN', 'NFMDF', 'FMDFD', 'GELIHV', 'DNSNII', 'NSNIIL', 'SNIILL', 'SQGDLR', 'MADGVAN', 'VANVEH', 'YMDSY', 'ESLEVV', 'VNSILQ', 'EPLGLF', 'VGMDTF', 'TFIHR', 'IYQPR', 'GKYLMG', 'LDSETL', 'DSETLC', 'AVKILK', 'KKKLR', 'LLRRL', 'LYNEE', 'KMYMV', 'MYMVM', 'YMVME', 'MVMEY', 'EYCVCG', 'YCVCGM', 'DSVPEK', 'PEKRF', 'EKRFP', 'CQAHGY', 'GYFRQ', 'FRQLI', 'LEYLH', 'GIVHKD', 'NLLLTT', 'LLLTTN', 'TLKISD', 'ISDLGVA', 'DLGVAEA', 'LGVAEAL', 'FAVDDT', 'DTFSGF', 'GVTLYN', 'YNITTG', 'ITTGLY', 'LYPFE', 'PFEGDN', 'DLHGRA', 'YTQDF', 'GQVLEE', 'LEEEVG', 'EEVGQN', 'QNGQSH', 'QSHSLP', 'SHSLPK', 'TEPQLS', 'GRPGTAN', 'PARKVC', 'RLSACK', 'NGSGLCK', 'RAVFPS', 'HQGVMV', 'KDSYVG', 'SYVGDE', 'DEAQSK', 'SKRGIL', 'RGILTL', 'EHGIIT', 'HSFYN', 'PTLLTE', 'NREKM', 'QIMFE', 'MFETF', 'FNVPAM', 'AMYVAI', 'LYASGR', 'SGDGVTH', 'VTHNVP', 'VPIYEG', 'YALPHA', 'GRDLTD', 'DYLMK', 'YLMKI', 'GYSFVT', 'FVTTAE', 'VTTAER', 'REIVR', 'FENEM', 'ANNVLSG', 'GTTMYP', 'TTMYPG', 'DRMQK', 'KEITAL', 'ITALAPS', 'PSTMKI', 'YSVWI', 'EAGPSIV', 'RDERP', 'PPWRL', 'RLRLL', 'SSAQAAH', 'QAAHIK', 'AAHIKK', 'AHIKKA', 'GGSEAAEG', 'EALEAAA', 'LEAAAAAG', 'GPPGLAR', 'GLARLF', 'VEGRPL', 'RPLWV', 'VLRLTA', 'AGLDAAGP', 'LPGRPQ', 'LVGNMH', 'VLVYLA', 'LARELA', 'LASGYR', 'YRRGD', 'LLNTTD', 'NPDGFE', 'PPGTSGR', 'RDLNR', 'QFSTGE', 'STGEPPS', 'VPEVRA', 'LHGGSVV', 'PFDDSP', 'EHKTTG', 'TGLYSK', 'GLYSKT', 'FRYLA', 'KTGEPH', 'GITNGAH', 'GGMQDY', 'MQDYN', 'DYNYV', 'ITLELS', 'LELSCC', 'CKYPPA', 'PASQLR', 'QLRQE', 'ESLITL', 'KVHIGI', 'VHIGIK', 'TGSGLEN', 'NATISVA', 'ISVAGIN', 'GINHNI', 'GTYNLT', 'LSTGYM', 'PLTINN', 'IMVKEG', 'SLRPTV', 'TVMSVM', 'SVMPGST', 'EAVTTPG', 'VTTPGTV', 'IPPGTPS', 'SSHQPI', 'HHHFP', 'EIFLR', 'SRELY', 'EISDNP', 'VHEPGE', 'PGEPEF', 'NEVVGR', 'VVGREL', 'CKNFGT', 'DPEVTD', 'PEVTDL', 'EVTDLV', 'RIHLM', 'MPSMNP', 'EKSQEG', 'GDSISVV', 'DSISVVG', 'TEPTQP', 'PTQPET', 'WVKAY', 'HGGSLVV', 'NYPYD', 'YPYDD', 'DDNEQG', 'VATYSK', 'ALSYSK', 'GRPCKD', 'KDMYL', 'MYLNE', 'NEYFP', 'FPHGIT', 'HGITNGA', 'SWYNV', 'WYNVP', 'YPFEN', 'IQFMK', 'QVHQGV', 'VHQGVK', 'NATLSVA', 'NHPVTT', 'ITASARG', 'SARGYN', 'ARGYNP', 'NPVTKN', 'NVTVRS', 'AVQVNF', 'SSADANN', 'DPTSKE', 'IKHLSA', 'HLSAEN', 'FMLSSS', 'ISNKPN', 'SNKPNI', 'NKPNIS', 'VAGIHGN', 'ELLLAL', 'YKRNP', 'VTQLVD', 'LNPDGR', 'QPETKA', 'ETKAII', 'TKAIIE', 'AIIENL', 'KPVQTV', 'PVQTVE', 'LKHLAS', 'KHLASL', 'PNNSDE', 'SDENIP', 'PGGVMRG', 'HLGSMK', 'TYGHCP', 'TVYTSC', 'VYTSCC', 'LPALWA', 'AENKKS', 'KKSLLS', 'LLSMLV', 'SMLVEV', 'VEVHKG', 'EVHKGV', 'VHKGVH', 'HGLVKD', 'LVKDKA', 'LNEGIK', 'VLLAPGV', 'VHNINA', 'INAIADG', 'ASSVVIV', 'LVVTVSG', 'CICSIK', 'NRHKD', 'DEYED', 'YEDEI', 'SKKSLL', 'HEFQD', 'EFQDE', 'RAACASV', 'ASVPFR', 'LVCGTGAG', 'TGAGTAIS', 'GAGTAISP', 'TAISPW', 'AISPWT', 'SPWTPS', 'TPSPRL', 'AGPGRPL', 'LLDVVI', 'STCHLI', 'EDHIY', 'DHIYE', 'YEKLD', 'AITDEE', 'DMLDLA', 'RLGCQV', 'CLTKAM', 'VRVPEA', 'RVPEAV', 'ADVRQS', 'ITLVVE', 'LVVETL', 'VVETLL', 'MQDKF', 'KFQIM', 'IDDMSS', 'KNIADL', 'NIADLM', 'ADLMTQ', 'DLMTQA', 'TVVSRI', 'LSLVAVA', 'VAVAAAVA', 'AVAAAVAT', 'VATEQQ', 'ATEQQV', 'QVPLVL', 'VADTHE', 'DFTAYG', 'AYGGVFG', 'NLENAL', 'LENALD', 'ENALDL', 'DWYAI', 'YAISTL', 'SPLHVD', 'LHVDLA', 'LATLKE', 'ATLKEL', 'REVLTG', 'KSEDVP', 'VARDIT', 'TQVASPA', 'PPVSYN', 'SYNDTA', 'NDTAPR', 'DTAPRI', 'TAPRIL', 'KDLTSL', 'SFWND', 'FWNDS', 'WNDSF', 'YEPLF', 'ATVTFK', 'FKFIL', 'SRFYP', 'RYWF', 'MERLE', 'RLEIH', 'HSNGSVA', 'SNGSVAH', 'SVAHFN', 'AHFNVS', 'SQVTGPS', 'KKGNLL', 'NLLVTN', 'WQMTL', 'TGEQFS', 'EQFSY', 'ASDCAGF', 'FFSPGI', 'GIWMGL', 'LFMLF', 'FMLFI', 'MLFIF', 'ILSLKT', 'TMDRF', 'DRFDD', 'DDHKGP', 'TITLTQ', 'DDDPVD', 'YKVFD', 'VFDPSC', 'VETDLV', 'PNFVVT', 'DRNHE', 'VDRER', 'ADREEG', 'ATSRKD', 'RKDEE', 'TGLPKR', 'TAAGPLF', 'FQQRP', 'LRANAE', 'EEVSVL', 'EEFDK', 'KDTEGK', 'GKSWCP', 'PVIREG', 'FIYCQ', 'RQKLK', 'KLKITA', 'VPTLLK', 'VESECC', 'MAGRKL', 'KLALKT', 'LSEKPP', 'PAIDWA', 'IDWAY', 'DFEKK', 'LKIPVP', 'IPVPED', 'ALVDQE', 'GSQLRI', 'LEKMR', 'EKMRN', 'TMSDKP', 'DKPDMA', 'PDMAEI', 'EKFDK', 'PLPSKE', 'LPSKET', 'PSKETI', 'KTIYF', 'IYFVAG', 'YFVAGL', 'ALQDTE', 'PRSFPA', 'SQTEAH', 'HSQGTF', 'DYSKY', 'YSKYL', 'VQWLM', 'TKRNR', 'RNRNN', 'HDEFE', 'RHAEGT', 'TFTSDV', 'AWLVKG', 'VKGRGR', 'AEELGR', 'SDEMST', 'NWLIQ', 'LDGDAW', 'TNRWV', 'RWVES', 'GDLEKD', 'TSQDAR', 'DARFY', 'RFYAL', 'FYALSA', 'KFEPF', 'DCGGGYV', 'YVKLF', 'GLDQKD', 'MHGDSE', 'EYNIM', 'NIMFGP', 'ICGPGTK', 'GTKKVH', 'VKIDNS', 'IDNSQV', 'LEDDW', 'WDFLP', 'KPEDW', 'WDERA', 'IDDPTD', 'TDSKPE', 'WDKPE', 'DPDAKK', 'PPVIQN', 'PVIQNP', 'AYDSFA', 'LDLWQ', 'LWQVK', 'GVTKAAE', 'KAAEKQ', 'AAEKQM', 'DEEQR', 'EEQRL', 'EQRLK', 'QRLKE', 'RLKEE', 'EDKKR', 'DKKRK', 'DMATCM', 'AVTEQGA', 'RRSAW', 'RSAWR', 'TTVLEL', 'TNPESK', 'NPESKV', 'SVQQLP', 'LPLVLL', 'LMLLLA', 'ARLYF', 'DQLALL', 'YPRPH', 'RNQHI', 'AMADRI', 'IKRKGA', 'ILLSVQ', 'LLSVQN', 'GGNDLPV', 'VWEYA', 'KHGIPD', 'HGIPDE', 'IPDETC', 'PDETCN', 'FNQCGT', 'TLWRV', 'ATEMMS', 'MSNYTG', 'YTGGIY', 'QDQAVI', 'AVINHI', 'SVAGWGV', 'VAGWGVS', 'WGVSND', 'GIEYW', 'WGEPW', 'GEPWGE', 'PWGEKG', 'KGWMR', 'RIVTST', 'GDSYNL', 'DRMKK', 'KIKRQ', 'EQIGLD', 'QIGLDE', 'GGGGGSDLG', 'RSVRGP', 'DMKMGS', 'MKMGSD', 'DEVQSP', 'VRMRN', 'ISTEDI', 'STEDIN', 'KLTLNS', 'KLDKLG', 'LDKLGE', 'SKLTDN', 'LTDNLV', 'LVALKE', 'VALKEI', 'IRLEH', 'EHEEGA', 'AIREVS', 'VSLLKD', 'LVFEY', 'VFEYL', 'EYLDK', 'DLKQY', 'KQYLD', 'KLFLF', 'LFLFQ', 'FLFQL', 'FQLLR', 'CHRQK', 'HRQKV', 'HRDLK', 'RAKSIP', 'KSIPTK', 'NEVVTL', 'PPDILL', 'STDYST', 'CIFYE', 'YEMATG', 'EQLHF', 'LHFIF', 'TPTEET', 'WPGILS', 'RAEALL', 'EALLSH', 'KHPFF', 'LSLGER', 'SLGERI', 'RIHKL', 'KLPDTT', 'NIRSTS', 'RSTSMP', 'MAEYGT', 'LTNNIT', 'TNNITL', 'EQLKSA', 'ACKEDI', 'PSEKSE', 'TGSAWF', 'YIEHI', 'EHIFE', 'LLTMVV', 'LTMVVD', 'RIPSAK', 'KYKDI', 'TGSWIL', 'GSWILV', 'ILVLVL', 'LVLVLN', 'PIQAFP', 'FPKPEG', 'SQDKSL', 'LHNRE', 'HNREL', 'LSAERP', 'SAERPL', 'KKAFPS', 'ESKPSE', 'SPPFDN', 'DHKFQ', 'HKFQD', 'HQLDGT', 'DGTPLTA', 'VHKIAT', 'IYEEN', 'YEEND', 'IVSKLL', 'VSKLLN', 'TESQAH', 'ESQAHT', 'LEDEVA', 'NNYEE', 'TLDKPT', 'NQDGKI', 'GKIPEK', 'PEKVTP', 'EKVTPV', 'SNGLER', 'LERRT', 'ERRTN', 'HREDD', 'LQYFP', 'QYFPN', 'YALLTS', 'DSEKEA', 'MMVKY', 'KYGTIS', 'TISPEE', 'EGVSYL', 'GVSYLE', 'ETIALQ', 'IALQTK', 'ALQTKN', 'DSKSKL', 'KLFPAP', 'KEEAAK', 'EEAAKM', 'EAAKME', 'AAKMEK', 'DDNSNL', 'DNSNLGG', 'SNLGGKT', 'GGKTDEA', 'ATGKTEA', 'GKTEAY', 'NIEWL', 'NKEDY', 'KEDYD', 'ADAYVE', 'AYVEKG', 'EKGILD', 'KGILDK', 'GILDKE', 'ANAIKR', 'KRIYS', 'RLVLGL', 'LVSCTF', 'LYSSSD', 'YSSSDD', 'SDDVIE', 'VIELTP', 'VVKVGAV', 'GAVNADK', 'KHQSLG', 'YGVQGF', 'ANKNKP', 'EDYQGG', 'GEAIVDA', 'AIVDAAL', 'DRLGGR', 'YSSGKQ', 'SKKDVV', 'DVVELT', 'VVELTD', 'KNVLDS', 'NVLDSE', 'VLDSED', 'LDSEDV', 'LEPEW', 'ATEVKE', 'KGKVKL', 'TVNQVL', 'QVLASR', 'LASRYG', 'ASRYGI', 'YDGGRT', 'RSDIVS', 'SDIVSR', 'SRALDL', 'INEDIA', 'DIAKKT', 'AGRNSY', 'LLKLAD', 'LWTEAG', 'YELEN', 'ENALGIG', 'GIGGFGY', 'GGFGYPA', 'AINARK', 'FLREL', 'FPTITP', 'WDGKDG', 'DDIDLS', 'DIDLSD', 'DLSDVE', 'LSDVEL', 'SDVELD', 'VPPKMP', 'PPKMPV', 'KMPVQA', 'LPKLPV', 'LQQTLA', 'VKRFGA', 'KRFGAP', 'FGAPGGLG', 'GETLQE', 'LERQE', 'TANWVS', 'WLNDM', 'RLALPV', 'PVNSSPA', 'VNSSPAV', 'AVIFAR', 'QHFQD', 'GVLSYK', 'TDWAKG', 'CMKQY', 'TQDTLV', 'SIMPEP', 'EHVIVA', 'LDVVIN', 'RLSEGD', 'SEGDLF', 'ARTVLL', 'VLLKDS', 'LLKDST', 'LKDSTN', 'GGCSLNGA', 'CSLNGAN', 'VVCEHS', 'VCEHSP', 'IVLVQC', 'VRVDSV', 'PETQGH', 'SAEKLQ', 'EKLQR', 'FIVYK', 'IVYKF', 'VYKFD', 'QKCSPD', 'PDGFIQ', 'IQVALQ', 'QVALQL', 'PTYESA', 'YESASI', 'GRVDNI', 'DNIRSA', 'ATPEALA', 'ASEKLQ', 'AITGMAI', 'ALRELA', 'EMFMD', 'MFMDE', 'FILSTS', 'QVPTTM', 'VPTTME', 'VPNGYGA', 'AITFCI', 'SFHGCK', 'HGCKET', 'TSSVEF', 'CSSRQP', 'SRQPAD', 'RQPADS', 'TAKERA', 'MLEGQT', 'LEGQTQ', 'QTQVAE', 'SEYGLT', 'SSKQKV', 'VDLQSL', 'DLQSLP', 'LDQTVV', 'QGLAVLA', 'GLAVLAK', 'PPNPIE', 'TPARLD', 'FDKEE', 'ETELDA', 'FFDHL', 'HLLAKS', 'SHNVSK', 'VSKEGR', 'LDNSVE', 'EFMQI', 'SSGFISA', 'LFLHH', 'SEAELE', 'MMKIF', 'DKNKDG', 'FLLQF', 'LQFKM', 'QFKMD', 'DVSKTGA', 'VSKTGAL', 'KTGALEG', 'GALEGPE', 'ELVQPS', 'PSISGVD', 'FREIL', 'NKDGKI', 'QKSELA', 'AFRKF', 'LVERSA', 'TVTKGGI', 'EKSQGK', 'QGKVLQ', 'VAVGSGGK', 'GSGGKGKS', 'SGGKGKSG', 'EIEPVS', 'GGTKVVL', 'TKVVLD', 'CMKSVT', 'MAREY', 'VLSLLE', 'FLIPNA', 'SQPESK', 'EVAAGDD', 'VAAGDDK', 'DDKKGI', 'DQSQQA', 'QYCFL', 'LLVPCM', 'LVPCML', 'MLTALE', 'IEPPDT', 'DEYLK', 'VLETDP', 'DPHFR', 'RSGRLS', 'ELDLVS', 'LDALQD', 'DTGMNH', 'TADLEQ', 'DLEQY', 'LEQYD', 'TRHEE', 'EMMKE', 'MMKEH', 'KTFFK', 'RELEK', 'NPQNAE', 'PQNAED', 'ELKKR', 'GIAPSGPA', 'IAPSGPAG', 'GPAGELK', 'IREANA', 'ANAHLAA', 'LAAVHR', 'RLLAAE', 'LLAAER', 'DQHLR', 'RAALDE', 'ELGRAK', 'SALQEQ', 'EQLLSS', 'QLLSSE', 'VRSLQA', 'QRDQM', 'ADLLQD', 'TRHRP', 'PLAALLA', 'LAALLAT', 'AALLATL', 'LATLEE', 'LLPDGPG', 'GPPLGNN', 'GQDDQD', 'DDQQPA', 'MSSNEC', 'ARECPT', 'DICYR', 'GESGHLA', 'YNCGRG', 'GRGGHIA', 'IAKDCK', 'GHLARD', 'DCDHAD', 'EQKCY', 'VKCYR', 'KCYRC', 'GETGHVA', 'KTSEVN', 'DAWRE', 'AWRED', 'IPLSEL', 'SELPEP', 'GLHENV', 'GSSCSQT', 'SQTPSR', 'RRVALG', 'GVQLPPG', 'TLFSTT', 'KFLME', 'CRNSPV', 'TPPKDL', 'PPKDLP', 'LPAIPGV', 'IPGVTSP', 'SDEPPM', 'EPPMQA', 'EDKRAG', 'AGGEESQ', 'QFEMD', 'LSFVLV', 'SFVLVC', 'QEPIW', 'AAEVAVI', 'FFQDL', 'FGISNH', 'SNHSEV', 'LNAEDI', 'IENLDA', 'RFIHV', 'NLHWV', 'HWVTE', 'PMIAAGL', 'AAKLFQ', 'GQILFV', 'ENGKVM', 'MSYFK', 'DDKWD', 'VTVEKV', 'RGFCEG', 'LKGLLQ', 'LQRDH', 'NPNCAR', 'RCGKIV', 'GKIVYP', 'NMKNY', 'HYPKQ', 'RLKQQ', 'QRIKK', 'NIKYH', 'FEKSR', 'KSRMGP', 'SYGGYK', 'EPAAPVS', 'IQRSAP', 'QRSAPGG', 'YRAVY', 'ADEDEV', 'SFQDGD', 'TIVNVQ', 'QQIDDG', 'DGWMY', 'VERTGD', 'ERTGDT', 'LPANYV', 'MASPAPP', 'ELPGPAS', 'LYSRM', 'ALEGER', 'GVAFFE', 'EAAVQVG', 'DLKTLS', 'TLSAIY', 'SAIYSQ', 'NAYFY', 'AYFYL', 'KEYAR', 'LQFHK', 'DRMGEA', 'EAKASGN', 'QRHLD', 'EQGDKV', 'GDKVGEA', 'VYHAKG', 'HAKGKQ', 'SWNAAQ', 'NAAQDPG', 'HLPPDV', 'SEFYE', 'NLSLVK', 'GDRAAQG', 'FTEATT', 'TEATTF', 'ERLAIA', 'RLAIAK', 'AAERRA', 'YSNLGN', 'HYKKT', 'LGNTYT', 'GNTYTL', 'LQDYE', 'HLRHL', 'LVIAQE', 'RVGEGR', 'SMGSPAQ', 'ARMNIA', 'SPAAAEK', 'AEKPDL', 'EKPDLA', 'DLAGYE', 'RTQRL', 'TQRLSA', 'RLSAET', 'EQNGET', 'THHTGD', 'MRSRK', 'IERRP', 'REGSHS', 'SSDEEC', 'EECFF', 'PLEEGQ', 'GQAGAAEA', 'TAAPSVE', 'SVEDRA', 'TEEFF', 'EFFDL', 'RRLDD', 'DQRASV', 'LPGLRI', 'GLRITL', 'ITLNNV', 'GDAQEPG', 'QEPGDE', 'DEFFN', 'FFNML', 'MLIKY', 'DEDFF', 'KRMDE', 'RMDEQ', 'MDEQR', 'TIFTDA', 'ESSTVF', 'SSTVFE', 'YKDDQ', 'PPELPD', 'PELPDV', 'GSANEQA', 'QWAGPL', 'EPPQHA', 'VTVDEL', 'TPTQVM', 'NRPSSI', 'PSSISW', 'DAPSRK', 'APSRKD', 'PKFRE', 'NMKGND', 'MKGNDI', 'ISSGTVL', 'YVGSGPP', 'PSGTGLH', 'VYEQE', 'EQPLSC', 'DEPILS', 'EPILSN', 'PILSNK', 'KSGDNR', 'GAPVAGTC', 'LVDNLT', 'ASQQEI', 'RHGPEA', 'KDFHQ', 'LIQECA', 'FISTLS', 'TLSYAI', 'EVIFGL', 'SSPDLR', 'AAQFIK', 'IDADVSG', 'DADVSGN', 'SHLLFG', 'VGQEQI', 'FPQER', 'APLLYP', 'DFMQE', 'EVGYGF', 'FCASIE', 'VREVTA', 'RTHSGL', 'GLTDGIP', 'IPLQSI', 'SISAPGSG', 'VLIDVL', 'ELNPSL', 'YELDH', 'ELDHPG', 'IRDSKG', 'RDSKGL', 'KGLHNV', 'WKHAE', 'KHAEGQ', 'SLINPE', 'SLLRLA', 'YEQVK', 'CPDMLV', 'LALLQI', 'ALLQIN', 'LQINTS', 'LRHEL', 'RHELI', 'ELISTL', 'LISTLM', 'MPIFLG', 'IMHAMA', 'VAQDLK', 'AFVIDL', 'FVIDLA', 'ALASRR', 'LDKWL', 'PFIQAC', 'LKRRC', 'GGLAPEK', 'LAPEKD', 'AGSVSQE', 'LSETIL', 'TILTMV', 'NCSNVM', 'CSNVMN', 'NVMNKA', 'GRPPSAS', 'PSASSLD', 'ASSLDAI', 'PVQIDP', 'DPLAGMA', 'LSIGGSAA', 'MQGFPP', 'KAFPPL', 'SGIGGLSS', 'SQLPGGL', 'LPGGLGTG', 'FVQRK', 'KLGTSGL', 'GTSGLNQ', 'SKMKPS', 'IDDEAN', 'PHPTMS', 'HPTMSV', 'ITACLF', 'TACLFGG', 'GGIIEKG', 'IIEKGL', 'YMALGL', 'FKNRL', 'YPQYC', 'QYCQH', 'QHLASI', 'FMQFP', 'SRDPPV', 'SIALAQA', 'ALAQAQA', 'PAKAPLA', 'INTTNI', 'NTTNID', 'TNIDTL', 'IDTLLV', 'SQSNMT', 'EFNKM', 'TYRNI', 'LLTSDK', 'SDKAAAN', 'MITLAK', 'TLAKNK', 'LAKNKP', 'AKNKPI', 'DLDVKS', 'LDVKSL', 'DVKSLL', 'VKSLLL', 'EAYVKG', 'FVAKVL', 'SIRSLV', 'MNVLAE', 'KNLALD', 'ALDINE', 'EELPAI', 'NVYSLA', 'PVVDRS', 'IKIAMT', 'DFALDS', 'NLTAGMA', 'MAMITC', 'PLLMSI', 'ISTNLK', 'STNLKN', 'TNLKNS', 'QAAAQLA', 'AAAQLAQ', 'LACCFI', 'QKTAVE', 'KTAVEK', 'TAVEKAG', 'AVEKAGP', 'GPEMDK', 'EMDKR', 'RKHAR', 'KVGGVDP', 'VGGVDPK', 'DPKQLA', 'LPTNDL', 'TNDLSQ', 'NDLSQP', 'AWATDD', 'QIYDK', 'LAMNPQ', 'ALRSLL', 'LLEVVV', 'LEVVVL', 'VVVLSR', 'VLSRNS', 'NSRDAI', 'IAALGLL', 'LGLLQK', 'LLQKAV', 'GLLDATS', 'LLDATSG', 'GADADLL', 'LLLRY', 'VLKALQ', 'WCNKQ', 'EAVELL', 'QYDLH', 'LHLAQS', 'LAQSME', 'MENGLN', 'GLNYMA', 'NYMAVA', 'DERSVA', 'VAHITE', 'RSNYE', 'AMIDRA', 'IDRAHG', 'MMHSGI', 'PPGLRE', 'GLREKA', 'YHSAAAG', 'HSAAAGR', 'AAGRDST', 'RDSTKA', 'DSTKAF', 'KAFSAF', 'QMHQQ', 'TDDLIT', 'EISYR', 'AQAEQQ', 'NTVTKI', 'TVTKIN', 'LLELNA', 'ELNAPE', 'LTAFCN', 'RPTKAP', 'WLELI', 'IARMLA', 'MLAHTP', 'LAHTPQ', 'WPMYA', 'AQLLID', 'MQILY', 'KGTLRV', 'VLLVLL', 'PPNCIQ', 'PRNMR', 'PDPFTP', 'TPNLKV', 'APRILT', 'NFTGVM', 'NLQVSN', 'QVSNEP', 'LQLINA', 'QLINAL', 'ALVLYV', 'SAHMDI', 'YLFLN', 'QLRYP', 'NTEAIQ', 'HEFVH', 'FVHCAP', 'HCAPEI', 'QCCMGQ', 'CCMGQK', 'ASLSRAA', 'SLSRAAA', 'RSAGGGGL', 'SAGGGGLPG', 'LPGAGVR', 'SGGQAEQ', 'DALVKK', 'ALVKKD', 'NVLDDP', 'GEFVGGC', 'VGGCDIL', 'LKKLGI', 'KLGIRS', 'ALVDEK', 'RSAVSAS', 'NKLGLM', 'DVKEAI', 'RRLPE', 'TMRHQ', 'RHQIL', 'PKDQW', 'KDQWT', 'DQWTK', 'TKYEE', 'EEDKF', 'FYLEP', 'VIRER', 'RRCTAA', 'AAARGLL', 'ARGLLH', 'ASAPSPAA', 'SAPSPAAA', 'HGSHET', 'TYFNK', 'YFNKP', 'AWELR', 'NTLVGY', 'TLVGYD', 'DLVPEP', 'RRLND', 'ISTPEE', 'STPEEL', 'EELGLD', 'ELGLDK', 'DLSSHN', 'LSSHNN', 'VLTPEL', 'RAKCTP', 'GFTLDD', 'LDDAIQ', 'YDVFK', 'FKDLF', 'TDLNPD', 'SRVRTG', 'VRTGRS', 'CLPPHC', 'RGERR', 'IEKLAV', 'AVEALSS', 'LDGDLSG', 'SMTEAE', 'DHFLF', 'HFLFD', 'FLFDK', 'PLLLASG', 'MARDW', 'RGIWH', 'FLVWI', 'EDHLR', 'GNMKEV', 'VFTRF', 'FTRFC', 'WNPHL', 'GYILTC', 'TGLRAGV', 'GVHIKL', 'KRGTGGV', 'GTGGVDTA', 'SNADRL', 'GFSEVE', 'SEVELV', 'MEQRL', 'EQRLE', 'QRLEQ', 'QAIDDL', 'AIDDLM', 'DLMPAQ', 'SVDLEL', 'LQAKVI', 'QAKVID', 'KVKLAD', 'LADIQI', 'RTKKH', 'TLVDET', 'GVGRMF', 'SYLER', 'YLERS', 'PNLRAV', 'AVGWNE', 'WNEVE', 'FKDFN', 'AFGFMT', 'VHITLS', 'HITLST', 'ITLSTH', 'GLSERD', 'DHDTST', 'TENADE', 'ADESNH', 'VSLPEQ', 'ELFKM', 'SENDLP', 'LPEWK', 'WKERG', 'KHKEK', 'KEKGTI', 'GTIRLL', 'RRDKT', 'ELKPNA', 'AWVWN', 'HADFAD', 'ADFADE', 'KPELLA', 'FKTKF', 'RKEIE', 'EIEER', 'IEERE', 'DNAEKV', 'EALSVR', 'ALSVRE', 'LSVREA', 'EQMNGP', 'RQENGA', 'NGASVIL', 'SLAGSSGP', 'TSLEVE', 'LRGVVQ', 'RGVVQD', 'QQAISK', 'LSSLEK', 'TQHVSP', 'QVEPPT', 'VEPPTK', 'KKGATPA', 'KGATPAE', 'PAEDDE', 'LFGSDE', 'AEKKAK', 'KAKKPT', 'TDMAQL', 'GASKLVP', 'IEHGIV', 'TFYNE', 'PVLLTE', 'GIVMDSG', 'THTVPI', 'GYSFTT', 'FTTTAE', 'DFEQE', 'YDESGP', 'FDQLF', 'DDESDP', 'KEAGGGGV', 'VGGPGAKS', 'SNAAGKQ', 'AGKQLR', 'KQLRK', 'QLRKE', 'QKDRK', 'PLPPSVG', 'LPPSVGV', 'SVGVADK', 'ETQPPV', 'RRPDQ', 'LIDRR', 'RFEKP', 'PLEEKG', 'ERPIR', 'RGRGGLG', 'GRGGLGR', 'RGGLGRG', 'GGLGRGR', 'GLGRGRG', 'LGRGRGG', 'SGLKHE', 'DKRGGSG', 'ELTESP', 'LTESPK', 'LDQSNV', 'QSNVTE', 'SNVTEE', 'GEEHPV', 'EHPVAD', 'ADTENK', 'EAHAED', 'NDITSQ', 'DITSQL', 'GDLGRPG', 'GGRGGRGG', 'ASAPDVD', 'DPEAFP', 'PTMPLL', 'LAALAVL', 'ALAVLAL', 'LAVLALA', 'PSPSCSP', 'CSPGPDP', 'VYTHSA', 'LRTCAD', 'AAYNHT', 'EYMLL', 'LRLLGC', 'RPCRH', 'VCEGLR', 'EGLREA', 'MAWPY', 'GCYDPL', 'LDVEEA', 'DVEEAL', 'PSGLPPT', 'RFAHH', 'RVLKR', 'RTAARC', 'VAKTYS', 'KTYSIG', 'SIGRSF', 'LIGNIH', 'IHGNEV', 'YLLGNP', 'LLGNPR', 'LGNPRI', 'PRIQR', 'LLPSMN', 'EGAGYNG', 'AGYNGW', 'GRQNAQ', 'NAQNLD', 'AQNLDL', 'STRGVR', 'ISQYY', 'WWGKV', 'KWIQT', 'IPFVLS', 'LVVSYP', 'HEKKM', 'DEKMF', 'KMFKL', 'KLLARA', 'AYADVH', 'ADVHPM', 'NFLKR', 'LKRGSI', 'KRGSII', 'INGADW', 'GMSDFN', 'DFNYL', 'LGCVKF', 'EMVHR', 'HRGIKG', 'TTAPDGD', 'LRMKK', 'GPKNFL', 'KNFLPG', 'QMDFE', 'QPASGSK', 'PWWW', 'TSLSPY', 'RQNYS', 'AVNRLV', 'EGAERL', 'RLLEF', 'NDRGGR', 'DVQKPS', 'QEAMEA', 'AMEAALA', 'NLNQAL', 'LNQALL', 'NQALLD', 'QALLDL', 'LHALGSA', 'ALGSART', 'HYLDK', 'YLDKE', 'MGNHLT', 'LRRVAG', 'AGPQPAQ', 'QPAQTGA', 'AQTGAPQ', 'LFERL', 'KAKTKT', 'QRATSN', 'SNVFAM', 'HDMLAS', 'DAYLDA', 'TDPEDV', 'NAFACF', 'AFACFD', 'IQEDY', 'QEDYL', 'RFTDE', 'EAPIDK', 'APIDKK', 'IDKKGN', 'NFNYI', 'YIEFT', 'RILKH', 'EEEYV', 'GFSDED', 'NLDCPD', 'LDCPDL', 'DCPDLI', 'SQKTAH', 'TAHETD', 'EGGKRK', 'GKRKAD', 'DSDSED', 'KGEESK', 'SEKPRG', 'KPRGFA', 'IIGATDS', 'MFLMK', 'VKCPQV', 'KCPQVV', 'SFYEE', 'PSEDDD', 'LAALVLA', 'AALVLAL', 'ALVLALA', 'QTPRPA', 'TPRPAE', 'PIYLAV', 'VFDVTS', 'TSGKEF', 'EFYGR', 'FYGRGA', 'RGAPYN', 'LAGKDSS', 'ADLTHD', 'LTAKEL', 'LEALDD', 'YKAKY', 'KYPIVG', 'VGYTAR', 'YTARR', 'RRILN', 'MAAAAGVS', 'AAAAGVSGA', 'AAAGVSGAA', 'GLLLLF', 'LLLFAP', 'FAPHGGS', 'FYKVI', 'KVIPKS', 'VIPKSK', 'FDTQY', 'TQYPY', 'SSEELL', 'EELLVA', 'ELLVAE', 'VAEVGIS', 'ISDYGD', 'GDKLNM', 'KYKLD', 'YKLDK', 'AIQRW', 'GMPGCLP', 'ARQAIL', 'LSVKET', 'YLKIM', 'IMGKIL', 'MGKILD', 'GKILDQ', 'KILDQG', 'ILDQGE', 'LDQGED', 'EMARIG', 'MARIGK', 'IGKLIE', 'GKLIEN', 'ALWMR', 'LWESH', 'VAQLEL', 'QLELGGG', 'GMSREV', 'YDEEL', 'ELEER', 'QCGPDVG', 'DRGRLG', 'GRLGFQ', 'QVWLK', 'VILSKL', 'ILSKLV', 'LSKLVN', 'KLVNSL', 'KPVKVP', 'PVKVPE', 'ENPPSM', 'PPSMVF', 'AQFLKA', 'QFLKAA', 'MAAVQR', 'GSLAVTK', 'DPNWF', 'KRDFT', 'KHVIGL', 'HVIGLQ', 'RPRQI', 'VSSARF', 'ARFRK', 'DEYDE', 'EYDEN', 'YDENK', 'EEDGGDG', 'EDGGDGQ', 'DGGDGQAG', 'GQAGPDE', 'QAGPDEG', 'AGPDEGE', 'GNMTAAL', 'PPINTK', 'LKVLIS', 'DIEKAV', 'LLMKY', 'LMKYI', 'PSDNSSA', 'WHEKA', 'HEKALA', 'MASGGGVP', 'GGVPTDE', 'EREIM', 'GLDPYN', 'KRIVGC', 'LHKGES', 'RIIRD', 'STSLHK', 'KFSYK', 'KASLLGS', 'ASLLGSK', 'KGANDGI', 'AFIRY', 'ADTLQQ', 'TSDWY', 'LAPRGY', 'APRGYD', 'YDLDAA', 'LDAAWP', 'KPPSRS', 'LNNEAL', 'REKQL', 'GQLQVE', 'LVAELQ', 'QKQGDV', 'GDVSQAT', 'QATVKK', 'CLQALE', 'AKELDT', 'SLGREN', 'PPDTEL', 'EGENTE', 'LVKDVV', 'KDVVPL', 'DVVPLQ', 'SGKKQE', 'GKKQES', 'QESAQL', 'RQLQE', 'ESLAHL', 'HLSSVE', 'QEKQH', 'TWQLQ', 'ETQLGQ', 'QVSQLV', 'QLVSDL', 'LVSDLE', 'LMQER', 'MQERD', 'ERDHL', 'QRVGTL', 'RVGTLE', 'TLEQLA', 'LREEQ', 'NQLLEG', 'LLEGKL', 'LQALQT', 'QALQTD', 'LQQRE', 'EAAIQGS', 'IQGSLAS', 'QGSLASL', 'GSLASLE', 'SLEAEQ', 'QMEASL', 'LLAVKK', 'TMKAQV', 'ALQSKE', 'RLQEE', 'EAQAQE', 'ALENQC', 'LIEVLS', 'LPQVNT', 'PQVNTD', 'SQAQLE', 'HQGEAQ', 'GEAQRL', 'VDLQAK', 'KLQVAL', 'REHKT', 'EALNRA', 'EQELR', 'LRALQA', 'QAELSQ', 'AHLEHA', 'DQLHR', 'RANTDT', 'LTAEKD', 'EKDRM', 'KDRME', 'EALASLA', 'ALASLAQ', 'QDSKEA', 'LQERK', 'ASSFSGL', 'EQLAQS', 'AQSLQE', 'DHQNR', 'HQNRL', 'LKTANE', 'TKEAVQ', 'LEITKA', 'KAAMEE', 'AAMEEK', 'TNRER', 'MLADLD', 'QQKLSA', 'QKLSAE', 'KLSAEE', 'LSAEEK', 'EEKCLG', 'CLGDME', 'KREFS', 'CFQKF', 'GTSQGEP', 'SPQSIGS', 'QSIGSQG', 'NSVCRP', 'RPPDDA', 'AVFDII', 'ESGSSLP', 'SGSSLPE', 'PETPTE', 'ETPTET', 'DSMDPN', 'NTAEQD', 'LTPEDT', 'DTEDVP', 'QDAEIC', 'ICLLKS', 'KLPLTV', 'LPLTVE', 'GLTISW', 'KSISFS', 'TEDTPL', 'TPLDQC', 'CKVLIP', 'KVLIPT', 'KVRIPG', 'IPGIYL', 'PGIYLL', 'YLLIF', 'FSRFI', 'IYDGSD', 'KPVPLD', 'PVPLDE', 'NEYGPV', 'EYGPVL', 'PTSKEN', 'LKEKGP', 'EKGPQN', 'NATDSY', 'YPHKQ', 'PHKQY', 'HKQYP', 'QGQDVE', 'TGTHPY', 'GTHPYP', 'ADTKIQ', 'KIQRM', 'SLAVALP', 'LAVALPS', 'TRVDAN', 'VDANRT', 'NRTGPAG', 'KKAPTM', 'PTMPKP', 'RVISGH', 'GHLGWV', 'WVRCI', 'IKIWD', 'KIWDL', 'LKLSLT', 'VIVSTR', 'GEDKQV', 'LEYNK', 'RHYHG', 'TCSRDS', 'CSRDST', 'WDVRT', 'GHTNAVA', 'TNAVATV', 'ATVRCQ', 'TVRCQA', 'RCQAAE', 'AEPQII', 'SHDTTI', 'VVLHPL', 'SPDNIK', 'LVSGADN', 'DNGTMH', 'NGTMHL', 'AVQPGSL', 'SGIFACA', 'IFACAF', 'FACAFD', 'EADKTI', 'ADKTIK', 'ETHPVS', 'WKPEI', 'LSMPLN', 'LNGLKE', 'ILWLK', 'TTVDLK', 'KPADLQ', 'ADLQNL', 'DLQNLA', 'APGTHPP', 'NSEVKT', 'VKTDVN', 'PESNTAG', 'NSRPEA', 'SRPEAN', 'EALERG', 'KLDEY', 'FSTRR', 'STRRF', 'DGDEMT', 'KVVAKK', 'FDIPKG', 'RYLTN', 'LTNAYS', 'YSRDE', 'RDEFT', 'CPSDKE', 'PSDKEV', 'IAYSDV', 'SDVAKR', 'KDPDAQ', 'DPDAQK', 'LRQQE', 'EAEEKA', 'ILDTVL', 'GWLALL', 'LGVLLGT', 'WARRS', 'QDLHCG', 'CRALVD', 'WEIAR', 'IARVDP', 'KTIQMG', 'RINPDG', 'VEVPYA', 'ARSEAH', 'DRMKE', 'IDPSTH', 'ELDLQG', 'FACESI', 'VEEYE', 'YEDEL', 'LIEFF', 'EADNVK', 'DLCDHA', 'LCDHAL', 'WPVTVA', 'AMLLIL', 'LVDAYP', 'AKPEAPG', 'PGEDASP', 'ELSRY', 'SLRHY', 'GKRDVP', 'DDSDSE', 'DSENLP', 'LETAAGN', 'GNGRRL', 'IPEAVF', 'MKQPGN', 'KQPGNE', 'LDEQY', 'KFMEL', 'KGQIPE', 'ILKYM', 'ESTNSM', 'STNSME', 'YCKASV', 'LWLGAN', 'ANVMLE', 'MLEYD', 'TKNLDS', 'VKRRN', 'REFLI', 'VQRAEA', 'THMATL', 'VHGRGF', 'LTKLNS', 'TKLNSI', 'LNSIID', 'IIDQAL', 'EKNSSL', 'KNSSLQ', 'NSSLQK', 'SGQVSAK', 'NLFRK', 'LFRKI', 'RQRKS', 'KTQLST', 'TVVSKH', 'LTQSPK', 'ATKEQV', 'RQDSNG', 'DILSEP', 'SEPAALS', 'AALSILS', 'NMNNSP', 'CHVLLS', 'LNHNSS', 'NHNSSL', 'SSLALSV', 'AGFGDCC', 'DCCNQS', 'CCNQSD', 'CNQSDT', 'NQSDTL', 'MLFRT', 'ELLRAA', 'LRAALK', 'RAALKI', 'AALKIR', 'ALKIRA', 'PEPFSP', 'QLLISC', 'LISCLQ', 'QSAASNP', 'QAMDLV', 'AMDLVQ', 'EFIQH', 'FIQHQ', 'LLTRDG', 'DGVPSEA', 'EAAEHL', 'AAEHLK', 'AEHLKA', 'LKALIN', 'INSVIK', 'NSVIKI', 'HHRDL', 'NQLSKS', 'GDVQYP', 'YPERC', 'RCCCIA', 'CAHQCL', 'TCVQIL', 'GVHSVGI', 'CCCMDP', 'MDPKSV', 'KLPALK', 'ALKAFQ', 'FQQHI', 'LNVLSK', 'NVLSKL', 'VLSKLL', 'LSKLLV', 'SKLLVD', 'AELSPR', 'VDSDQL', 'GTLCGAGP', 'PSYRF', 'YRFQG', 'FQGILP', 'LPSSGSE', 'PSSGSED', 'EDLLW', 'DLLWK', 'FVFQE', 'EDRLH', 'NIQIAN', 'IANHIC', 'ANHICN', 'NLLQKG', 'LLQKGN', 'VQWKL', 'NYIFN', 'PVLQRG', 'VLQRGV', 'RGVELV', 'SAQTHM', 'LKTLPV', 'KTLPVL', 'GLDIQQ', 'LPSLSVG', 'LSVGPSL', 'ASLREP', 'ESEDTS', 'EDTSGY', 'SEPLSH', 'LSLEDV', 'EDVVLP', 'RLGGFQ', 'SATAPDL', 'QSQPVL', 'ARACQQ', 'LSVENI', 'SVENIL', 'LREHL', 'KVAETE', 'ELAKPL', 'PLFDAL', 'ALGNHSA', 'EAEDSQ', 'CCSLKL', 'CSLKLL', 'LKLLGE', 'PEDVDT', 'ELNPEA', 'LNPEAE', 'GFSGSIV', 'ENLTHG', 'NLTHGE', 'LTHGEI', 'THGEII', 'KAKLDV', 'LDVLAH', 'NISLLI', 'LLIQQG', 'GFLNIL', 'SDFQAC', 'VLVDLL', 'LVDLLV', 'LLVSLM', 'TCSEDL', 'CSEDLT', 'SEDLTL', 'RIFLE', 'LEKSPC', 'PCTEIL', 'GIHKIV', 'FTMSPS', 'LHTPSL', 'TPSLSNG', 'PSLSNGV', 'VLSQKP', 'LSQKPP', 'LRRAR', 'RRARI', 'SFPYR', 'SVSLVGL', 'KKRNK', 'KPSVLE', 'GDRPEV', 'EVTESI', 'INPGDR', 'NPGDRL', 'PGDRLI', 'EDGCIH', 'ISLGSKA', 'SLGSKAL', 'ALMIQV', 'MIQVW', 'SGTFIF', 'VCMDSN', 'ENIFF', 'PQGKKN', 'QGKKNV', 'NVHGEI', 'VSGQRK', 'SGQRKT', 'TDVILD', 'TFCMIG', 'MIGHCL', 'GKWDLG', 'GNLLLF', 'LLLFNG', 'FLYACG', 'YGQPVI', 'YSKYI', 'SLSVVY', 'SVVYTT', 'LKGQVK', 'FSSKEA', 'KEAQSI', 'SILLEP', 'ILLEPS', 'FVFLF', 'ELSSCE', 'LSSCEE', 'QALALR', 'KCIVGF', 'CIVGFH', 'ESHAII', 'SHAIIQ', 'LLDWK', 'LDWKI', 'RVEHH', 'EHHQQ', 'HHQQQ', 'LLNAHV', 'NAHVVH', 'HHFLL', 'LLTCQV', 'LTCQVL', 'QLTSMP', 'VKIIAE', 'KIIAEV', 'AEVLGSP', 'NFLLAV', 'VCHNPT', 'NFYFS', 'FYFSL', 'YLRHS', 'FLVISP', 'AFTAAPP', 'PPEGTSS', 'PQRMAA', 'AAQMVR', 'LQNIAD', 'DANPEK', 'KQNLLG', 'NLLGRP', 'LGRPYA', 'GRPYAL', 'LLGAHAS', 'NGVSRGS', 'PRFPR', 'DHKDVG', 'PGDESY', 'GDESYP', 'RSQSTV', 'DDWEN', 'FTFSPA', 'RATSTH', 'DCLIPI', 'ICCGLY', 'LLVLPD', 'PDAMLE', 'AMLEDV', 'IQADIL', 'QADILL', 'ILLVLV', 'LLVLVN', 'LVNHPS', 'QQGVIK', 'FLKNR', 'LLANQL', 'QLYLH', 'FVEMF', 'VEMFF', 'RPIGLD', 'IGLDEE', 'HMELF', 'MELFQ', 'LIETSL', 'VLLHNA', 'LQVLNS', 'KVADML', 'ADMLLD', 'GLLYVL', 'LNGLEK', 'NGLEKN', 'GLEKNI', 'NEYKL', 'ACDIQQ', 'CSSSGTQ', 'IVLLGY', 'VLLGYL', 'TQNMAL', 'LEFIR', 'ANHDSE', 'HDSESP', 'FPLAQT', 'ASDELH', 'QRRMS', 'RRMSQ', 'EHPSQA', 'LNSDII', 'NSDIID', 'SDIIDI', 'PENTSQ', 'VSQTEI', 'EDMHH', 'DMHHE', 'PSVYNP', 'YNPFQ', 'DGFKVC', 'GFKVCI', 'FKVCIG', 'SVSKQQ', 'SKQQW', 'QWTKI', 'GRLLAH', 'HILSPT', 'ILSPTH', 'ERKQI', 'TAELLM', 'LLMNAL', 'LKLCGH', 'IPPSAPS', 'PSKPEL', 'IREEQ', 'LFQRL', 'KSKDIS', 'SKDISK', 'IERKK', 'IQHIR', 'LSASRH', 'WQECI', 'YDPIY', 'DPIYY', 'TEGPNR', 'PNRER', 'RLQRC', 'EGVLRP', 'DKTHSS', 'FSSTVK', 'STVKDK', 'CISVAPS', 'AGELLLG', 'ELLLGK', 'YFVED', 'ASDAVES', 'QGELEP', 'KEVHR', 'RWWQ', 'NGRTLL', 'GRTLLL', 'TLLLAF', 'RDDVY', 'QSILTN', 'LTNNLP', 'TNNLPN', 'YSGQIT', 'TNFEY', 'FEYLT', 'HAGRSF', 'DLMQY', 'SETLDL', 'NLSKPI', 'VQYKE', 'EKEDR', 'DRYVD', 'KGARED', 'DDPMPP', 'PMPPVQ', 'MPPVQP', 'SNSGTVL', 'GTVLHF', 'HFLVR', 'FTKMF', 'DRTFH', 'NTTWR', 'TTWRL', 'LSSFES', 'FYLPE', 'YLPEF', 'EGFDFG', 'HVNLPP', 'LIHRQ', 'ALESDH', 'ESDHVS', 'SDHVSQ', 'NICHW', 'ICHWI', 'CHWID', 'VFHPAT', 'GMDVSAV', 'ALETMI', 'KLNIEG', 'LNIEGE', 'NIEGEL', 'GLLVQF', 'QFAFR', 'FAFRE', 'PVKEVT', 'EVTHPS', 'PSAPVPV', 'SAPVPVV', 'FSQPHG', 'LPTRAI', 'NKEQGV', 'EQGVRS', 'QGVRSM', 'WSAILS', 'SAILSW', 'SKQSEP', 'QSEPPI', 'HQVTSC', 'AWVPDS', 'CQLFTG', 'VITAYT', 'YTNRL', 'NRLTSS', 'TPSEIE', 'ISVSRD', 'GTCIVW', 'NRLCY', 'HKSPVT', 'GDIATVC', 'RLWTV', 'VNGDLVG', 'SINVIAG', 'IAGGLEN', 'NGIVRL', 'PKSNKP', 'SNKPII', 'NKPIIS', 'KPIISL', 'ISLTFS', 'HHLYT', 'LYTANS', 'ANSEGTV', 'SEGTVIA', 'RKDQQ', 'KDQQR', 'LPMFY', 'FYSFL', 'FGGDDAP', 'DCYVGD', 'GVVTNW', 'LRVAPD', 'VAPDEH', 'EAFNTP', 'QEMVTA', 'LERSY', 'RSYEL', 'LGIESR', 'IESRGI', 'GRRWP', 'PRRRS', 'SRRGPT', 'PLPRGGA', 'PRGGAGR', 'GRDTLL', 'RDTLLG', 'TLLGAKA', 'LLGAKAT', 'ACAALPQ', 'PQPVPH', 'PVPHEP', 'HEPRE', 'EPREAA', 'AFRLAP', 'FRLAPA', 'ASERGAS', 'RGASVSP', 'PRAART', 'RAARTP', 'LLCNKS', 'LCNKSN', 'ECLQW', 'RAGYSL', 'QPYEAA', 'ARMFF', 'GIFTTM', 'FTTMSS', 'DSIVLQ', 'FDHIF', 'HIFTTG', 'VIQKLA', 'PSNIHV', 'HVSELS', 'SELSLQ', 'ELSLQS', 'KYVFI', 'KLVQW', 'WLVSIG', 'LVSIGAN', 'VSIGANI', 'WVMDQ', 'HINHR', 'HRDDAG', 'DAGCTVL', 'CTVLHV', 'AAAHFPG', 'FPGYTS', 'MLLSFG', 'DPTLLD', 'LDGHAR', 'VVDVLK', 'AIDKIN', 'IPAGLVC', 'PAGLVCD', 'AGLVCDV', 'VNRDCA', 'FIKFL', 'FLLER', 'LERQK', 'VLLLLT', 'RKVSGQ', 'LRNGVI', 'DCDLSD', 'DLSDLD', 'LSDLDI', 'CTVIPH', 'IDSGALP', 'LRHED', 'AFLLLT', 'AALHIF', 'YYLNP', 'NVQDSN', 'GNTLMH', 'NKAGKGV', 'KNDALL', 'NKALTE', 'ALTESR', 'ESRRK', 'AHLGRL', 'GRLSRS', 'LPCGTAD', 'CGTADTT', 'LPDVQV', 'QEPEAV', 'PEAVRT', 'LRDRL', 'VQDITV', 'DITVLI', 'QQVELG', 'QVELGM', 'PEDSPQ', 'KVAAGTE', 'TEGKKD', 'GSSGCSGN', 'ASEAGDGA', 'GDGAQAGP', 'DGAQAGPG', 'PGASQLV', 'GASQLVP', 'LGVASDN', 'QENWT', 'EIQACL', 'IQACLQ', 'QACLQD', 'TWEIE', 'LKKLSS', 'KIILAI', 'IILAIQ', 'IQQLGN', 'LGNGEW', 'GEWTQG', 'TQGLQK', 'QKRLK', 'LKHSKG', 'KHSKGN', 'KGNIQL', 'DKGARM', 'KGARML', 'ARMLW', 'FSARCS', 'CSENSE', 'GTERNT', 'EKSGRV', 'RVYTE', 'RKKLK', 'KLKGIN', 'LKGINK', 'KIQKR', 'KRIPR', 'RCYVE', 'EAEKSL', 'AEKSLE', 'DPEYF', 'PPASAVE', 'PASAVET', 'IMKFH', 'FSTNMA', 'NMALNI', 'MALNIL', 'VIDLNP', 'GTGKTTC', 'KTTCCL', 'CLYRL', 'EKAEQA', 'LPKRR', 'PKRRL', 'RRLEV', 'EEEEH', 'EEEHE', 'EEHEE', 'EHEEE', 'HEEEE', 'SIKVET', 'DGIDEE', 'VEPAGDS', 'CVPDHP', 'QRNFI', 'IELSKS', 'VTSKQL', 'RNEDGS', 'DGSLKR', 'KRTIVG', 'VGWSTQ', 'EADGNY', 'NYNEE', 'KATETQ', 'EIWPK', 'IWPKM', 'WPKMI', 'PKMIKG', 'FEALSC', 'RSEIY', 'FCLYQ', 'NLSWR', 'LSWRL', 'RVLPW', 'WSIHE', 'IHELY', 'GDEIQD', 'CINDPN', 'AQSIMK', 'KGVAFR', 'FRFSD', 'STVDKQ', 'CAVRKP', 'RIHQL', 'IHQLY', 'QNYRS', 'NYRSH', 'LNLASGV', 'LFDGPK', 'PKPTLL', 'IEFGAH', 'LVANEK', 'VYEAKG', 'LLYNF', 'FTDSEA', 'IISSFT', 'EKWPL', 'VDVPLE', 'ARSLMV', 'PEMYK', 'MYKLL', 'KLLNGE', 'LNGELK', 'EKRAPA', 'AFKYF', 'VQVVKT', 'CWKVAA', 'KCYQK', 'ALAHYT', 'LAHYTA', 'AHYTAL', 'TALNMK', 'LELAKT', 'LSLKCL', 'KIRDAA', 'MYCQE', 'YCQEE', 'VEKYE', 'KLSYSA', 'ANKSKE', 'VLSKLD', 'LSKLDV', 'SRKCLA', 'REGRR', 'QHGCLL', 'HGCLLE', 'GCLLEAA', 'LLEAAR', 'LEAARL', 'DFQASC', 'FQASCL', 'ARFNVA', 'ARDSDI', 'FLLGII', 'FIKFD', 'MLNHSA', 'SLCESE', 'QKVLAL', 'VLLNLV', 'TNNAEK', 'CQIAQN', 'GPILRI', 'DLTLSE', 'IVTDQV', 'DQVKLA', 'QVKLAL', 'LLGRLC', 'QILLGK', 'PGICMR', 'ICMRF', 'CMRFI', 'FIVGLK', 'DFHRP', 'FHRPL', 'LRRCE', 'HLVAIN', 'EMDYI', 'DTYGLC', 'YGLCKS', 'LFPRH', 'RSFRC', 'EYIHD', 'LLSSSY', 'SSSYPE', 'YPEDF', 'EDFEK', 'LRQEE', 'RQEED', 'SYNRE', 'EGKFGM', 'MDQLY', 'RFMNV', 'TVVLLE', 'VLCLPK', 'RESGEV', 'ESGEVF', 'DVTRAI', 'LVKVLC', 'YENMN', 'YVISGE', 'VLCLVM', 'CLVMLV', 'MLVNAE', 'LVNAEE', 'NAEEVL', 'MSMDW', 'ALQDLL', 'LLFER', 'DEEYL', 'HEEVR', 'RLNRL', 'KLRRV', 'RWRR', 'DREVR', 'EVRPGN', 'ADVDRT', 'LCGVKF', 'TRSPES', 'EGTATEA', 'RAELEG', 'AELEGR', 'CRERI', 'QHIRL', 'HIRLE', 'AYQKY', 'QKYLD', 'LDFFH', 'IEEGKV', 'VQGIEQ', 'HLGSKE', 'MLQRK', 'KAWAEA', 'AWAEAE', 'KILTLS', 'WLKKT', 'ELRPR', 'PATPGVGA', 'ATPGVGAP', 'PGVGAPGN', 'EKYDN', 'KYDNM', 'ELFAVV', 'LFAVVK', 'VKTMQA', 'NEYTAA', 'FRQVQ', 'RQVQGS', 'DEFCR', 'MERIK', 'ERIKE', 'IKDDKG', 'KDDKGN', 'GNLNRC', 'RCIADV', 'VVSLFI', 'RELME', 'ETMHR', 'TMHRM', 'HRMSH', 'QWLQT', 'LDDSQV', 'SAYNAF', 'PPESTAA', 'AAAAAADI', 'AHPAMF', 'GGSASALN', 'ASALNAAG', 'ALNAAGTG', 'NAAGTGVS', 'VSGAAPSS', 'AAPSSED', 'QPPPPAA', 'PQSLNL', 'SQAQLQ', 'QLQGQP', 'PAQISAS', 'IAEDTE', 'SEILDV', 'EILDVS', 'DVSLSR', 'GEPERS', 'QEAETP', 'EAETPGA', 'GAVSPNQ', 'HHSSHA', 'PPSSPVS', 'SPVSRK', 'GSSDGGVP', 'VPVAPPP', 'GLPASVM', 'IRTPST', 'TNNVNI', 'NAASISGG', 'SISGGPGV', 'GGPGVTSV', 'GVTSVVN', 'NSSILSG', 'TVSSSPV', 'SPVANSV', 'ITVGVVS', 'VGVVSSQ', 'VSSQQQ', 'TVNTSR', 'TSRFR', 'STSEPF', 'KKGRW', 'VPATEGV', 'KVVETV', 'VVETVK', 'TVKQTP', 'VKQTPT', 'PTEASSS', 'VSSSVST', 'GSGEMMG', 'MMGAPAV', 'PPGLQGV', 'VALQQL', 'SSPAPQS', 'QSIAAVS', 'SQSQMS', 'LQPVPL', 'ATGIQPS', 'VGVTAAVG', 'TAAVGQQ', 'AAVGQQP', 'LAQPQL', 'QNPTSE', 'QSSSTGT', 'SSSTGTGA', 'PVAQAQG', 'PGQPTAV', 'QTQPAGA', 'TQPAGAAG', 'PAGAAGQP', 'GAAGQPIG', 'AGQPIGQ', 'GQPIGQA', 'GQAQTAV', 'AQTAVST', 'TAVSTVP', 'AVSTVPT', 'VPTGGQI', 'ASIGQQA', 'QQANIP', 'PSVIQQ', 'LPAPTGI', 'PAPTGII', 'TGIIHQ', 'IIHQGV', 'HQGVQT', 'QQLVIA', 'TVAQGVV', 'VVSQQL', 'VSQQLP', 'SQQLPT', 'PTGSPLP', 'PLPSAST', 'NQVSSAA', 'VSSAAPSG', 'PSGMPSV', 'SGMPSVP', 'NLVPPQ', 'LVPPQN', 'VPPQNI', 'PQNIAQ', 'QNIAQP', 'PATQNGS', 'TNINLP', 'LPLAQQ', 'PLAQQI', 'LAQQIP', 'AQQIPL', 'STQFST', 'AQAIGSQ', 'PAEPSLG', 'EPSLGGL', 'SLGGLPQ', 'LPQTMS', 'AAPASLF', 'PLKVLP', 'KVLPLT', 'SSGASVVA', 'ASVVAID', 'AIDNKI', 'AMDLVK', 'LAQFQA', 'GTTQPPA', 'PPAQPAS', 'QPASQGS', 'VLHRF', 'LHRFT', 'QCLASR', 'LGPLLAS', 'RVRQN', 'LEERE', 'DKDDW', 'DDWEC', 'GLRAME', 'NVNQSL', 'DLHTLA', 'KGDPHL', 'THYLH', 'LGDHVH', 'GDHVHN', 'TMGAPAAG', 'NSKAKK', 'PPTVEQ', 'AQDPVF', 'PEDPPD', 'EDPPDP', 'PDPSPR', 'SPRAED', 'PRAEDS', 'SRAYVA', 'AYVAMN', 'LRQAGE', 'GEALRQ', 'LRQAGQ', 'AGQRLE', 'AGRGGRV', 'LLALCAA', 'LALCAAL', 'ALVAGGW', 'AQEPGAP', 'QEPGAPA', 'EPGAPAAG', 'PGAPAAGM', 'APAAGMR', 'GMRRR', 'RLQQE', 'EDGISF', 'DGISFE', 'RYPEL', 'YPELR', 'REALVS', 'ALVSVW', 'TVGRSF', 'LVIELS', 'VIELSD', 'ELSDNP', 'QKGNET', 'TIVNLI', 'RIHIM', 'IMPSLN', 'QPGELK', 'VNEKEG', 'IVDQNS', 'TKAVIH', 'VIHWI', 'WIMDI', 'HGGDLVA', 'LVANYP', 'GTAHEY', 'YSSCPD', 'PDDAIF', 'DDSSFV', 'DSSFVD', 'NGGAWY', 'SVPGGMQ', 'LSSNCF', 'ANATISV', 'DGIDHD', 'DHDVTS', 'VTSAKDG', 'AKDGDY', 'LAITKK', 'PFSPAVG', 'FSPAVGV', 'FSERK', 'MGDSHE', 'SHEDTS', 'SATVPEA', 'SKIQTT', 'TGRNII', 'GRNIIV', 'TAEEFA', 'LSKDAH', 'EEYDL', 'SLPEID', 'IDKSLV', 'SLVVFC', 'VVFCMA', 'VFCMAT', 'MATYGE', 'WLQET', 'KFAVFG', 'AVFGLGN', 'KTYEH', 'HFNAMG', 'FNAMGK', 'AMGKYV', 'QLGAQR', 'RIFEL', 'DGNLEE', 'GNLEED', 'FFGVEA', 'SIRQY', 'RLKSY', 'KPPFDA', 'PPFDAK', 'SKIRY', 'IRYES', 'VYPAND', 'NQIGEI', 'QIGEIL', 'IGEILGA', 'GEILGAD', 'LDVIMS', 'DVIMSL', 'VIMSLN', 'SLNNLD', 'LTYYL', 'TYYLD', 'LDITNP', 'AQYASE', 'SEPSEQ', 'KMASSSG', 'MASSSGE', 'YLSWV', 'EARRH', 'ILAILQ', 'LAILQD', 'PSLRPP', 'PNSVHI', 'NSVHIC', 'RVNKGV', 'NKGVATS', 'VATSWL', 'EPAGENG', 'GRRALV', 'ALVPMF', 'VIMVGPG', 'IMVGPGT', 'GVAPFMG', 'APFMGF', 'ERAWL', 'AWLRE', 'TLLYY', 'FHKDGA', 'HLLKR', 'KRDKE', 'LWKLI', 'IYVCGD', 'DARNMA', 'NMAKDV', 'AKDVQN', 'QNTFY', 'GPMEHT', 'HTQAVD', 'KLMTKG', 'AYAKDV', 'GADARAL', 'DARALM', 'ARALML', 'LLADAVA', 'VAVTMGP', 'KGRTVI', 'WGSPKV', 'LKDKY', 'DKYKN', 'AKLVQD', 'NTNEEA', 'EEAGDGT', 'TTATVLA', 'SKGANPV', 'VEIRR', 'VIAELK', 'VTTPEE', 'VATISAN', 'TISANGD', 'RKGVIT', 'VITVKD', 'LEIIEG', 'VKAPGFG', 'APGFGDN', 'DMAIATG', 'EEGLNL', 'LEDVQA', 'HDLGKV', 'VIVTKD', 'DDAMLL', 'KRIQE', 'VLKVGGT', 'EKKDR', 'DRVTDA', 'RVTDAL', 'RAAVEE', 'GIVLGGGC', 'LRCIPA', 'RCIPAL', 'LDSLKP', 'LKPANE', 'KPANED', 'PANEDQ', 'DQKIGI', 'KIGIEI', 'GIEIIK', 'LKIPAM', 'MTIAKN', 'TIAKNAG', 'IAKNAGV', 'KNAGVEG', 'GVEGSLI', 'KILQSS', 'SEVGYD', 'GYDAML', 'YDAMLG', 'NMVEKG', 'MVEKGI', 'EKGIID', 'IDPTKV', 'KVVRTA', 'VRTALL', 'TALLDAA', 'ASLLTTA', 'EEKDPG', 'EKDPGM', 'KDPGMGA', 'DPGMGAM', 'PGMGAMGG', 'RDLPDG', 'PDGPDAP', 'EVLRR', 'GVASSEE', 'LVRQR', 'GGFINY', 'YFKSK', 'KPVEPP', 'VEPPPE', 'KEAINT', 'INTSKD', 'NLRRL', 'DSVPKE', 'TYHQK', 'IDSNLD', 'LDELSV', 'ELSVGLG', 'LKDIAL', 'LGMQTE', 'GMQTEI', 'LTTKVD', 'DVNIKS', 'QCSALR', 'CSALRK', 'VVLKGR', 'VLKGRP', 'LKGRPC', 'GRPCKI', 'KHGHAK', 'AKVHLV', 'KYEDI', 'QLIGIQ', 'DGYLSL', 'SGEVRE', 'DLGKEI', 'GEEILI', 'ILITVL', 'SAMTEE', 'RADQLA', 'TRRML', 'EESKDA', 'EQLER', 'ERIEE', 'GMDQIN', 'AEKNLT', 'KFCGLC', 'CPCNKL', 'NKLKSS', 'KLKSSD', 'KSSDAY', 'SSDAYK', 'QDGVVAS', 'DGVVASQ', 'VVASQPA', 'QPARVV', 'DEREQ', 'EQMAIS', 'RVTNDA', 'VTNDAR', 'RENEM', 'QVSGIIG', 'SGIIGNL', 'NLRHM', 'LDMGNE', 'DMGNEI', 'MGNEID', 'RIMEK', 'TVPLLE', 'VPLLEP', 'PNSLLK', 'LLKSTL', 'KSTLSM', 'TRTTPT', 'NQESCP', 'SCPPGIP', 'TGSASRP', 'SRPDTP', 'ESNPKT', 'SNPKTQ', 'MLPLDS', 'LPLDSQ', 'LDSQGAS', 'DSQGASL', 'VKYYT', 'WVILH', 'FLEEH', 'LREQAG', 'STDARE', 'GELHPD', 'LITTVE', 'VIPAISA', 'AVALMY', 'NTDEQA', 'TDEQAL', 'TNPEHA', 'RAMNGE', 'ESLDGR', 'SLDGRQ', 'RSYSR', 'SRGGGDQ', 'PGGYGYG', 'GYGYGR', 'GYDRY', 'LETVPL', 'LERKK', 'RKLFI', 'TDCVVM', 'MRDPAS', 'RDPASK', 'DPASKR', 'VTFSSM', 'FSSMAE', 'DGRVVE', 'AVAREE', 'HHLRD', 'PVDKIV', 'QKYHT', 'AEVRKA', 'VRKALS', 'QEVQSS', 'SRSGRGG', 'PGPGSNF', 'PGSNFR', 'FGDGYN', 'NGYGGGPG', 'GYGGGPGGG', 'GGYGGGGPG', 'GGGPGYGN', 'GGGNYGSG', 'SNYGPM', 'GGSRNMG', 'GSRNMGG', 'GGPYGGGN', 'GGGNYGPG', 'GGNYGPGG', 'MAPPVPG', 'QCINEA', 'CINEAQ', 'FLYRA', 'DSIVLE', 'SIVLEL', 'RSVDVT', 'AASIYF', 'TLHQGD', 'DGLECM', 'CMAMTI', 'AMTIQI', 'RKELK', 'QDEDAT', 'AWVNLA', 'LAVGGEK', 'AVGGEKL', 'QEAYY', 'AYYIF', 'ELADKC', 'PTLLLL', 'LNGQAAC', 'GQAACHS', 'ACHSAQG', 'QGRWE', 'EGVLQE', 'VLQEAL', 'LIVLSQ', 'SQHLGK', 'KPPEVT', 'TNRYL', 'RYLSQ', 'TSGGASVV', 'SVVCFL', 'LNNFY', 'NFYPK', 'NVKWK', 'VKWKI', 'IDGSER', 'GVLNSW', 'STYSMS', 'TYSMSS', 'YSMSST', 'TLTLTK', 'RHNSY', 'SYTCEA', 'THKTST', 'YDQHL', 'VEETVT', 'VTTIEI', 'EETYE', 'ETYEE', 'DGVVLVA', 'ASMSHH', 'SSISSTS', 'RTFGPP', 'FSYSSS', 'SSSSRF', 'SSSRFS', 'SSRFSS', 'SRFSSS', 'SSSARLG', 'SSARLGS', 'ATRSNE', 'QNAALR', 'LRGELS', 'QEPARA', 'ARADQL', 'ADQLCQ', 'QQELR', 'QELRE', 'ELELLG', 'ERDRV', 'RDRVQ', 'KQRLE', 'QRLEE', 'RLEEE', 'ELERK', 'ERKIE', 'EEELR', 'LQVSVE', 'SVESQQ', 'QVEVEA', 'LRDIR', 'NRNHE', 'ESRRQ', 'RELEE', 'EELRQ', 'RQLKE', 'RISVPV', 'KTTVPE', 'TTVPEM', 'QKEQH', 'QPAMLL', 'AAALAAVS', 'VSSAPVD', 'APVDNR', 'RDHNE', 'IIEVLS', 'IEVLSN', 'KKSGKE', 'RLLRD', 'ADASGTR', 'ASGTRW', 'SGTRWA', 'GTRWAS', 'GAPVEDS', 'PVEDSQ', 'KVGNEK', 'NEKWT', 'KWTEGG', 'SLRPSN', 'NQQASK', 'SKEAKI', 'SEERVG', 'REDAGE', 'EEKKH', 'KRSEAS', 'SAKKKD', 'DESVAR', 'QESGEE', 'SGEETR', 'EESQEG', 'ESQEGE', 'SQEGEE', 'KSSYEG', 'SEERR', 'SPKESK', 'ESKEAD', 'TVRLGE', 'SRSYR', 'ESTANR', 'GRGREP', 'RGREPG', 'REPGAH', 'EPGAHSA', 'GAHSALD', 'EEKRL', 'HYPVR', 'QSKWQ', 'SKWQE', 'EKNYL', 'NYGEEG', 'YGEEGD', 'EEQLGP', 'EQLGPE', 'QLGPEE', 'GPEESR', 'PDRQY', 'YEPYP', 'RLGALF', 'PLQWK', 'RGNPDD', 'FPEYN', 'YDWW', 'WWER', 'NWGYE', 'EKRSF', 'VAELDQ', 'QLLHY', 'LATALTS', 'AVLCRD', 'PCDICK', 'KTVVTE', 'TVVTEAG', 'LLKDNA', 'KDNATQ', 'DNATQE', 'EILHY', 'LHYLE', 'IHDSSL', 'KEVVDS', 'LPVILD', 'LDMIKG', 'GEVCSAL', 'LNLCQS', 'NLCQSL', 'MSNIPL', 'QPQPKA', 'PKANED', 'KANEDV', 'TNSSFI', 'GFVDHV', 'DCDRLG', 'GFCNEV', 'LPALEM', 'QNLVQA', 'VQAHNV', 'AHNVIL', 'VILCQT', 'SELIVN', 'NNATEE', 'NATEEL', 'ATEELL', 'LPDPAR', 'PARTKC', 'VVGTFGP', 'NPSSLCG', 'LCGVIGL', 'ELVEAL', 'LVEALE', 'PAIVSAL', 'ALLKEP', 'KEPTPP', 'EPTPPK', 'PTPPKQ', 'TPPKQP', 'PKQPAQ', 'KQPAQP', 'QPAQPK', 'PAQPKQ', 'LYLEH', 'YLEHN', 'EKGCSF', 'SFLPDP', 'CDDFVA', 'CPSAYK', 'METAAR', 'RCNAVD', 'AVDHCK', 'SSEERA', 'EERAAP', 'APFTLE', 'FTLEY', 'DVPIYA', 'VPIYAD', 'MVVEVP', 'EIATKD', 'DPLNPI', 'QDVKKG', 'DVKKGK', 'VKKGKL', 'LFPYK', 'KGYIW', 'QTWED', 'TGCCGDN', 'NDPIDV', 'PIDVCE', 'IGSKVCA', 'RVKVLG', 'ILAMID', 'ETDWK', 'INVDDP', 'NVDDPD', 'KDISDV', 'DISDVE', 'TVDWF', 'GKPENE', 'FAVDII', 'VDIIKS', 'DIIKST', 'KKTDGK', 'TDGKGIS', 'ISCMNT', 'CMNTTV', 'SACSLPT', 'PTDVDK', 'FLVALL', 'LTCHVC', 'LLAVTR', 'LAVTRI', 'RCPTPV', 'LFYKC', 'KCCQW', 'CNGEGPP', 'NGEGPPT', 'GEGPPTD', 'PPTDQL', 'LKEQPG', 'PGKASGR', 'KASGRR', 'LTGFMV', 'TGFMVL', 'GFMVLT', 'LTANGLS', 'NGLSALC', 'GLSALCL', 'SSKRAK', 'RAKAKT', 'KAKTTK', 'AKTTKK', 'DEYLE', 'DHLRE', 'HLREL', 'MYREA', 'FNYVE', 'NYVEF', 'SGALDVL', 'DFQME', 'RKSDGI', 'SDGIYI', 'TWEKL', 'AARAIVA', 'VISSRN', 'ISSRNT', 'FTPGTF', 'GTFTNQ', 'TNQIQA', 'QIQAAF', 'REPRL', 'TDPRAD', 'TEASYV', 'ASYVNL', 'WMLAR', 'REVLR', 'VLRMR', 'HPWEV', 'KEEQAA', 'EKAVTK', 'KAVTKE', 'AVTKEE', 'EWTAPA', 'SVPIQQ', 'EDWSAA', 'STDIAGL', 'EESFR', 'ESFRK', 'FRKFA', 'KFAIHG', 'IHGDPK', 'AKLCKD', 'KSARVI', 'INYEE', 'LEELAT', 'KGKSKE', 'GKSKEE', 'KEPANI', 'PANIGVT', 'TGGAVDR', 'AVDRLT', 'DILDDS', 'LDDSGY', 'RVAVLL', 'LSLVLGA', 'AGAVPVGV', 'KHWVV', 'YRHQA', 'YQIIH', 'QIIHR', 'RNGIPD', 'MMYDD', 'SEENPT', 'INRPNG', 'NGTDVY', 'DVTPEN', 'NFLAVL', 'SGPRDH', 'RDHVF', 'DHGATGI', 'HGATGIL', 'YQKMV', 'MVFYI', 'VFYIE', 'GSMMNH', 'PDDINV', 'NVYATT', 'RGTYLG', 'DWYSV', 'WYSVN', 'YSVNW', 'VNWME', 'WMEDS', 'SDVEDL', 'HVMQY', 'KSISTM', 'SISTMK', 'ASSPISL', 'QNLIGQ', 'LIGQIQ', 'VIEKSV', 'KIVSLL', 'IVSLLAG', 'TMLTAH', 'YLYVL', 'APYPID', 'FPEGAGP', 'GAGPYVD', 'DLDEAGG', 'AGGSTGLL', 'LLMDLA', 'FNDFE', 'FEDLF', 'TLQKMG', 'VEEAEP', 'RVVNGK', 'VEYFL', 'KEKDGT', 'EKDGTK', 'GTKRKS', 'DSESDD', 'KKKRD', 'KRDAAD', 'RDAADK', 'LVLAKE', 'CPQIVI', 'ILIQW', 'LIQWI', 'CREDVG', 'TVLCKL', 'CKLINS', 'QAPVKK', 'KKIQAS', 'SSMAFK', 'QFLQAA', 'YGINTT', 'INTTDI', 'ACVQRT', 'MNLGGLA', 'SGDPNW', 'QEGKNV', 'MPRHH', 'PRHHAG', 'GAAGLWV', 'WVRSGA', 'RSGAAAAA', 'GAAAAAGAGG', 'AAAAAGAGGG', 'GMKDVE', 'GRVLLN', 'LLLGTR', 'RAAALGGG', 'GGGLRES', 'RMSLLG', 'PLSYTS', 'SYTSSQ', 'YTSSQS', 'SSQSCR', 'RRNVK', 'RRVQN', 'YLYNV', 'LERPR', 'RPRGW', 'EFIIR', 'IRIWS', 'RFARK', 'IDTIVL', 'DTIVLI', 'IASIAVV', 'AVVSAKT', 'LRFLQ', 'RFLQI', 'LRMVR', 'VRMDR', 'SVVYAH', 'VVYAHS', 'KELITA', 'LVEKDA', 'VEKDAN', 'LTTIGY', 'DKTPLT', 'GRLLSAG', 'PAGILGSG', 'KRRNP', 'VSIATW', 'TWKPH', 'KALHTC', 'SFKER', 'KSRQAS', 'ITAEGSP', 'WSFND', 'TRFRP', 'KSSQPK', 'PKPVID', 'EKGCQC', 'VEDLTP', 'EDLTPP', 'LTPPLK', 'PLKTVI', 'TVIRAI', 'DQILGK', 'ETTDDP', 'DDPSML', 'PSMLAR', 'SMLARV', 'ARVVKV', 'SIESKL', 'RKGSASA', 'TLASFQ', 'VDSKDL', 'KDLSGSA', 'DLSGSAQ', 'LSGSAQN', 'ISRGLQ', 'TMHSQA', 'MHSQAT', 'HSQATQ', 'VPMSQN', 'NDGSSVV', 'SSVVATN', 'NNIANQ', 'NIANQI', 'ANQISAA', 'PAAPTTL', 'PTTLQI', 'TTLQIP', 'TLQIPP', 'LSAIKH', 'SAIKHL', 'AIKHLS', 'EPLLSN', 'LQESIS', 'QESISD', 'SKESVQ', 'SNLTKD', 'GGETLLS', 'LSVQNL', 'SVQNLI', 'SGSRGSQ', 'QDFYP', 'DFYPK', 'TPPPAGE', 'GEAAFSS', 'SRSSQN', 'GDSTDAL', 'LEVEGK', 'GKMVSR', 'IDDSLI', 'DDSLIGG', 'DSLIGGN', 'EGPEGEG', 'AYKKY', 'KYIKD', 'IKDYM', 'NFNNY', 'IGENMN', 'REDGVT', 'IFFKD', 'LWWGL', 'LPGPSTP', 'GPSTPVL', 'PVLLLL', 'LLLLSL', 'FSREE', 'GGDQDW', 'DQDWD', 'GSGAVALC', 'ESPGVW', 'SPGVWE', 'LGVREP', 'MRLRV', 'VVQALGS', 'LLCWH', 'CWHPE', 'HPEISS', 'SSCGRTG', 'SCGRTGP', 'GSLPLDA', 'LDALSPG', 'DALSPGD', 'LRNSSP', 'ELLAQP', 'SRPVAF', 'RNARR', 'IRKRV', 'KLWEP', 'KGQGER', 'EDSAPR', 'APRAVR', 'RAVRTA', 'PGPRPPG', 'PRPPGF', 'RPPGFP', 'PPGFPTG', 'PGFPTGP', 'QILSTN', 'ILSTNQ', 'STNQAR', 'FPQYN', 'TLVPEN', 'TSVLRV', 'SVLRVV', 'QSGLIR', 'AALDRE', 'VTAQDH', 'DHGSPR', 'HGSPRL', 'VAVTVAD', 'TVADRN', 'RETLR', 'ENVEEG', 'TDGDAPP', 'FVGSPAV', 'RVDRE', 'DQGQEP', 'DENDNA', 'APQFSE', 'FSEKR', 'REDVR', 'ATDKDK', 'YNIISG', 'SRGHFA', 'AIDSLTG', 'GEIQVM', 'AEREY', 'EREYA', 'IRAQDA', 'SNNTGLA', 'NNTGLAS', 'NTGLASI', 'VVDIND', 'PFQVSV', 'QVSVLE', 'GENSRL', 'SRLEY', 'RLEYS', 'EYSLTG', 'PFVINS', 'SVSGPLD', 'HYFFG', 'RDHGSP', 'ASVTVTV', 'VLDVND', 'TMKEY', 'NEDAAVG', 'VVSVTAV', 'TAVDRD', 'DRDANS', 'DANSAIS', 'SAISYQ', 'SYQITG', 'FAISTQ', 'VGLVTLA', 'LDYKQ', 'YFKLV', 'HDHCY', 'EDRPVG', 'GSTVVVI', 'VVVISAS', 'ASDDDVG', 'RIDADS', 'DSGAITL', 'GAITLQA', 'DYEDQ', 'VTYTLA', 'RDNGIP', 'IPQKAD', 'ADTTYV', 'VMVNDV', 'VNDVND', 'NAPQFV', 'VASHYT', 'SHYTGL', 'EDAPPF', 'RVQYT', 'YTFQN', 'DGDFTI', 'REAVPV', 'EAVPVY', 'AVPVYE', 'ELTAYA', 'LRTPVS', 'RTPVSI', 'TVQDVN', 'NSIVGSV', 'GSVVAQI', 'IMYQI', 'ELFQM', 'MDIFSG', 'DIFSGE', 'VVQATSA', 'VQATSAP', 'TSAPLVS', 'SRATVH', 'QNDNSP', 'FQILF', 'LFNNY', 'VSNRSD', 'IIGRIP', 'GRIPAY', 'IPAYDP', 'PAYDPD', 'PDVSDH', 'LFYSF', 'GNELQL', 'VVNRTS', 'SGELRL', 'GELRLS', 'NRPLVA', 'VVIITE', 'TEELLA', 'AVLATPT', 'NDTDVGG', 'LNVSFS', 'SALAPRG', 'ARSLLD', 'LLDVLP', 'DDNVCL', 'MKCVSV', 'CVSVLR', 'AGLRCR', 'RCPPGF', 'CPPGFTG', 'TGDFCE', 'GDFCET', 'PCRNGGA', 'TCVCRP', 'RPRFT', 'ELDTEA', 'CTNAPNG', 'NAPNGGF', 'GGFRCQ', 'RCQCPA', 'AFEGPR', 'PRCEVA', 'ARSFPP', 'PSSFVM', 'RFHLT', 'TLSLSF', 'LLFYN', 'EKHDF', 'ALELVAG', 'GQVRLT', 'QWHTV', 'YNKPR', 'PRTDAL', 'ALGGAQGP', 'PSKDKV', 'SKKSLD', 'KSLDLT', 'TGPLLLG', 'PLLLGGV', 'LLLGGVP', 'LLGGVPN', 'LGGVPNL', 'VPNLPE', 'MRDLH', 'DLHIDG', 'ANNGTMA', 'GTMAGCQ', 'CQAKSH', 'ASGPCKN', 'FSCDCP', 'LTMAHP', 'QGNGTLS', 'NGTLSW', 'GTLSWD', 'LSWDF', 'VPWYL', 'MQVQLG', 'HSVLLC', 'VTLNRA', 'DGRWH', 'GRWHD', 'LQEEPG', 'EPGGRR', 'GGELQGL', 'QGLKVK', 'GLKVKQ', 'HVGGLPP', 'PSSKEE', 'GHQGLVG', 'VGCIQGV', 'GPCPPHA', 'PHADCK', 'WQTFS', 'PHGYTC', 'CVSGYF', 'KGFDPN', 'PNCNKT', 'GQCHCK', 'GSDSCLP', 'RSCAPH', 'CRPGALG', 'QCNSCD', 'PQTKFG', 'TVPCPR', 'CPRGALG', 'AVRLCD', 'FNCTSP', 'LSLLLD', 'GLELNK', 'ELNKTA', 'TVEAKK', 'RLREV', 'LREVTG', 'DHYFS', 'VTARLL', 'HQQGFG', 'GFGLTAT', 'ENLLW', 'NLLWAG', 'QRAPGGS', 'GLVQHL', 'EYAATL', 'VTPNIM', 'PNIMLS', 'RMEHP', 'RYHSN', 'ASQPSPS', 'SSVVSPP', 'VVSPPAP', 'PLEPES', 'IVILLV', 'RGARLP', 'LPQNPV', 'PVVSVAV', 'SPINLE', 'LEFRL', 'EFRLL', 'FRLLQ', 'QTANRS', 'TANRSK', 'RSKAIC', 'WDPPGP', 'VHRNGS', 'SRTGTF', 'TFGVLM', 'MDASPR', 'DASPRE', 'PRERL', 'LEGDLE', 'EGDLEL', 'AVFTHV', 'VVVAVSV', 'VVAVSVT', 'TAAVLLS', 'RGIHAN', 'VAAALGVA', 'ALGVAEL', 'LLFLLG', 'LFLLGI', 'YFFLS', 'LVQGLH', 'VQGLHL', 'GVPAVLL', 'VPAVLLG', 'PDFCW', 'AGPIVLV', 'VLVIVM', 'STGQRE', 'KKTSVL', 'LTLRSS', 'LLAVNH', 'YLHAGL', 'LHAGLCG', 'HAGLCGL', 'AVLLLF', 'ARAAWT', 'ETRPAP', 'ESGLIR', 'GLIRIT', 'IRITLG', 'STVSSVS', 'TVSSVSS', 'RSGRAQ', 'GRAQDQ', 'LVRHGS', 'EHTER', 'AHAGPTD', 'LEEER', 'RSLSIP', 'SEDNGR', 'QRPLR', 'AHPKDV', 'WPALGE', 'GECEAAP', 'EAAPCAL', 'AAPCALQ', 'APCALQA', 'QAWGSE', 'NQPELA', 'QPELAL', 'LALTSGD', 'GDETSLG', 'KGILKN', 'RLQYP', 'RGTPEL', 'RAVPAAS', 'ASYGRI', 'YGRIY', 'YAGGGTGS', 'ASRYSS', 'RYSSR', 'LLLRR', 'KERLE', 'ERLEE', 'EEVPVP', 'PVLHPL', 'RPGSQE', 'PGSQER', 'RLDTAP', 'LPRRQ', 'SRDALD', 'DALDLGA', 'RNRDL', 'HPPLPL', 'SRPLDS', 'NSREGL', 'SREGLD', 'PSREAL', 'REDPAS', 'EDPASGP', 'PSTEQL', 'SGPHTTA', 'SEVPRS', 'PTHQTS', 'THQTSV', 'GNVVPSP', 'SPLPTR', 'VYKGVC', 'HGFITP', 'PGTKHE', 'GTKHET', 'AASRALS', 'IPEAGEG', 'TEGSEF', 'EFLRL', 'FLRLQ', 'QYKFS', 'LDTVIN', 'VINPDD', 'RVVVDS', 'NNPQAQ', 'SCGSSFS', 'LCGPRAG', 'GPRAGGVG', 'GVGILVL', 'GLLRLP', 'TSTPLR', 'RLRRA', 'ELARAL', 'RARAEA', 'EAQEAE', 'AQLLRA', 'RAWGSP', 'AWGSPR', 'WGSPRA', 'ASDPPLA', 'PDDDPD', 'PDAPAAQ', 'QLARAL', 'LDPAALA', 'AAQLVPA', 'AAPRPR', 'YDDGPT', 'DETPDV', 'ETPDVD', 'PDVDPE', 'LLRYL', 'LRYLL', 'LGRILT', 'GSSEPEA', 'APAPRR', 'SVDQDL', 'PEVPPE', 'PPENVL', 'LRVKR', 'RVKRL', 'ENPSPQ', 'PSPQAPA', 'AASRRL', 'RLMKE', 'FRNIQ', 'IQVDEA', 'QVDEAN', 'DEANLL', 'LIVPDN', 'PPYDKG', 'FRIEI', 'NIDEKG', 'DEKGQV', 'ENWKP', 'VIQSLI', 'VNDPQP', 'RADLAE', 'YSKDR', 'KDRKK', 'LPRAGR', 'RAGRPL', 'GRPLDL', 'DYTYD', 'YTYDL', 'DLGEED', 'EDAPEL', 'DPCKAAA', 'KAAAFLG', 'DIALDE', 'RPSIKA', 'AGNSSALG', 'SSALGGQG', 'LGGQGTSG', 'QGTSGQP', 'GTSGQPQ', 'SGQPQR', 'QRESR', 'WRGRP', 'RPRSR', 'PRSRR', 'RVWPD', 'VWPDGV', 'FGIVVH', 'ELGHVI', 'GFWHE', 'IQPGQE', 'FLKME', 'SLGETY', 'ETYDF', 'MHYAR', 'DTIVPK', 'QRTRL', 'RLSKGD', 'DIAQAR', 'PEYPNG', 'MHCVW', 'CVWRI', 'RISVTP', 'GEKIIL', 'VEVRDG', 'APLRGR', 'RFCGGK', 'LPEPIV', 'DSRLW', 'KGFFAV', 'YPDDY', 'VCIWR', 'VSEGFH', 'SEGFHV', 'FHVGLT', 'HVGLTF', 'IERHD', 'DSCAYD', 'VRDGHS', 'IGRYCG', 'PDDIKS', 'SRLWL', 'FFKEV', 'TLGSYK', 'LGSYKC', 'ELAPDK', 'PDKRR', 'DKRRC', 'ACGGFLT', 'NGSITSP', 'SPGWPK', 'PPNKNC', 'QYRIS', 'LQFDF', 'FFETE', 'CKYDF', 'KYDFV', 'YDFVE', 'ADSKLH', 'SKLHGK', 'KFCGSE', 'FCGSEK', 'EVITSQ', 'QYNNM', 'NTVSKK', 'CRSGFV', 'GCEHKV', 'HKVTST', 'TSTSGTI', 'STSGTIT', 'TSGTITS', 'TPGHRV', 'YDHLE', 'FCGSKK', 'PVLATGN', 'SVQRKG', 'FQASHS', 'HAQFGD', 'PGGVDCE', 'VAEEGY', 'FQTFE', 'TDCGYD', 'APRLGR', 'LGRYCG', 'GSGPPEE', 'SVRIVT', 'RIVTAD', 'MASPLPG', 'QSPLTQ', 'PLTQLP', 'LTQLPV', 'PVKKVP', 'VKKVPV', 'KKVPVV', 'VPVVRV', 'VVRVFG', 'ATPAGQK', 'PYDGYG', 'DGYGQQ', 'SQMAFS', 'SIDRAL', 'LGNPSST', 'GMPFYG', 'YGYHE', 'HFMKI', 'MKIYL', 'YLYNP', 'YQPHE', 'LFIDY', 'FIDYN', 'VKFRK', 'FRKAR', 'LFKHQ', 'FKHQL', 'KHQLSG', 'ELEVDA', 'EVDAVAA', 'DAVAADI', 'AVAADIL', 'DEKQR', 'NESSQI', 'SQPESQ', 'PESQDC', 'QDCRF', 'QKRLQ', 'SQEFSA', 'FSAELT', 'SPEMLP', 'PANMIE', 'ANMIEV', 'VEEALI', 'EAILNL', 'GSSPDES', 'DGYQGE', 'EKNRM', 'NPQNSD', 'PQNSDD', 'MELSVV', 'ELSVVM', 'RWDSD', 'NAHRSS', 'SSTEED', 'SSSEEE', 'NLSIPQ', 'LSIPQL', 'SIPQLD', 'ENSDNP', 'NSRAHS', 'SAKITF', 'LTEDLS', 'DLSQPN', 'QPNSTE', 'PNSTEK', 'STEKGR', 'KGRDNS', 'NSVTFT', 'SMKYSG', 'MKYSGS', 'TVHSDN', 'CESSVF', 'VFDYE', 'DYEED', 'YEEDI', 'IPSVTR', 'YSNMR', 'MRKIE', 'RKIEK', 'SCIHVN', 'ISETIL', 'NCDLLP', 'LESITD', 'GLNKLK', 'QEHKM', 'EHKME', 'HKMEK', 'AHYMF', 'PQKLSP', 'QKLSPV', 'KKLIGL', 'KLIGLQ', 'LIGLQE', 'TKSTET', 'DSCTHN', 'SCTHND', 'ASEKEN', 'SAKATHG', 'LEAEQS', 'FGLYGN', 'TEDSES', 'DSESSF', 'ESSFVT', 'SSFVTQ', 'VTQNSK', 'QNSKIS', 'PMEIGE', 'EIGENL', 'KSRKR', 'RRKMS', 'SKKLPP', 'KLPPVI', 'PPVIIK', 'KLGKID', 'KKLAPL', 'KLAPLK', 'FWPKV', 'SPATKY', 'PATKYP', 'LTPKKS', 'RRKSK', 'KQHRT', 'TNSENI', 'KKRTH', 'AVLSPPS', 'IAETED', 'AETEDC', 'ERSTSP', 'SPTDPR', 'LFKGPN', 'KGPNVY', 'ARAQVK', 'KARLAN', 'ANSSVVT', 'TTKLVD', 'DGKKKP', 'NEKSLS', 'IPADEK', 'KPHSEA', 'PHSEAE', 'NHQSVS', 'GAQALSK', 'QALSKQ', 'ALSKQK', 'VDHPLP', 'DHPLPP', 'PPAQPTG', 'AQPTGIS', 'QPTGISA', 'PTGISAQ', 'LSNCFS', 'SSFLES', 'ESKKSV', 'PSSRDD', 'VVYSSI', 'IGPGISK', 'GPGISKI', 'LSNHLS', 'CSGIISP', 'ISPKTE', 'CGSSMGK', 'KLNEY', 'NEYRS', 'VSEQSK', 'KTSETC', 'GNAASEE', 'ESQTPN', 'NCFVTS', 'CFVTSL', 'PIKQIA', 'LAVLKE', 'AVLKEL', 'LQKRQ', 'TNVVQD', 'TSTHQP', 'PDKNIS', 'DKNISV', 'SVSNEH', 'NKRTR', 'PRRLK', 'RRLKV', 'VDPLNL', 'LNLQTS', 'FSDPGF', 'SLSPEH', 'EHNYN', 'NYNFD', 'YNFDI', 'QFVPAD', 'FVPADQ', 'QDLFPG', 'DLFPGQ', 'KSELLS', 'SDSSPW', 'KVAMDN', 'NENHR', 'CVQQAE', 'QAENCL', 'LNRSSV', 'VFLSLP', 'KEADQS', 'ADQSLH', 'NTSFTT', 'TSFTTI', 'GELVDAA', 'NNDVLT', 'KPLMSP', 'MSPPSR', 'SPPSRE', 'ATLLDH', 'HDLSEA', 'SDVPEK', 'EGDFSL', 'GDFSLE', 'DFSLEG', 'SLEGLR', 'QNPRPG', 'VVNKES', 'NKESSN', 'IVIMPC', 'PCKYAP', 'CKYAPS', 'APSRQL', 'SPSLNPG', 'NPGDTCA', 'SPQVDK', 'SAHTKE', 'EVSKSQ', 'QIALQT', 'LAAASAAV', 'SPDSPGI', 'PGIPPW', 'IKDGIQ', 'EPQVIS', 'PLCLHS', 'LCLHST', 'HSTPVM', 'PEATGLS', 'EATGLSP', 'TGLSPLS', 'GLSPLSV', 'LSVEPK', 'VLLTTQ', 'LLTTQV', 'VSIQNL', 'EAKALH', 'AKALHE', 'LISVEL', 'PLPDTE', 'HQDIR', 'SGITGLE', 'TYAADE', 'LLQRAA', 'AAERDD', 'DDYGSD', 'YGSDTM', 'SDTMSE', 'SEINIV', 'INIVGR', 'NIVGRI', 'GRITLN', 'NYTFE', 'YTFEN', 'FTFRV', 'LSDWF', 'SDWFD', 'TDLYR', 'WKMVD', 'VSRVRG', 'GNLQML', 'ARLFGI', 'IAKPMN', 'TPSIQQ', 'RSQMR', 'RAPQCV', 'QCVPLI', 'CVPLIM', 'CFSTCL', 'LGHVEN', 'GHVENL', 'HVENLG', 'TSLRVP', 'SLRVPP', 'QIRHD', 'VTVSPNG', 'TVSPNGV', 'FVKPSV', 'VRKGVL', 'RKGVLP', 'KGVLPR', 'GVLPRM', 'RMLEE', 'QLGLKL', 'LKLIAN', 'TAANFSG', 'GRMPCI', 'DTKKW', 'VYGDTD', 'FVLLKG', 'IAEAVTA', 'AVTATNP', 'LPCVLQ', 'EPVFDA', 'PVFDAK', 'GIETVR', 'DSCPAVS', 'FETRD', 'MKLVEG', 'KLVEGK', 'QDFIF', 'AKEYR', 'FSYRP', 'YVIIY', 'AEVLQD', 'VLQDPT', 'LPPLAR', 'PPLARI', 'GTISQY', 'TTLHCP', 'TLHCPV', 'LHCPVC', 'HCPVCD', 'DLTQHG', 'LTQHGI', 'AIILNQ', 'LERKQ', 'PYLRQ', 'PQRAGAA', 'EAEKPV', 'AEKPVK', 'PVKTKT', 'VKTKTV', 'SRSAEK', 'SAEKRS', 'DLPTKP', 'PTKPTK', 'QTARKA', 'AISIRL', 'PKETVP', 'KETVPT', 'EMPPEA', 'MPPEAK', 'PPEAKM', 'NIGRDT', 'RDTPTS', 'KGKHGF', 'HGFSDN', 'GFSDNQ', 'HLGNVH', 'GNVHDQ', 'MVKLAN', 'RICNAV', 'CNAVSSS', 'ILKVSN', 'DNPGRI', 'HGKLDT', 'DTKQSV', 'TKQSVD', 'DWNKL', 'WNKLL', 'EGLAETG', 'IERFL', 'FHQQL', 'LAIKRA', 'KRAVGH', 'YRLNT', 'TKAASAE', 'SAEGKEG', 'EPIPIC', 'PICSFC', 'FCLGTK', 'LGTKEQ', 'GTKEQN', 'KEQNR', 'EQNRE', 'QNREK', 'NREKQ', 'PEELVS', 'SCADCGN', 'CADCGNS', 'CGNSGHP', 'TVRVKA', 'KALRW', 'ALRWQ', 'WQCIE', 'ECKTCS', 'CSSCRD', 'NADNML', 'QKKAAQ', 'PIGRPK', 'RPKNR', 'RLKKQ', 'KQNTVS', 'QNTVSK', 'PGRGRK', 'RGRKR', 'SSEEGY', 'DGLDFC', 'NKKTKG', 'KKTKGL', 'RGEVVD', 'WESKQ', 'ENEER', 'RCPSVI', 'SSPYPQ', 'EYSRL', 'YLCEF', 'FCLKY', 'QQHMK', 'QHMKK', 'KNNISV', 'TLYYD', 'LYYDV', 'YFSKE', 'CQQKY', 'SPEKPL', 'PEKPLS', 'KPLSDL', 'RLSYM', 'LYHQN', 'YHQND', 'SIKKLS', 'VCPQDI', 'CPQDIT', 'ITSTLH', 'LHHLR', 'HLRML', 'VDVDPE', 'DVDPEC', 'VDPECL', 'LRWTP', 'VVSEDE', 'QEREL', 'REKKD', 'SLIETD', 'SSRLSK', 'GRCGRK', 'FGDKDS', 'KSPETL', 'SPETLK', 'CRLPEG', 'GNDRLP', 'CYTDGD', 'FFRGF', 'SESSEE', 'PESPRS', 'PPILTK', 'PILTKP', 'ILTKPT', 'LTKPTL', 'TTEVLD', 'HCLSSQ', 'RCQSSS', 'DTPVLK', 'KGWPKG', 'GWPKGK', 'PKGKSR', 'GKSRKP', 'PGFKLN', 'EIIAASA', 'IIAASAQ', 'PIKPGR', 'LEERK', 'KEEMH', 'IRAMSP', 'RAMSPL', 'PEAEPK', 'AEPKEP', 'TEANQN', 'LDSLKT', 'PEEQPA', 'DKEEPG', 'KEEPGI', 'EEPGIQ', 'DEAEPD', 'QPSHEA', 'GSEEDH', 'VQSLTQ', 'VEDCHA', 'SEHNSP', 'PSQSVR', 'QSVRSV', 'LESGYT', 'SGYTQI', 'GYTQIS', 'TQISPE', 'NMETSP', 'VSDHSQ', 'SQQVVD', 'QQVVDS', 'ENYEN', 'YDSTMG', 'QSSCSY', 'SYGGLSS', 'YGGLSSS', 'SSSSSLT', 'CVVTQQ', 'VTQQMA', 'SCSMLQ', 'NCNIKS', 'IKSPQT', 'QTCVVE', 'PPSNQQ', 'PSNQQP', 'SNQQPP', 'PPQQQA', 'AAPQPPP', 'QQPPLS', 'QPPLSQ', 'LSQCSM', 'SQCSMN', 'GNISIY', 'YERIP', 'IPGDFGA', 'GAGSYSQ', 'SYSQPS', 'TFSLAK', 'DPHAMP', 'VSLSNTG', 'LSNTGLA', 'TGLAQLA', 'GLAQLAP', 'AGTPQAQ', 'GTPQAQA', 'SATNIGI', 'LQGQMP', 'MPVKGH', 'SIRSKS', 'YGRSPP', 'SPPAVAM', 'VAMQAGP', 'VNLMPT', 'NLMPTP', 'PAYNVN', 'AYNVNS', 'NVNSMN', 'MNSYR', 'NSYRM', 'YPMQM', 'MQMQM', 'NPHGNM', 'HHSYM', 'AAGVPKQ', 'VPKQSL']
print([x for x in s if 'SRM' in x])

['KSRMGP', 'LYSRM']
